# Spherical t-designs

In [ ]:
#@title Verification code

import numpy as np
import numba
from scipy.special import gegenbauer, binom

njit = numba.njit



# Memoization cache for expensive calculations
_gegenbauer_cache = {}
_dim_k_cache = {}


def get_dim_k(k: int, d: int) -> float:
  """Computes the dimension of the space of spherical harmonics of degree k on S^d."""
  if (k, d) in _dim_k_cache:
    return _dim_k_cache[(k, d)]
  # This formula for dimension of harmonic polynomials in d+1 variables is valid for k>=0
  # scipy's binom handles k-2 < 0 correctly (returns 0)
  dim = binom(d + k, k) - binom(d + k - 2, k - 2)
  _dim_k_cache[(k, d)] = dim
  return dim


def get_gegenbauer_poly(k: int, d: int):
  """Returns a memoized Gegenbauer polynomial object."""
  lam = (d - 1) / 2.0
  if (k, lam) in _gegenbauer_cache:
    return _gegenbauer_cache[(k, lam)]
  poly = gegenbauer(k, lam)
  _gegenbauer_cache[(k, lam)] = poly
  return poly


def calculate_design_score(points: np.ndarray, d: int, t: int, N: int) -> float:
  """Calculates the score for a potential spherical design."""
  # --- Safety checks ---
  if not isinstance(points, np.ndarray):
    points = np.array(points, dtype=np.float64)

  if points.ndim != 2 or points.shape != (N, d + 1):
    return -1_000_000.0

  if np.any(np.isnan(points)) or np.any(np.isinf(points)):
    return -1_000_000.0

  # --- Normalization ---
  norms = np.linalg.norm(points, axis=1, keepdims=True)
  # Handle zero-norm vectors
  if np.any(norms < 1e-9):
    return -1_000_000.0
  points_normalized = points / norms

  # --- Score Calculation ---
  if d == 1:
    # Special case for the circle S^1 using Fourier series condition
    # Convert points (x, y) to complex numbers z
    z = points_normalized[:, 0] + 1j * points_normalized[:, 1]
    total_error = 0.0
    # z_powers will be z^k in the k-th iteration (1-based index)
    z_powers = z.copy()
    for k in range(1, t + 1):
      # Sum of z_i^k over all points i
      sum_z_powers = np.sum(z_powers)
      # Add squared magnitude |sum(z_i^k)|^2 to error
      total_error += sum_z_powers.real**2 + sum_z_powers.imag**2
      # Update z_powers for next iteration: z^(k+1) = z^k * z
      if k < t:
        z_powers *= z
    return -total_error

  # General case for d >= 2
  dot_products = np.clip(points_normalized @ points_normalized.T, -1.0, 1.0)

  total_error = 0.0

  for k in range(1, t + 1):
    dim_k = get_dim_k(k, d)
    if abs(dim_k) < 1e-9:
      continue

    gegen_poly = get_gegenbauer_poly(k, d)

    # Denominator C_k(1), which is C_n^(\alpha)(1) = \binom{n+2\alpha-1}{n}
    denom = binom(k + d - 2, k)
    if abs(denom) < 1e-9:
      continue

    # Evaluate polynomial on all dot products
    poly_vals = gegen_poly(dot_products)

    # Sum over all pairs (i, j)
    error_k = np.sum(poly_vals)

    total_error += dim_k * error_k / denom

  return -total_error


In [ ]:
#@title Data and verification

spherical_design_215120 = np.array([[-6.797361715228318e-01, -3.327703556996885e-01, 6.536226950550188e-01], [ 9.801860614519381e-01, 1.554125061541841e-01, -1.228097629108250e-01], [-3.823572101009497e-01, 8.762371879340353e-01, -2.932769243654745e-01], [ 3.500258116703874e-01, 4.693794618537001e-01, -8.106570495310685e-01], [-9.252524737951290e-01, 3.114818462886376e-01, -2.165338753373607e-01], [-8.028751796956476e-01, 5.942166758497915e-01, -4.793733378803254e-02], [ 9.411767170248629e-01, -1.135741274099919e-01, -3.182566651515096e-01], [ 5.455875165157450e-01, 6.106872986159895e-01, -5.739296865743110e-01], [-6.412871504747886e-01, 1.360826579554759e-01, 7.551372728449436e-01], [-2.372972585702130e-01, 8.094674259027496e-01, 5.370777387654770e-01], [-3.947825317577580e-01, 2.399523201360250e-01, 8.868876122036395e-01], [ 9.113700671353424e-01, 5.530084675286726e-02, 4.078558778271283e-01], [-5.397254725893667e-01, 3.668417370670187e-01, -7.577094127591639e-01], [ 8.016751181740949e-01, -1.644484048849244e-01, 5.746944640688258e-01], [ 9.247921934476934e-01, -3.526297456645481e-01, -1.428693858416152e-01], [ 6.298850910467809e-01, 7.707875605024817e-01, -9.555788115912547e-02], [ 5.753135085302468e-01, -8.121852101855198e-01, -9.679644238566061e-02], [ 5.084524342052282e-01, 1.572836913939574e-01, -8.466037813359142e-01], [-6.094915645826794e-01, 3.426643934788353e-02, -7.920516674036961e-01], [-2.926482842293009e-01, 3.191284227462997e-01, -9.013956021265609e-01], [ 8.872812948007908e-01, 4.610629898154560e-01, -1.236217291032897e-02], [ 3.606370906222492e-01, 6.205956366785359e-01, 6.962772038513699e-01], [-2.386815227516121e-04, -5.904344860629173e-01, -8.070855349334105e-01], [ 2.097949552610305e-01, 9.170896438783542e-01, 3.390171999148355e-01], [ 5.263130313155542e-01, -6.146006873538451e-01, 5.875887917341629e-01], [-2.532145881445451e-01, 7.720523582024891e-01, -5.829387005035468e-01], [-4.766962534546765e-01, -3.633399212557076e-01, -8.004653543809196e-01], [ 7.982824780514204e-02, -9.646719759466557e-01, 2.510685756432607e-01], [-1.721338781703552e-01, -6.932396864262174e-01, 6.998490302556066e-01], [-9.206723701316694e-01, -3.017944348124743e-01, -2.475530367261804e-01], [-1.100620830473755e-01, 2.331048543041246e-01, 9.662031177630951e-01], [-7.237506489439083e-01, -2.431994720182243e-01, -6.457855797115081e-01], [ 1.691792369652005e-01, 3.482147848907411e-01, 9.220221523170511e-01], [-6.375965030959236e-01, 7.230487578409648e-01, -2.658405405962148e-01], [-3.249827959289296e-01, 9.450224998290604e-01, 3.631331941656037e-02], [-9.238473914609822e-02, -8.027955968882853e-01, -5.890537238569068e-01], [-8.513121953522199e-02, 5.745802759937275e-02, -9.947116418967035e-01], [ 7.308387946766719e-01, 4.820146839222374e-01, 4.832561439639386e-01], [ 7.542680675658688e-02, 8.706689490997980e-01, -4.860518263477237e-01], [ 7.496716868220968e-01, -5.847703488261156e-01, -3.098967587941103e-01], [-4.939773254262090e-01, 7.657996239188646e-01, 4.117491201818101e-01], [-9.945653849995024e-01, 5.388534520486815e-02, -8.908459200638333e-02], [-4.522250444900823e-01, -8.768969268503268e-01, -1.629241811898884e-01], [-5.556166432290568e-01, 5.157976600818351e-01, 6.521065247495831e-01], [-8.250561012310679e-01, -4.035484195363219e-02, 5.636079457852659e-01], [-7.371846180314873e-02, -9.969841836527431e-01, -2.425130791629632e-02], [ 3.139040818227862e-01, 9.368371667063761e-01, -1.542736221541560e-01], [-2.931028949151613e-01, 5.538719468024826e-01, 7.793051774097101e-01], [-5.830203880741195e-02, -3.817517115064857e-01, 9.224242532765321e-01], [ 2.413331483859359e-01, 1.039858696224139e-01, -9.648550411377878e-01], [ 4.090532678759426e-01, 8.166764497878966e-01, -4.070810734999228e-01], [-7.443020454173138e-01, -5.941195181215128e-01, 3.050187918385755e-01], [-5.078253185037975e-01, -7.228716411589976e-01, -4.685830089691837e-01], [-7.770682622262217e-01, 3.448146194523967e-01, 5.265622413852123e-01], [-8.711077171255694e-01, -3.176067170262966e-01, 3.745628364694724e-01], [ 1.396843855354699e-01, -9.517559241336087e-01, -2.732195697862046e-01], [-3.596826910508459e-01, -5.225191736985900e-02, -9.316104866786307e-01], [-7.323177894457898e-01, -6.508187309485975e-01, -2.003637559732016e-01], [ 7.749723898223471e-01, -4.512187827763033e-01, 4.425148642508082e-01], [ 1.542402338790827e-01, -5.950273647087267e-01, 7.887663694026982e-01], [ 6.797361715228318e-01, 3.327703556996885e-01, -6.536226950550188e-01], [-9.801860614519381e-01, -1.554125061541841e-01, 1.228097629108250e-01], [ 3.823572101009497e-01, -8.762371879340353e-01, 2.932769243654745e-01], [-3.500258116703874e-01, -4.693794618537001e-01, 8.106570495310685e-01], [ 9.252524737951290e-01, -3.114818462886376e-01, 2.165338753373607e-01], [ 8.028751796956476e-01, -5.942166758497915e-01, 4.793733378803254e-02], [-9.411767170248629e-01, 1.135741274099919e-01, 3.182566651515096e-01], [-5.455875165157450e-01, -6.106872986159895e-01, 5.739296865743110e-01], [ 6.412871504747886e-01, -1.360826579554759e-01, -7.551372728449436e-01], [ 2.372972585702130e-01, -8.094674259027496e-01, -5.370777387654770e-01], [ 3.947825317577580e-01, -2.399523201360250e-01, -8.868876122036395e-01], [-9.113700671353424e-01, -5.530084675286726e-02, -4.078558778271283e-01], [ 5.397254725893667e-01, -3.668417370670187e-01, 7.577094127591639e-01], [-8.016751181740949e-01, 1.644484048849244e-01, -5.746944640688258e-01], [-9.247921934476934e-01, 3.526297456645481e-01, 1.428693858416152e-01], [-6.298850910467809e-01, -7.707875605024817e-01, 9.555788115912547e-02], [-5.753135085302468e-01, 8.121852101855198e-01, 9.679644238566061e-02], [-5.084524342052282e-01, -1.572836913939574e-01, 8.466037813359142e-01], [ 6.094915645826794e-01, -3.426643934788353e-02, 7.920516674036961e-01], [ 2.926482842293009e-01, -3.191284227462997e-01, 9.013956021265609e-01], [-8.872812948007908e-01, -4.610629898154560e-01, 1.236217291032897e-02], [-3.606370906222492e-01, -6.205956366785359e-01, -6.962772038513699e-01], [ 2.386815227516121e-04, 5.904344860629173e-01, 8.070855349334105e-01], [-2.097949552610305e-01, -9.170896438783542e-01, -3.390171999148355e-01], [-5.263130313155542e-01, 6.146006873538451e-01, -5.875887917341629e-01], [ 2.532145881445451e-01, -7.720523582024891e-01, 5.829387005035468e-01], [ 4.766962534546765e-01, 3.633399212557076e-01, 8.004653543809196e-01], [-7.982824780514204e-02, 9.646719759466557e-01, -2.510685756432607e-01], [ 1.721338781703552e-01, 6.932396864262174e-01, -6.998490302556066e-01], [ 9.206723701316694e-01, 3.017944348124743e-01, 2.475530367261804e-01], [ 1.100620830473755e-01, -2.331048543041246e-01, -9.662031177630951e-01], [ 7.237506489439083e-01, 2.431994720182243e-01, 6.457855797115081e-01], [-1.691792369652005e-01, -3.482147848907411e-01, -9.220221523170511e-01], [ 6.375965030959236e-01, -7.230487578409648e-01, 2.658405405962148e-01], [ 3.249827959289296e-01, -9.450224998290604e-01, -3.631331941656037e-02], [ 9.238473914609822e-02, 8.027955968882853e-01, 5.890537238569068e-01], [ 8.513121953522199e-02, -5.745802759937275e-02, 9.947116418967035e-01], [-7.308387946766719e-01, -4.820146839222374e-01, -4.832561439639386e-01], [-7.542680675658688e-02, -8.706689490997980e-01, 4.860518263477237e-01], [-7.496716868220968e-01, 5.847703488261156e-01, 3.098967587941103e-01], [ 4.939773254262090e-01, -7.657996239188646e-01, -4.117491201818101e-01], [ 9.945653849995024e-01, -5.388534520486815e-02, 8.908459200638333e-02], [ 4.522250444900823e-01, 8.768969268503268e-01, 1.629241811898884e-01], [ 5.556166432290568e-01, -5.157976600818351e-01, -6.521065247495831e-01], [ 8.250561012310679e-01, 4.035484195363219e-02, -5.636079457852659e-01], [ 7.371846180314873e-02, 9.969841836527431e-01, 2.425130791629632e-02], [-3.139040818227862e-01, -9.368371667063761e-01, 1.542736221541560e-01], [ 2.931028949151613e-01, -5.538719468024826e-01, -7.793051774097101e-01], [ 5.830203880741195e-02, 3.817517115064857e-01, -9.224242532765321e-01], [-2.413331483859359e-01, -1.039858696224139e-01, 9.648550411377878e-01], [-4.090532678759426e-01, -8.166764497878966e-01, 4.070810734999228e-01], [ 7.443020454173138e-01, 5.941195181215128e-01, -3.050187918385755e-01], [ 5.078253185037975e-01, 7.228716411589976e-01, 4.685830089691837e-01], [ 7.770682622262217e-01, -3.448146194523967e-01, -5.265622413852123e-01], [ 8.711077171255694e-01, 3.176067170262966e-01, -3.745628364694724e-01], [-1.396843855354699e-01, 9.517559241336087e-01, 2.732195697862046e-01], [ 3.596826910508459e-01, 5.225191736985900e-02, 9.316104866786307e-01], [ 7.323177894457898e-01, 6.508187309485975e-01, 2.003637559732016e-01], [-7.749723898223471e-01, 4.512187827763033e-01, -4.425148642508082e-01], [-1.542402338790827e-01, 5.950273647087267e-01, -7.887663694026982e-01]])

if calculate_design_score(spherical_design_215120, 2, 15, 120) < 1e-8:
  print("Correct numerical construction of a (2, 15, 120) design")


spherical_design_215124 = np.array([[ 0.968016503923547, 0.094384568368492, -0.232455590136108], [ 0.213420935709447, 0.42838327405718 , -0.87803147706045 ], [ 0.450298713202835, -0.87871107851976 , 0.158423197084618], [-0.287913424875414, -0.827452950562545, 0.482107326621208], [-0.071178762952695, 0.782839646241402, -0.61813887758105 ], [-0.753801242661322, 0.424777335776776, -0.501346089615378], [ 0.409778889053801, 0.909188567458444, 0.073874291121438], [-0.657775972422586, -0.559791201811746, -0.503948985987354], [ 0.760702321972088, -0.635029740447818, 0.134421747098638], [-0.833278484603953, -0.146550721053752, -0.533075841935053], [ 0.678529636830423, 0.730976461475484, -0.072601272106999], [-0.700323485749332, 0.180458660284079, 0.690638608272359], [ 0.634116076593457, -0.668797096609384, -0.388081492695258], [ 0.50049144869314 , -0.377171461475895, 0.779262470823008], [-0.144728454254037, -0.958635025274354, -0.245097047812647], [-0.226808491427417, -0.116534061454924, 0.966942459889542], [ 0.012342930357556, 0.732935352434344, 0.680186313609832], [ 0.421599443042752, -0.292544346076687, -0.858295820335043], [-0.864093632648976, 0.473127652073621, 0.171733569399811], [ 0.427035105837154, -0.635262066930372, 0.643492909597303], [ 0.27803057335025 , -0.873185776169465, 0.400306883000856], [-0.513832102721341, -0.787547763505291, -0.340213301343114], [ 0.079766688593259, 0.009819475199288, 0.996765194666064], [-0.15423150428061 , 0.565434782579145, -0.81024449997332 ], [-0.742613371252577, 0.175347466805235, -0.646357986507376], [ 0.761595387387436, 0.625088329215931, 0.17098843992568 ], [-0.078330773308647, 0.907150728006402, 0.413451141769284], [-0.285804091227766, -0.798120472108104, -0.530395827132348], [ 0.904362416989269, -0.091356410873374, -0.416872432441465], [-0.149203018953398, 0.528670012071466, 0.835611439289547], [ 0.352502894678271, 0.579309779992524, 0.734943459082706], [-0.610247240681979, -0.070471721915371, -0.789070365461987], [-0.358221891139747, -0.911505072050656, 0.202078154025097], [-0.227021247999347, -0.664854472322536, 0.711638871612239], [-0.381930721185112, 0.160806350241914, -0.910093534718764], [-0.744201761844958, -0.134183709840876, 0.654338192130186], [ 0.529091643475125, -0.511984802775284, -0.676707909316822], [-0.092329648079961, 0.31884371886853 , -0.943299485329827], [ 0.339816087104421, 0.187544345665573, 0.921603030243452], [ 0.594232900226178, 0.372145521724395, 0.713018212214291], [ 0.791490493846378, -0.392128613960184, -0.468804808277904], [ 0.612238208591679, 0.700806456297158, -0.36610747978247 ], [-0.900966928671856, 0.373984581155742, -0.21998665072537 ], [-0.011764381081604, -0.949463319041119, 0.313657464654022], [ 0.515983579227231, 0.35245352599622 , -0.780728799251499], [ 0.599632788527533, -0.688771780410048, 0.407472641331251], [-0.135499231562571, 0.228057082597533, 0.964173182225608], [ 0.373693231955409, -0.871339562018692, -0.317994867964555], [ 0.871097800280595, 0.348854034768772, 0.345672510870993], [ 0.48676041882986 , 0.029522013923396, -0.873036508603441], [-0.855679598553485, -0.466147780898432, 0.224763589094885], [ 0.016506437467623, 0.999275597775865, -0.034290191189536], [-0.328896427141236, 0.71931607221479 , 0.611891762052094], [-0.637345261638735, 0.764179310995438, 0.099100949103372], [-0.936924381598185, -0.347623414203623, -0.036478282090521], [ 0.819703449637646, 0.332133301929796, -0.466662323743147], [ 0.993938446281733, 0.046959566240547, 0.099404044895336], [ 0.933328976955715, -0.090059477636959, 0.347543250923616], [ 0.968663603531549, -0.236066859774847, -0.077222152969972], [ 0.264934717021541, -0.962960093477765, -0.050174237273819], [-0.158446205661142, -0.418667423021456, -0.89421048350607 ], [-0.567992571009645, -0.563971932729501, 0.599433147541245], [-0.968016503923547, -0.094384568368492, 0.232455590136108], [-0.213420935709447, -0.42838327405718 , 0.87803147706045 ], [-0.450298713202835, 0.87871107851976 , -0.158423197084618], [ 0.287913424875414, 0.827452950562545, -0.482107326621208], [ 0.071178762952695, -0.782839646241402, 0.61813887758105 ], [ 0.753801242661322, -0.424777335776776, 0.501346089615378], [-0.409778889053801, -0.909188567458444, -0.073874291121438], [ 0.657775972422586, 0.559791201811746, 0.503948985987354], [-0.760702321972088, 0.635029740447818, -0.134421747098638], [ 0.833278484603953, 0.146550721053752, 0.533075841935053], [-0.678529636830423, -0.730976461475484, 0.072601272106999], [ 0.700323485749332, -0.180458660284079, -0.690638608272359], [-0.634116076593457, 0.668797096609384, 0.388081492695258], [-0.50049144869314 , 0.377171461475895, -0.779262470823008], [ 0.144728454254037, 0.958635025274354, 0.245097047812647], [ 0.226808491427417, 0.116534061454924, -0.966942459889542], [-0.012342930357556, -0.732935352434344, -0.680186313609832], [-0.421599443042752, 0.292544346076687, 0.858295820335043], [ 0.864093632648976, -0.473127652073621, -0.171733569399811], [-0.427035105837154, 0.635262066930372, -0.643492909597303], [-0.27803057335025 , 0.873185776169465, -0.400306883000856], [ 0.513832102721341, 0.787547763505291, 0.340213301343114], [-0.079766688593259, -0.009819475199288, -0.996765194666064], [ 0.15423150428061 , -0.565434782579145, 0.81024449997332 ], [ 0.742613371252577, -0.175347466805235, 0.646357986507376], [-0.761595387387436, -0.625088329215931, -0.17098843992568 ], [ 0.078330773308647, -0.907150728006402, -0.413451141769284], [ 0.285804091227766, 0.798120472108104, 0.530395827132348], [-0.904362416989269, 0.091356410873374, 0.416872432441465], [ 0.149203018953398, -0.528670012071466, -0.835611439289547], [-0.352502894678271, -0.579309779992524, -0.734943459082706], [ 0.610247240681979, 0.070471721915371, 0.789070365461987], [ 0.358221891139747, 0.911505072050656, -0.202078154025097], [ 0.227021247999347, 0.664854472322536, -0.711638871612239], [ 0.381930721185112, -0.160806350241914, 0.910093534718764], [ 0.744201761844958, 0.134183709840876, -0.654338192130186], [-0.529091643475125, 0.511984802775284, 0.676707909316822], [ 0.092329648079961, -0.31884371886853 , 0.943299485329827], [-0.339816087104421, -0.187544345665573, -0.921603030243452], [-0.594232900226178, -0.372145521724395, -0.713018212214291], [-0.791490493846378, 0.392128613960184, 0.468804808277904], [-0.612238208591679, -0.700806456297158, 0.36610747978247 ], [ 0.900966928671856, -0.373984581155742, 0.21998665072537 ], [ 0.011764381081604, 0.949463319041119, -0.313657464654022], [-0.515983579227231, -0.35245352599622 , 0.780728799251499], [-0.599632788527533, 0.688771780410048, -0.407472641331251], [ 0.135499231562571, -0.228057082597533, -0.964173182225608], [-0.373693231955409, 0.871339562018692, 0.317994867964555], [-0.871097800280595, -0.348854034768772, -0.345672510870993], [-0.48676041882986 , -0.029522013923396, 0.873036508603441], [ 0.855679598553485, 0.466147780898432, -0.224763589094885], [-0.016506437467623, -0.999275597775865, 0.034290191189536], [ 0.328896427141236, -0.71931607221479 , -0.611891762052094], [ 0.637345261638735, -0.764179310995438, -0.099100949103372], [ 0.936924381598185, 0.347623414203623, 0.036478282090521], [-0.819703449637646, -0.332133301929796, 0.466662323743147], [-0.993938446281733, -0.046959566240547, -0.099404044895336], [-0.933328976955715, 0.090059477636959, -0.347543250923616], [-0.968663603531549, 0.236066859774847, 0.077222152969972], [-0.264934717021541, 0.962960093477765, 0.050174237273819], [ 0.158446205661142, 0.418667423021456, 0.89421048350607 ], [ 0.567992571009645, 0.563971932729501, -0.599433147541245]])

if calculate_design_score(spherical_design_215124, 2, 15, 124) < 1e-8:
  print("Correct numerical construction of a (2, 15, 124) design")


spherical_design_215126 = np.array([[-0.496857493277738, 0.774850235157378, 0.390819324561003], [-0.303650871752059, 0.101224653803253, -0.947391005628946], [-0.59121750427765 , 0.033264797923843, -0.805825859509853], [-0.938492420358856, -0.33899756923135 , -0.065670579289449], [-0.813355774122845, 0.050013300671519, -0.57961284876801 ], [ 0.963272500359698, -0.034946336904818, 0.266242077042137], [ 0.957627866656405, -0.274029579210967, -0.08863779510211 ], [ 0.34782328968558 , 0.487378453943386, 0.800925216099515], [ 0.748214663706164, -0.615486346615033, -0.247692095444227], [ 0.481188199191891, 0.284975554456564, -0.829003528533286], [-0.179393246868758, -0.473187296885297, 0.862503243497826], [ 0.801443150637979, 0.520621692574238, 0.294350011918812], [-0.821736975652455, 0.569865434542461, 0.001315051058927], [ 0.564325449111976, -0.541828366027226, -0.622863395338683], [-0.78841923352489 , 0.404204254172761, 0.463696056826739], [ 0.066637440173488, 0.69885841961421 , 0.712149114232022], [-0.887147240552663, -0.235583855865157, -0.396824924161147], [-0.06033419192117 , 0.891936768194974, 0.448116711166996], [ 0.915740950055369, -0.332020836015783, 0.226231467402462], [ 0.568359917401545, -0.806237877618673, 0.164156903553421], [-0.14738478569107 , 0.986708512376302, -0.068440021558607], [ 0.070077811733079, 0.37911649123086 , 0.922691598736818], [-0.786198089132587, 0.339456723046191, -0.516392968409727], [ 0.420638610353555, 0.773970440900199, -0.473321155340199], [-0.998161386868251, -0.054855647959753, 0.025781847338124], [-0.611349476345443, -0.525906101448124, -0.591332892905331], [-0.453139233523616, 0.360308105409631, 0.815378994221553], [-0.845746190752205, -0.485560026321 , 0.22123481115612 ], [ 0.504574894140358, -0.857830221924728, -0.09762933245607 ], [-0.722261777156872, 0.613366998650072, -0.319560401528724], [ 0.932214626282317, 0.199684127099987, -0.301831310386507], [ 0.324136162619918, 0.745016648395666, 0.582997377091289], [-0.266362511951491, -0.938074216079323, 0.221512476745795], [ 0.31106903251301 , -0.900865867485838, 0.302781680110288], [ 0.021217933401874, -0.271972993773986, 0.962070938112034], [ 0.468706438366365, 0.876282290095214, 0.111550986994305], [-0.082401860004919, -0.093714813805892, -0.992183182250665], [-0.214150117399102, -0.918190693302317, -0.333265026594392], [ 0.092545265138853, 0.784120178093296, -0.61367004180367 ], [ 0.178212693439735, -0.538173685167656, -0.823777470249108], [ 0.311643477514179, -0.763179885341952, 0.566069611913866], [ 0.285445730128652, -0.74375943346684 , -0.604435803274761], [-0.74585456436383 , -0.665778712482681, -0.020968424417311], [ 0.239603363230566, 0.105694987582317, -0.96510040821076 ], [ 0.0139725074552 , 0.921633596614952, -0.38780959584054 ], [ 0.926777257890591, -0.162794366587421, -0.338499495515425], [ 0.306396611987113, -0.430110871496573, 0.849191235460704], [ 0.687733748075067, 0.575041266474785, -0.443113793070917], [-0.68608090460152 , -0.359579875080065, 0.632451820914894], [ 0.106931381417756, -0.62744778440649 , 0.771281373761533], [-0.566497875887596, -0.802599049050654, 0.186855353356089], [-0.425167744332446, -0.222720067063331, -0.877284538166817], [ 0.562058460164519, -0.348860518121375, 0.749924413693524], [-0.555990989504233, 0.011226364513189, 0.831112500405283], [-0.541670745355424, 0.620577990716185, -0.566988325333743], [-0.244761143785183, 0.189174265973412, 0.950949567320155], [ 0.828044390671499, 0.108826030256459, -0.549999438377973], [ 0.724806924033062, -0.162501561966667, -0.66951337942727 ], [ 0.21786980030853 , -0.952755171674393, -0.211637267420506], [-0.702132811923491, -0.26896733220456 , -0.659292111758645], [ 0.56529080158462 , 0.742898815571002, 0.358535436835774], [-0.387050273375052, -0.598469981484608, 0.701445484084195], [ 0.142900763092007, 0.988717373488411, 0.044914666534438], [ 0.496857493277738, -0.774850235157378, -0.390819324561003], [ 0.303650871752059, -0.101224653803253, 0.947391005628946], [ 0.59121750427765 , -0.033264797923843, 0.805825859509853], [ 0.938492420358856, 0.33899756923135 , 0.065670579289449], [ 0.813355774122845, -0.050013300671519, 0.57961284876801 ], [-0.963272500359698, 0.034946336904818, -0.266242077042137], [-0.957627866656405, 0.274029579210967, 0.08863779510211 ], [-0.34782328968558 , -0.487378453943386, -0.800925216099515], [-0.748214663706164, 0.615486346615033, 0.247692095444227], [-0.481188199191891, -0.284975554456564, 0.829003528533286], [ 0.179393246868758, 0.473187296885297, -0.862503243497826], [-0.801443150637979, -0.520621692574238, -0.294350011918812], [ 0.821736975652455, -0.569865434542461, -0.001315051058927], [-0.564325449111976, 0.541828366027226, 0.622863395338683], [ 0.78841923352489 , -0.404204254172761, -0.463696056826739], [-0.066637440173488, -0.69885841961421 , -0.712149114232022], [ 0.887147240552663, 0.235583855865157, 0.396824924161147], [ 0.06033419192117 , -0.891936768194974, -0.448116711166996], [-0.915740950055369, 0.332020836015783, -0.226231467402462], [-0.568359917401545, 0.806237877618673, -0.164156903553421], [ 0.14738478569107 , -0.986708512376302, 0.068440021558607], [-0.070077811733079, -0.37911649123086 , -0.922691598736818], [ 0.786198089132587, -0.339456723046191, 0.516392968409727], [-0.420638610353555, -0.773970440900199, 0.473321155340199], [ 0.998161386868251, 0.054855647959753, -0.025781847338124], [ 0.611349476345443, 0.525906101448124, 0.591332892905331], [ 0.453139233523616, -0.360308105409631, -0.815378994221553], [ 0.845746190752205, 0.485560026321 , -0.22123481115612 ], [-0.504574894140358, 0.857830221924728, 0.09762933245607 ], [ 0.722261777156872, -0.613366998650072, 0.319560401528724], [-0.932214626282317, -0.199684127099987, 0.301831310386507], [-0.324136162619918, -0.745016648395666, -0.582997377091289], [ 0.266362511951491, 0.938074216079323, -0.221512476745795], [-0.31106903251301 , 0.900865867485838, -0.302781680110288], [-0.021217933401874, 0.271972993773986, -0.962070938112034], [-0.468706438366365, -0.876282290095214, -0.111550986994305], [ 0.082401860004919, 0.093714813805892, 0.992183182250665], [ 0.214150117399102, 0.918190693302317, 0.333265026594392], [-0.092545265138853, -0.784120178093296, 0.61367004180367 ], [-0.178212693439735, 0.538173685167656, 0.823777470249108], [-0.311643477514179, 0.763179885341952, -0.566069611913866], [-0.285445730128652, 0.74375943346684 , 0.604435803274761], [ 0.74585456436383 , 0.665778712482681, 0.020968424417311], [-0.239603363230566, -0.105694987582317, 0.96510040821076 ], [-0.0139725074552 , -0.921633596614952, 0.38780959584054 ], [-0.926777257890591, 0.162794366587421, 0.338499495515425], [-0.306396611987113, 0.430110871496573, -0.849191235460704], [-0.687733748075067, -0.575041266474785, 0.443113793070917], [ 0.68608090460152 , 0.359579875080065, -0.632451820914894], [-0.106931381417756, 0.62744778440649 , -0.771281373761533], [ 0.566497875887596, 0.802599049050654, -0.186855353356089], [ 0.425167744332446, 0.222720067063331, 0.877284538166817], [-0.562058460164519, 0.348860518121375, -0.749924413693524], [ 0.555990989504233, -0.011226364513189, -0.831112500405283], [ 0.541670745355424, -0.620577990716185, 0.566988325333743], [ 0.244761143785183, -0.189174265973412, -0.950949567320155], [-0.828044390671499, -0.108826030256459, 0.549999438377973], [-0.724806924033062, 0.162501561966667, 0.66951337942727 ], [-0.21786980030853 , 0.952755171674393, 0.211637267420506], [ 0.702132811923491, 0.26896733220456 , 0.659292111758645], [-0.56529080158462 , -0.742898815571002, -0.358535436835774], [ 0.387050273375052, 0.598469981484608, -0.701445484084195], [-0.142900763092007, -0.988717373488411, -0.044914666534438]])

if calculate_design_score(spherical_design_215126, 2, 15, 126) < 1e-8:
  print("Correct numerical construction of a (2, 15, 126) design")


spherical_design_215128 = np.array([[ 0.122143391409144, 0.986522160559333, -0.108880754315985], [-0.01955124468593 , 0.367736939296741, -0.929724309840232], [-0.271501372502468, -0.702247428427949, 0.658130347268388], [-0.931583583317265, 0.272263648049302, -0.240882820567698], [ 0.219809963342569, -0.576658388368511, 0.786860014957915], [-0.161581511959057, 0.965449634498867, -0.204446614643113], [ 0.992238734262413, -0.003443193497122, 0.124299793434525], [ 0.189898196573066, 0.518156172953486, -0.833938160398296], [-0.493477025934003, -0.394103199792715, 0.77534707891916 ], [ 0.84960715651116 , 0.527029140815367, 0.0201981270527 ], [-0.644139547449607, -0.760459723060946, -0.082372647240968], [-0.470958724103185, 0.878948254945199, -0.075152147806231], [-0.418147911295086, -0.907334650371095, 0.043544879326052], [-0.676865462547018, 0.734820863036708, -0.043490744498096], [-0.308373005747337, 0.753357041517913, 0.580826357289104], [ 0.549729163337197, -0.713153608540679, 0.434982502638891], [-0.837434177064694, -0.453764708920345, 0.304633530692885], [ 0.807626729456024, -0.548629621337407, -0.216205005629702], [-0.709228946842877, -0.379276497873998, 0.594258899067215], [-0.031251486058254, -0.142224412168925, -0.989340973174747], [-0.08082024921065 , 0.897315042970433, 0.433928336222122], [ 0.741553006171717, -0.440093613203836, 0.506376096053992], [-0.867437635756741, 0.084590921579861, -0.490302278251922], [ 0.928917879372486, -0.364081219866645, -0.067501397930262], [-0.692561276057609, 0.190972892566262, -0.695620753866883], [-0.797783172714211, 0.55910669470261 , -0.2257027099367 ], [-0.862801202843017, 0.224368892095969, 0.453026141223959], [-0.809712563898555, -0.085063705472388, 0.58062873669509 ], [-0.174214223799147, -0.110897795162353, 0.978443193677686], [ 0.965971193441872, 0.256238182920879, 0.035237012556275], [ 0.550763516462884, 0.654633734495055, -0.517797472564801], [ 0.203642940511156, -0.972292912500043, -0.114786955191576], [-0.068338684443893, -0.890728595963125, 0.449368882480793], [-0.592256163968618, -0.492474509762564, -0.637731521469105], [-0.418518566475971, -0.150330527848846, 0.89568015603316 ], [ 0.320102784396358, 0.207017480799358, 0.924487950200102], [ 0.495415719548048, -0.249746589264123, -0.831979510550366], [ 0.113609227843547, 0.970504614880561, 0.212635217789356], [ 0.551364117784842, 0.071660771863485, 0.831181293939699], [-0.366069139190318, -0.880555703473282, -0.301023318719885], [ 0.362383403542346, 0.864148376235044, -0.349178826230058], [ 0.214415569169735, -0.135018556927875, 0.967365470224533], [-0.61882889568972 , -0.682248317775178, -0.389343075901422], [-0.499345302853782, 0.542213312308956, 0.67576548629892 ], [ 0.335438544243628, -0.84159537090231 , 0.42331810109127 ], [ 0.820169534911003, 0.464640471970205, 0.33381307016229 ], [ 0.723495920265648, -0.394068182901337, -0.566801482517294], [-0.245112987503829, 0.241623056755184, 0.93890250921021 ], [-0.358483713086821, -0.491144978075254, -0.793892963794833], [ 0.193527381268524, -0.549320499495144, -0.8128924538546 ], [-0.712895778298506, -0.226154982100128, -0.663802330031657], [-0.429978299781633, 0.265264782282802, -0.862990878861042], [ 0.66629652117581 , 0.705448183527144, -0.241643965840794], [ 0.893197357436405, 0.198484421179732, 0.403487812972794], [ 0.063960841672081, 0.754234203673848, 0.653482805237503], [-0.321384719590147, -0.75525156892362 , -0.571232815629821], [ 0.974795483783448, -0.093512598219767, -0.202556556965166], [ 0.626360549390441, -0.689671429522438, -0.363353521336704], [-0.067128522488634, -0.481496146087307, -0.873873688110338], [-0.936831292942885, -0.181277484232777, 0.299141441918697], [ 0.649746436470128, -0.028419065602705, -0.759619592299093], [-0.488149739579088, 0.528585324181658, -0.694483539623971], [-0.09654448531825 , 0.759941316800029, -0.642781733852846], [ 0.430043358505433, -0.858536102941368, -0.279246252887334], [-0.122143391409144, -0.986522160559333, 0.108880754315985], [ 0.01955124468593 , -0.367736939296741, 0.929724309840232], [ 0.271501372502468, 0.702247428427949, -0.658130347268388], [ 0.931583583317265, -0.272263648049302, 0.240882820567698], [-0.219809963342569, 0.576658388368511, -0.786860014957915], [ 0.161581511959057, -0.965449634498867, 0.204446614643113], [-0.992238734262413, 0.003443193497122, -0.124299793434525], [-0.189898196573066, -0.518156172953486, 0.833938160398296], [ 0.493477025934003, 0.394103199792715, -0.77534707891916 ], [-0.84960715651116 , -0.527029140815367, -0.0201981270527 ], [ 0.644139547449607, 0.760459723060946, 0.082372647240968], [ 0.470958724103185, -0.878948254945199, 0.075152147806231], [ 0.418147911295086, 0.907334650371095, -0.043544879326052], [ 0.676865462547018, -0.734820863036708, 0.043490744498096], [ 0.308373005747337, -0.753357041517913, -0.580826357289104], [-0.549729163337197, 0.713153608540679, -0.434982502638891], [ 0.837434177064694, 0.453764708920345, -0.304633530692885], [-0.807626729456024, 0.548629621337407, 0.216205005629702], [ 0.709228946842877, 0.379276497873998, -0.594258899067215], [ 0.031251486058254, 0.142224412168925, 0.989340973174747], [ 0.08082024921065 , -0.897315042970433, -0.433928336222122], [-0.741553006171717, 0.440093613203836, -0.506376096053992], [ 0.867437635756741, -0.084590921579861, 0.490302278251922], [-0.928917879372486, 0.364081219866645, 0.067501397930262], [ 0.692561276057609, -0.190972892566262, 0.695620753866883], [ 0.797783172714211, -0.55910669470261 , 0.2257027099367 ], [ 0.862801202843017, -0.224368892095969, -0.453026141223959], [ 0.809712563898555, 0.085063705472388, -0.58062873669509 ], [ 0.174214223799147, 0.110897795162353, -0.978443193677686], [-0.965971193441872, -0.256238182920879, -0.035237012556275], [-0.550763516462884, -0.654633734495055, 0.517797472564801], [-0.203642940511156, 0.972292912500043, 0.114786955191576], [ 0.068338684443893, 0.890728595963125, -0.449368882480793], [ 0.592256163968618, 0.492474509762564, 0.637731521469105], [ 0.418518566475971, 0.150330527848846, -0.89568015603316 ], [-0.320102784396358, -0.207017480799358, -0.924487950200102], [-0.495415719548048, 0.249746589264123, 0.831979510550366], [-0.113609227843547, -0.970504614880561, -0.212635217789356], [-0.551364117784842, -0.071660771863485, -0.831181293939699], [ 0.366069139190318, 0.880555703473282, 0.301023318719885], [-0.362383403542346, -0.864148376235044, 0.349178826230058], [-0.214415569169735, 0.135018556927875, -0.967365470224533], [ 0.61882889568972 , 0.682248317775178, 0.389343075901422], [ 0.499345302853782, -0.542213312308956, -0.67576548629892 ], [-0.335438544243628, 0.84159537090231 , -0.42331810109127 ], [-0.820169534911003, -0.464640471970205, -0.33381307016229 ], [-0.723495920265648, 0.394068182901337, 0.566801482517294], [ 0.245112987503829, -0.241623056755184, -0.93890250921021 ], [ 0.358483713086821, 0.491144978075254, 0.793892963794833], [-0.193527381268524, 0.549320499495144, 0.8128924538546 ], [ 0.712895778298506, 0.226154982100128, 0.663802330031657], [ 0.429978299781633, -0.265264782282802, 0.862990878861042], [-0.66629652117581 , -0.705448183527144, 0.241643965840794], [-0.893197357436405, -0.198484421179732, -0.403487812972794], [-0.063960841672081, -0.754234203673848, -0.653482805237503], [ 0.321384719590147, 0.75525156892362 , 0.571232815629821], [-0.974795483783448, 0.093512598219767, 0.202556556965166], [-0.626360549390441, 0.689671429522438, 0.363353521336704], [ 0.067128522488634, 0.481496146087307, 0.873873688110338], [ 0.936831292942885, 0.181277484232777, -0.299141441918697], [-0.649746436470128, 0.028419065602705, 0.759619592299093], [ 0.488149739579088, -0.528585324181658, 0.694483539623971], [ 0.09654448531825 , -0.759941316800029, 0.642781733852846], [-0.430043358505433, 0.858536102941368, 0.279246252887334]])

if calculate_design_score(spherical_design_215128, 2, 15, 128) < 1e-8:
  print("Correct numerical construction of a (2, 15, 128) design")


spherical_design_215130 = np.array([[-0.867267106883638, 0.055443243075175, 0.494746209803564], [-0.97087331565506 , 0.239478873167382, 0.007407715938823], [ 0.907276828613544, -0.297990770269208, -0.296732635709849], [ 0.9794480341064 , 0.18679073257907 , 0.076097113661971], [ 0.328865685812915, 0.8820568850465 , 0.337376662258766], [-0.089103155106453, 0.75616258665444 , -0.648289110115342], [-0.227921439135609, -0.162165392298572, -0.960080310767283], [ 0.472125872891334, 0.261410342843101, -0.841879915903241], [ 0.571579860449469, -0.640183203825906, 0.513285426120559], [ 0.957193178150123, -0.155452473842011, 0.244163363507438], [-0.630631543724712, -0.72532400195024 , -0.276059682413559], [ 0.113819063857892, 0.970240839104064, -0.213723968794223], [-0.815108107019787, 0.52955750696307 , -0.234877884633011], [ 0.738411389325318, 0.60456567930886 , -0.298745643510446], [ 0.831118161297094, -0.554675017903184, -0.039726898648653], [ 0.059232760156726, -0.659900016376516, -0.749014985504622], [ 0.612797965421487, -0.790091411850324, 0.015304067944409], [ 0.893068596305583, 0.442135678021292, -0.083333813745848], [ 0.543868961008293, -0.785261236078178, 0.295924558569801], [ 0.023057245477744, 0.909142863695828, 0.415845664666506], [ 0.314301148641478, -0.688808186609678, -0.653267227114705], [ 0.227469204691534, 0.698859344384369, -0.678124898292085], [ 0.480021206587709, 0.515357481340314, -0.709919930451771], [-0.199527112610095, -0.290541587310527, 0.935828252072226], [ 0.509508299541406, 0.83724381126344 , -0.198555013030378], [ 0.265608720024493, -0.944508012007856, 0.193278615216269], [ 0.183989443642296, 0.743624771284641, 0.642783077064156], [ 0.459140289426893, -0.870904254202035, -0.175259734787506], [-0.486953356967765, 0.167149330162031, -0.857284975701901], [-0.896938967982807, -0.253653932927113, 0.362160420289882], [-0.810090170986684, 0.372592187261529, -0.452690818177743], [-0.235500783713189, 0.882950179878211, 0.406126040440044], [ 0.245601527914448, 0.478883702650812, 0.842822809860731], [ 0.551197720414493, -0.701987041352189, -0.450993643839318], [-0.292317454148224, 0.384382652764161, 0.875671446522203], [-0.498405784423665, -0.123259203684737, -0.85813684384254 ], [-0.226978661222809, 0.424659947067733, -0.876438598366099], [ 0.350781965135737, -0.065192454982283, -0.934185183327638], [-0.866369726032976, -0.416529816345126, -0.275511179281423], [-0.983798625591528, -0.039811257413103, 0.174800823989501], [ 0.074687863653138, -0.00519367387163 , -0.997193435986542], [-0.486657677484974, -0.735902128438331, 0.470757222254599], [-0.535664726747843, 0.448149937968275, 0.715698912684091], [-0.741845472739466, 0.335940639358327, 0.580352635389447], [-0.918612903404631, -0.098775990781587, -0.382614214769434], [-0.214759467459078, 0.869580209588297, -0.444644386256103], [-0.069732896730654, -0.516045290680157, 0.853718092276592], [-0.743276161976444, 0.585836942779623, 0.323010252952118], [ 0.168978930389119, -0.165103559374229, 0.971692819653675], [-0.176978801411961, 0.982134486883932, 0.063955871693077], [-0.724758560142085, -0.422954781095271, 0.543906501753218], [-0.73890407853608 , -0.672510465844111, -0.04183821283089 ], [ 0.73698306840377 , 0.451781284648806, 0.50274210856784 ], [-0.179222279770771, -0.890817843645075, 0.417519992188765], [ 0.726979640461014, 0.176857921758935, 0.663492183726594], [ 0.117217137008776, 0.988853582972691, 0.091808138166168], [-0.720730984133746, -0.129609130770797, 0.680990691368421], [ 0.362982631017957, -0.609626810828438, 0.704697638067867], [ 0.42218169612431 , 0.905973793453933, 0.031210591668393], [ 0.591501293457958, -0.107553960244072, -0.799098470448655], [-0.029248290025677, 0.353773658072318, 0.93487364728326 ], [-0.504548960119934, -0.444381482757379, -0.740240126326752], [ 0.788072512355533, -0.096358606825903, 0.607994024773442], [ 0.60726841549829 , -0.355934693207345, 0.710306670185202], [-0.481131968897622, -0.686997515730952, -0.54456077887064 ], [ 0.867267106883638, -0.055443243075175, -0.494746209803564], [ 0.97087331565506 , -0.239478873167382, -0.007407715938823], [-0.907276828613544, 0.297990770269208, 0.296732635709849], [-0.9794480341064 , -0.18679073257907 , -0.076097113661971], [-0.328865685812915, -0.8820568850465 , -0.337376662258766], [ 0.089103155106453, -0.75616258665444 , 0.648289110115342], [ 0.227921439135609, 0.162165392298572, 0.960080310767283], [-0.472125872891334, -0.261410342843101, 0.841879915903241], [-0.571579860449469, 0.640183203825906, -0.513285426120559], [-0.957193178150123, 0.155452473842011, -0.244163363507438], [ 0.630631543724712, 0.72532400195024 , 0.276059682413559], [-0.113819063857892, -0.970240839104064, 0.213723968794223], [ 0.815108107019787, -0.52955750696307 , 0.234877884633011], [-0.738411389325318, -0.60456567930886 , 0.298745643510446], [-0.831118161297094, 0.554675017903184, 0.039726898648653], [-0.059232760156726, 0.659900016376516, 0.749014985504622], [-0.612797965421487, 0.790091411850324, -0.015304067944409], [-0.893068596305583, -0.442135678021292, 0.083333813745848], [-0.543868961008293, 0.785261236078178, -0.295924558569801], [-0.023057245477744, -0.909142863695828, -0.415845664666506], [-0.314301148641478, 0.688808186609678, 0.653267227114705], [-0.227469204691534, -0.698859344384369, 0.678124898292085], [-0.480021206587709, -0.515357481340314, 0.709919930451771], [ 0.199527112610095, 0.290541587310527, -0.935828252072226], [-0.509508299541406, -0.83724381126344 , 0.198555013030378], [-0.265608720024493, 0.944508012007856, -0.193278615216269], [-0.183989443642296, -0.743624771284641, -0.642783077064156], [-0.459140289426893, 0.870904254202035, 0.175259734787506], [ 0.486953356967765, -0.167149330162031, 0.857284975701901], [ 0.896938967982807, 0.253653932927113, -0.362160420289882], [ 0.810090170986684, -0.372592187261529, 0.452690818177743], [ 0.235500783713189, -0.882950179878211, -0.406126040440044], [-0.245601527914448, -0.478883702650812, -0.842822809860731], [-0.551197720414493, 0.701987041352189, 0.450993643839318], [ 0.292317454148224, -0.384382652764161, -0.875671446522203], [ 0.498405784423665, 0.123259203684737, 0.85813684384254 ], [ 0.226978661222809, -0.424659947067733, 0.876438598366099], [-0.350781965135737, 0.065192454982283, 0.934185183327638], [ 0.866369726032976, 0.416529816345126, 0.275511179281423], [ 0.983798625591528, 0.039811257413103, -0.174800823989501], [-0.074687863653138, 0.00519367387163 , 0.997193435986542], [ 0.486657677484974, 0.735902128438331, -0.470757222254599], [ 0.535664726747843, -0.448149937968275, -0.715698912684091], [ 0.741845472739466, -0.335940639358327, -0.580352635389447], [ 0.918612903404631, 0.098775990781587, 0.382614214769434], [ 0.214759467459078, -0.869580209588297, 0.444644386256103], [ 0.069732896730654, 0.516045290680157, -0.853718092276592], [ 0.743276161976444, -0.585836942779623, -0.323010252952118], [-0.168978930389119, 0.165103559374229, -0.971692819653675], [ 0.176978801411961, -0.982134486883932, -0.063955871693077], [ 0.724758560142085, 0.422954781095271, -0.543906501753218], [ 0.73890407853608 , 0.672510465844111, 0.04183821283089 ], [-0.73698306840377 , -0.451781284648806, -0.50274210856784 ], [ 0.179222279770771, 0.890817843645075, -0.417519992188765], [-0.726979640461014, -0.176857921758935, -0.663492183726594], [-0.117217137008776, -0.988853582972691, -0.091808138166168], [ 0.720730984133746, 0.129609130770797, -0.680990691368421], [-0.362982631017957, 0.609626810828438, -0.704697638067867], [-0.42218169612431 , -0.905973793453933, -0.031210591668393], [-0.591501293457958, 0.107553960244072, 0.799098470448655], [ 0.029248290025677, -0.353773658072318, -0.93487364728326 ], [ 0.504548960119934, 0.444381482757379, 0.740240126326752], [-0.788072512355533, 0.096358606825903, -0.607994024773442], [-0.60726841549829 , 0.355934693207345, -0.710306670185202], [ 0.481131968897622, 0.686997515730952, 0.54456077887064 ]])

if calculate_design_score(spherical_design_215130, 2, 15, 130) < 1e-8:
  print("Correct numerical construction of a (2, 15, 130) design")


spherical_design_219198 = np.array([[ 2.527754619599024e-01, -9.566794162279668e-01, -1.444612764607689e-01], [-9.071780161379499e-01, 1.596229165164883e-02, 4.204441131485148e-01], [-6.164708780571496e-01, -2.079705417702330e-01, -7.594155056773878e-01], [-6.669234790483386e-01, 5.278082682594426e-01, 5.259577027205025e-01], [-6.636958444333798e-01, 3.035315648773164e-01, 6.836493364327872e-01], [ 3.181767179657207e-01, -4.481701237164781e-01, -8.354083530540731e-01], [ 2.326403415952969e-01, -2.475393988441263e-01, 9.405332091331555e-01], [-3.319416667817200e-01, -9.192339534181356e-01, 2.117160096389462e-01], [-9.899011550967391e-01, 1.415574599986714e-01, -7.562318220369787e-03], [-6.940971646568641e-01, 6.760456966183580e-01, -2.473688381730833e-01], [-8.911202517174470e-02, 9.893780742393793e-01, -1.148480438848112e-01], [ 2.735315319063146e-01, 7.929937352708630e-01, 5.443725166410857e-01], [ 3.211498740191649e-01, -6.978193443976749e-01, -6.402428609534616e-01], [ 2.160613201002565e-01, 7.253525452614101e-02, -9.736817461611172e-01], [ 8.306685222602077e-01, -4.996648627125291e-01, 2.456111379732449e-01], [-7.035961399107711e-01, 2.468113759227220e-01, -6.663607255967261e-01], [ 4.543383597653380e-01, 7.882959811463807e-01, -4.149290312260727e-01], [-5.919090358368607e-01, -7.718700615542744e-01, 2.320782225261097e-01], [-2.249576936505070e-01, 2.145140044439518e-01, 9.504618761238480e-01], [-8.011025453303287e-01, 5.981993295849898e-01, 1.980590693044467e-02], [ 9.650792727240828e-02, -7.614518003297946e-01, -6.409971729642011e-01], [-4.266305887077118e-01, -9.044256581291615e-01, -7.547824482565929e-04], [ 5.235354935901722e-01, -8.265603725325779e-01, 2.066604401188114e-01], [-9.261323200044729e-01, -3.380056783928196e-01, 1.674248703668078e-01], [ 9.465140670260545e-01, -2.113846895741718e-01, -2.437778372523406e-01], [ 1.240057045030078e-01, 8.885318356130942e-01, 4.417394733921091e-01], [-1.293949085487952e-01, -9.899593844738367e-01, 5.689793259716198e-02], [ 4.100397008350419e-01, -5.310451154888398e-01, 7.415244628834265e-01], [ 6.121488896350307e-01, 7.885838833306796e-01, 5.838831963418752e-02], [-7.078020326836322e-01, 6.476887108990562e-01, 2.819851348969254e-01], [-1.194174109333504e-01, 8.959378425659503e-01, 4.278257404881637e-01], [ 6.086090243295595e-01, -1.651204877934196e-03, -7.934685431868567e-01], [-3.081362434841293e-01, -5.828647293185515e-01, -7.518781568631323e-01], [-9.126930195001084e-01, 1.009234202181679e-01, -3.959872667236179e-01], [-9.075847580080885e-01, 4.135337749515982e-01, 7.266170935011333e-02], [-7.874330204597536e-01, -2.277030623556658e-01, -5.728006229775603e-01], [ 3.561718528355594e-01, -7.157128899153343e-01, 6.007467606710211e-01], [ 4.971254388061538e-01, -3.049862662866634e-01, -8.123113168412154e-01], [ 6.355128637904900e-01, -7.707274677124771e-01, -4.585379450299186e-02], [-8.235830294757543e-01, 2.616115548642214e-01, 5.032597618735906e-01], [-4.201357400099681e-01, 5.916441792288146e-02, 9.055305249510494e-01], [-2.253384808887542e-01, -8.181435109508091e-01, 5.290215161218226e-01], [-9.830919480156279e-01, -1.814692250874678e-01, -2.447737921003454e-02], [-8.033877059588190e-01, -4.784019030480327e-01, 3.545416944087772e-01], [ 5.398044024339611e-01, 2.398737188084147e-01, -8.068902069909732e-01], [ 1.847833401650680e-01, 7.036040116425582e-02, 9.802573800515075e-01], [ 4.043070734752842e-01, 1.196349008076413e-01, 9.067652843192647e-01], [ 6.778777955299986e-01, -4.876073892763494e-01, 5.502006254544718e-01], [-8.358101865225558e-01, 4.387296529383495e-01, 3.300570007403659e-01], [-1.498456246602617e-01, -6.995634017135060e-01, 6.986825715252984e-01], [ 9.630190996489819e-02, 6.802849918749052e-01, 7.265936085370368e-01], [-3.374445218671575e-01, -2.667697276595186e-01, 9.027541786479372e-01], [-3.038743924704718e-01, 9.408787690713419e-01, -1.496913341229145e-01], [ 7.727519028802673e-02, 4.376050021541193e-01, 8.958406147611541e-01], [ 5.002298245294255e-01, 4.114399907826821e-01, 7.618971430816674e-01], [ 2.129927177656035e-01, 9.597273280589687e-01, 1.831872210488892e-01], [-3.325360856013789e-02, 9.811691832350971e-01, 1.902662118913964e-01], [-5.550664481301818e-01, -7.413685214163005e-01, -3.771921971795716e-01], [ 4.473826692631221e-01, 6.394806856618824e-01, -6.252305174161067e-01], [-5.132743118038068e-01, 3.594213914485606e-01, -7.793367335186434e-01], [-9.082727207991121e-01, -3.678183113537082e-01, -1.993749093669625e-01], [-9.768754467245306e-01, -2.555834031648304e-02, -2.122760769069738e-01], [ 4.191412107224712e-01, -1.457415933828401e-01, 8.961473279725448e-01], [ 9.819722339960717e-01, 5.107526040883529e-02, -1.819940917583148e-01], [-6.456474689156548e-01, -4.088957599514469e-01, 6.449368987556358e-01], [-7.301676629146590e-01, -6.539135382177232e-01, -1.981218528313398e-01], [ 9.002401487782238e-01, 2.173390803794627e-01, 3.772683377485724e-01], [-6.586103690033770e-01, -5.896530407350642e-01, 4.674844097861748e-01], [-2.856519115798793e-01, -3.388344915136407e-01, -8.964341430196942e-01], [ 4.561570687705105e-01, -8.851097629634888e-01, -9.220323268417704e-02], [ 7.541963115053172e-01, 6.406484260432386e-01, -1.440746956272877e-01], [ 8.073812773870909e-01, -2.094685439783286e-02, 5.896581231659794e-01], [ 1.221382481056316e-01, 9.305768704823116e-01, -3.451216256235389e-01], [ 1.118204382413834e-01, -9.155428785210348e-01, 3.863643710034904e-01], [-8.860365005155635e-01, -2.365125284686243e-01, 3.987494747727099e-01], [-2.179652885557556e-02, 2.209721440966156e-01, 9.750365238611284e-01], [ 1.430693307304775e-01, 5.109212304138933e-01, -8.476382854240874e-01], [-2.009648897987387e-02, 8.804010533461014e-02, -9.959141885641315e-01], [ 4.026367248507784e-01, 8.707976665219648e-01, 2.821260210993299e-01], [ 4.959530123515291e-01, -5.845748172565104e-01, -6.421081626711822e-01], [ 1.788188063675312e-01, -4.589311733615876e-01, 8.702907632545848e-01], [-3.655614569252785e-01, 8.407635588409698e-01, -3.993512981520591e-01], [-1.179456284140268e-01, 6.437532672194255e-01, -7.560889892614064e-01], [ 8.624212784692293e-01, 5.060958510348700e-01, 9.824866858670378e-03], [ 9.398677207317259e-01, -2.930045749820879e-01, 1.754907022212762e-01], [-3.526849454875763e-01, 8.468220761178133e-01, 3.981277441047554e-01], [ 6.754392544921588e-01, 4.675915954028452e-01, 5.702104115146430e-01], [ 8.877168126974362e-02, -8.062034123562707e-01, 5.849407205090512e-01], [ 5.310154512278381e-01, -7.630797357554068e-01, -3.684181149682437e-01], [-3.919846588219221e-01, -4.962132089826344e-01, 7.746744338620049e-01], [ 7.726351110481466e-01, 5.087844222309769e-01, 3.797017209214478e-01], [-6.194283705963191e-01, 3.579291718821567e-02, -7.842368014697771e-01], [ 5.985068718037222e-02, 3.316528745152836e-01, -9.415010706684229e-01], [-1.282471071248554e-01, 5.369778945564482e-01, 8.337909931582553e-01], [-5.807510898257759e-01, 6.554449654241324e-01, -4.828250914837911e-01], [-4.927203579527381e-01, -6.488507967014375e-01, -5.798441967277377e-01], [ 8.340824319523842e-01, -3.325402853788726e-01, 4.401402677653276e-01], [-7.684867837654284e-01, 3.229495196674979e-02, 6.390501539435953e-01], [ 7.643681235826285e-01, 2.256857024157286e-01, -6.039928272553331e-01], [-2.527754619599024e-01, 9.566794162279668e-01, 1.444612764607689e-01], [ 9.071780161379499e-01, -1.596229165164883e-02, -4.204441131485148e-01], [ 6.164708780571496e-01, 2.079705417702330e-01, 7.594155056773878e-01], [ 6.669234790483386e-01, -5.278082682594426e-01, -5.259577027205025e-01], [ 6.636958444333798e-01, -3.035315648773164e-01, -6.836493364327872e-01], [-3.181767179657207e-01, 4.481701237164781e-01, 8.354083530540731e-01], [-2.326403415952969e-01, 2.475393988441263e-01, -9.405332091331555e-01], [ 3.319416667817200e-01, 9.192339534181356e-01, -2.117160096389462e-01], [ 9.899011550967391e-01, -1.415574599986714e-01, 7.562318220369787e-03], [ 6.940971646568641e-01, -6.760456966183580e-01, 2.473688381730833e-01], [ 8.911202517174470e-02, -9.893780742393793e-01, 1.148480438848112e-01], [-2.735315319063146e-01, -7.929937352708630e-01, -5.443725166410857e-01], [-3.211498740191649e-01, 6.978193443976749e-01, 6.402428609534616e-01], [-2.160613201002565e-01, -7.253525452614101e-02, 9.736817461611172e-01], [-8.306685222602077e-01, 4.996648627125291e-01, -2.456111379732449e-01], [ 7.035961399107711e-01, -2.468113759227220e-01, 6.663607255967261e-01], [-4.543383597653380e-01, -7.882959811463807e-01, 4.149290312260727e-01], [ 5.919090358368607e-01, 7.718700615542744e-01, -2.320782225261097e-01], [ 2.249576936505070e-01, -2.145140044439518e-01, -9.504618761238480e-01], [ 8.011025453303287e-01, -5.981993295849898e-01, -1.980590693044467e-02], [-9.650792727240828e-02, 7.614518003297946e-01, 6.409971729642011e-01], [ 4.266305887077118e-01, 9.044256581291615e-01, 7.547824482565929e-04], [-5.235354935901722e-01, 8.265603725325779e-01, -2.066604401188114e-01], [ 9.261323200044729e-01, 3.380056783928196e-01, -1.674248703668078e-01], [-9.465140670260545e-01, 2.113846895741718e-01, 2.437778372523406e-01], [-1.240057045030078e-01, -8.885318356130942e-01, -4.417394733921091e-01], [ 1.293949085487952e-01, 9.899593844738367e-01, -5.689793259716198e-02], [-4.100397008350419e-01, 5.310451154888398e-01, -7.415244628834265e-01], [-6.121488896350307e-01, -7.885838833306796e-01, -5.838831963418752e-02], [ 7.078020326836322e-01, -6.476887108990562e-01, -2.819851348969254e-01], [ 1.194174109333504e-01, -8.959378425659503e-01, -4.278257404881637e-01], [-6.086090243295595e-01, 1.651204877934196e-03, 7.934685431868567e-01], [ 3.081362434841293e-01, 5.828647293185515e-01, 7.518781568631323e-01], [ 9.126930195001084e-01, -1.009234202181679e-01, 3.959872667236179e-01], [ 9.075847580080885e-01, -4.135337749515982e-01, -7.266170935011333e-02], [ 7.874330204597536e-01, 2.277030623556658e-01, 5.728006229775603e-01], [-3.561718528355594e-01, 7.157128899153343e-01, -6.007467606710211e-01], [-4.971254388061538e-01, 3.049862662866634e-01, 8.123113168412154e-01], [-6.355128637904900e-01, 7.707274677124771e-01, 4.585379450299186e-02], [ 8.235830294757543e-01, -2.616115548642214e-01, -5.032597618735906e-01], [ 4.201357400099681e-01, -5.916441792288146e-02, -9.055305249510494e-01], [ 2.253384808887542e-01, 8.181435109508091e-01, -5.290215161218226e-01], [ 9.830919480156279e-01, 1.814692250874678e-01, 2.447737921003454e-02], [ 8.033877059588190e-01, 4.784019030480327e-01, -3.545416944087772e-01], [-5.398044024339611e-01, -2.398737188084147e-01, 8.068902069909732e-01], [-1.847833401650680e-01, -7.036040116425582e-02, -9.802573800515075e-01], [-4.043070734752842e-01, -1.196349008076413e-01, -9.067652843192647e-01], [-6.778777955299986e-01, 4.876073892763494e-01, -5.502006254544718e-01], [ 8.358101865225558e-01, -4.387296529383495e-01, -3.300570007403659e-01], [ 1.498456246602617e-01, 6.995634017135060e-01, -6.986825715252984e-01], [-9.630190996489819e-02, -6.802849918749052e-01, -7.265936085370368e-01], [ 3.374445218671575e-01, 2.667697276595186e-01, -9.027541786479372e-01], [ 3.038743924704718e-01, -9.408787690713419e-01, 1.496913341229145e-01], [-7.727519028802673e-02, -4.376050021541193e-01, -8.958406147611541e-01], [-5.002298245294255e-01, -4.114399907826821e-01, -7.618971430816674e-01], [-2.129927177656035e-01, -9.597273280589687e-01, -1.831872210488892e-01], [ 3.325360856013789e-02, -9.811691832350971e-01, -1.902662118913964e-01], [ 5.550664481301818e-01, 7.413685214163005e-01, 3.771921971795716e-01], [-4.473826692631221e-01, -6.394806856618824e-01, 6.252305174161067e-01], [ 5.132743118038068e-01, -3.594213914485606e-01, 7.793367335186434e-01], [ 9.082727207991121e-01, 3.678183113537082e-01, 1.993749093669625e-01], [ 9.768754467245306e-01, 2.555834031648304e-02, 2.122760769069738e-01], [-4.191412107224712e-01, 1.457415933828401e-01, -8.961473279725448e-01], [-9.819722339960717e-01, -5.107526040883529e-02, 1.819940917583148e-01], [ 6.456474689156548e-01, 4.088957599514469e-01, -6.449368987556358e-01], [ 7.301676629146590e-01, 6.539135382177232e-01, 1.981218528313398e-01], [-9.002401487782238e-01, -2.173390803794627e-01, -3.772683377485724e-01], [ 6.586103690033770e-01, 5.896530407350642e-01, -4.674844097861748e-01], [ 2.856519115798793e-01, 3.388344915136407e-01, 8.964341430196942e-01], [-4.561570687705105e-01, 8.851097629634888e-01, 9.220323268417704e-02], [-7.541963115053172e-01, -6.406484260432386e-01, 1.440746956272877e-01], [-8.073812773870909e-01, 2.094685439783286e-02, -5.896581231659794e-01], [-1.221382481056316e-01, -9.305768704823116e-01, 3.451216256235389e-01], [-1.118204382413834e-01, 9.155428785210348e-01, -3.863643710034904e-01], [ 8.860365005155635e-01, 2.365125284686243e-01, -3.987494747727099e-01], [ 2.179652885557556e-02, -2.209721440966156e-01, -9.750365238611284e-01], [-1.430693307304775e-01, -5.109212304138933e-01, 8.476382854240874e-01], [ 2.009648897987387e-02, -8.804010533461014e-02, 9.959141885641315e-01], [-4.026367248507784e-01, -8.707976665219648e-01, -2.821260210993299e-01], [-4.959530123515291e-01, 5.845748172565104e-01, 6.421081626711822e-01], [-1.788188063675312e-01, 4.589311733615876e-01, -8.702907632545848e-01], [ 3.655614569252785e-01, -8.407635588409698e-01, 3.993512981520591e-01], [ 1.179456284140268e-01, -6.437532672194255e-01, 7.560889892614064e-01], [-8.624212784692293e-01, -5.060958510348700e-01, -9.824866858670378e-03], [-9.398677207317259e-01, 2.930045749820879e-01, -1.754907022212762e-01], [ 3.526849454875763e-01, -8.468220761178133e-01, -3.981277441047554e-01], [-6.754392544921588e-01, -4.675915954028452e-01, -5.702104115146430e-01], [-8.877168126974362e-02, 8.062034123562707e-01, -5.849407205090512e-01], [-5.310154512278381e-01, 7.630797357554068e-01, 3.684181149682437e-01], [ 3.919846588219221e-01, 4.962132089826344e-01, -7.746744338620049e-01], [-7.726351110481466e-01, -5.087844222309769e-01, -3.797017209214478e-01], [ 6.194283705963191e-01, -3.579291718821567e-02, 7.842368014697771e-01], [-5.985068718037222e-02, -3.316528745152836e-01, 9.415010706684229e-01], [ 1.282471071248554e-01, -5.369778945564482e-01, -8.337909931582553e-01], [ 5.807510898257759e-01, -6.554449654241324e-01, 4.828250914837911e-01], [ 4.927203579527381e-01, 6.488507967014375e-01, 5.798441967277377e-01], [-8.340824319523842e-01, 3.325402853788726e-01, -4.401402677653276e-01], [ 7.684867837654284e-01, -3.229495196674979e-02, -6.390501539435953e-01], [-7.643681235826285e-01, -2.256857024157286e-01, 6.039928272553331e-01]])

if calculate_design_score(spherical_design_219198, 2, 19, 198) < 1e-8:
  print("Correct numerical construction of a (2, 19, 198) design")


spherical_design_219200 = np.array([[-0.016645223332694, -0.34301334375171 , 0.939183039959983], [ 0.199800987805441, 0.188841679508623, -0.96146678848119 ], [-0.337328402820051, -0.207913096722562, -0.918140344861344], [ 0.557272080796684, 0.824873903343072, 0.095030897860115], [-0.207885048084773, 0.766402335786565, -0.607792124400842], [-0.017398098849326, 0.881072823346621, 0.472660540045859], [ 0.742833440018621, 0.334003057371384, 0.580207237163301], [-0.811070427575855, -0.107627303086817, 0.574961846683915], [-0.467351650343033, 0.398926811404843, 0.788948562368934], [-0.520649403826713, 0.144011703401574, -0.841537181340353], [-0.526653200982222, 0.801469947893829, -0.283341363937994], [-0.645914190286317, 0.537924092088522, -0.541694129502536], [ 0.277716443411556, 0.853508204685416, 0.440905116315874], [-0.853667515014562, 0.440201706167119, -0.278341933054328], [-0.456058452938282, -0.839013789412143, 0.29676008606249 ], [-0.124612740576602, -0.955411701791326, -0.26769412575959 ], [-0.010424546373886, -0.212722309957381, -0.97705708516918 ], [ 0.838251106136731, 0.524974518008731, -0.147434183627978], [ 0.991959637951426, 0.106162437561863, -0.068888413584566], [ 0.914729803833028, 0.216927583985381, -0.340898532243622], [-0.71359604807947 , -0.381249693890056, 0.587732380488008], [ 0.474155326112657, 0.407308364814402, 0.780561735336301], [ 0.591841694722172, 0.199492796839383, 0.78097761324995 ], [ 0.710707949131369, 0.644659216059876, 0.281618014676882], [ 0.694911359167278, 0.035353677739789, 0.718225814330394], [-0.804899676703494, -0.445747387049662, -0.391721555931008], [-0.333972155957743, 0.576879304737105, 0.745434683128556], [ 0.37538526923058 , 0.00416928121037 , 0.92685949136796 ], [ 0.279278671770358, -0.899087478547319, 0.337112932729683], [-0.438948428584403, -0.858729705926135, -0.264400395618647], [-0.944749832347239, -0.318173766076026, 0.078823910464158], [ 0.004159526898207, 0.597336374672766, 0.801980020841405], [-0.249025546301066, -0.965898078271688, -0.070902607007887], [ 0.165555811571675, -0.192000553868132, 0.967329861303358], [-0.87689794658436 , 0.070552365549247, -0.475470666804523], [-0.463754779464564, -0.154498195697366, 0.872388567124783], [-0.590100729644147, -0.527876240708552, -0.610841880824205], [ 0.949172667032748, -0.312559495188914, 0.037118864815489], [-0.777444071521729, -0.626114372724306, -0.059678370652744], [-0.156884770865897, 0.954776340193638, 0.252565454638589], [-0.464749486306461, 0.883524826134882, -0.058239132730562], [-0.358617711237953, 0.329507379735897, -0.873394655288224], [ 0.070373362018437, 0.981466322948235, -0.178245467927498], [ 0.56122865652331 , 0.697510768350718, 0.445534648632215], [ 0.095058564771282, 0.759966038119736, 0.642973942060031], [ 0.202220023495776, -0.968194876003073, 0.147328694349611], [-0.179316274914743, -0.917518840114806, 0.354971620816484], [ 0.166647100712017, 0.378179960817545, 0.910608950680982], [ 0.648650595650254, 0.203521247518982, -0.733369965686411], [ 0.946752979141536, -0.192261894822022, 0.258252125424111], [-0.712959662393853, 0.207647602210156, -0.66975442745503 ], [-0.638606855808292, 0.349127116688006, 0.685778054553925], [ 0.275238564046921, 0.55698719917187 , 0.783587259225202], [ 0.059900140997436, -0.894264850265529, 0.443511387326253], [-0.69405440822944 , 0.634096437113273, -0.340896152600076], [ 0.449372788709154, -0.106343748995522, -0.88699216671701 ], [-0.162702130100806, -0.502365280599526, 0.84920971597646 ], [ 0.688463505330094, 0.646670929812405, -0.328381958036916], [-0.440477968819595, 0.741657497117829, -0.505888639873917], [ 0.630288112515893, 0.767270424206114, -0.11846092756569 ], [ 0.188687689434671, -0.730065980594231, -0.65681094679884 ], [-0.987405530512703, 0.125923111746693, 0.095779372732114], [ 0.877649322597667, -0.454050225166136, -0.15352543623269 ], [ 0.634030768346919, -0.082683250814887, -0.768874804388919], [ 0.198602751864554, -0.015195535053196, -0.979962265940007], [-0.940049659364742, -0.21268425140512 , -0.266593411644917], [ 0.359604339129607, 0.927383220618704, -0.103175003726865], [-0.64495741126795 , 0.569015772870259, 0.510148005852621], [ 0.133265813774489, -0.442069787463885, -0.887025662475815], [ 0.55795650866218 , -0.412298588263738, 0.720204421367461], [-0.289920688661014, -0.648555925782214, 0.703790597705808], [ 0.384450071430028, 0.373544051979806, -0.844193688562024], [ 0.915518394856399, -0.246411429167115, -0.317974018208066], [ 0.359954103666641, 0.701293680886983, 0.615321230254191], [ 0.820199914967553, 0.419668550107567, -0.388780667649294], [-0.35227399262971 , -0.786722485126341, 0.506922839802427], [ 0.569651648366847, 0.639840374598508, -0.515850069832868], [-0.81679970060785 , 0.572582343696041, -0.070623712550921], [ 0.038516449804762, -0.998497539788184, -0.038976225206932], [ 0.80093356404725 , -0.360440488167845, -0.478108858390947], [ 0.796653720495921, -0.126383256200926, -0.591075394659647], [-0.271967957647126, 0.26470611587414 , 0.925183280346158], [ 0.677209203612915, -0.734242176106835, 0.047708713751908], [ 0.901465219807989, 0.404673484083938, 0.15362235760429 ], [ 0.084622154123705, 0.005799802055982, 0.996396233095838], [ 0.47558814912896 , -0.819699488317526, -0.319231360082427], [ 0.345285763278275, -0.932015544681829, -0.11011251585888 ], [-0.98441214927349 , -0.018031422650028, -0.174950244812529], [ 0.485596221621728, -0.674669106644768, -0.555893790292581], [ 0.631351932733037, -0.751571581608738, -0.191141033669904], [-0.514516925156017, -0.499533278123833, 0.69694966660072 ], [ 0.808272897498155, -0.327847520703709, 0.489091940578015], [-0.853567733902433, -0.135945461501044, -0.502932356423738], [-0.389888223103682, 0.598570773597485, -0.69978582614966 ], [ 0.761207605588326, -0.582561016054552, -0.2849309456131 ], [ 0.042442028792566, -0.644163334772899, 0.763709547096426], [ 0.196260766712348, -0.497875514133927, 0.844749479949741], [ 0.095692031984443, 0.785716226418486, -0.611140774746198], [ 0.28269913609294 , -0.843417877295711, -0.456867029572394], [-0.934925766637204, 0.00920542628358 , 0.354723936329048], [ 0.016645223332694, 0.34301334375171 , -0.939183039959983], [-0.199800987805441, -0.188841679508623, 0.96146678848119 ], [ 0.337328402820051, 0.207913096722562, 0.918140344861344], [-0.557272080796684, -0.824873903343072, -0.095030897860115], [ 0.207885048084773, -0.766402335786565, 0.607792124400842], [ 0.017398098849326, -0.881072823346621, -0.472660540045859], [-0.742833440018621, -0.334003057371384, -0.580207237163301], [ 0.811070427575855, 0.107627303086817, -0.574961846683915], [ 0.467351650343033, -0.398926811404843, -0.788948562368934], [ 0.520649403826713, -0.144011703401574, 0.841537181340353], [ 0.526653200982222, -0.801469947893829, 0.283341363937994], [ 0.645914190286317, -0.537924092088522, 0.541694129502536], [-0.277716443411556, -0.853508204685416, -0.440905116315874], [ 0.853667515014562, -0.440201706167119, 0.278341933054328], [ 0.456058452938282, 0.839013789412143, -0.29676008606249 ], [ 0.124612740576602, 0.955411701791326, 0.26769412575959 ], [ 0.010424546373886, 0.212722309957381, 0.97705708516918 ], [-0.838251106136731, -0.524974518008731, 0.147434183627978], [-0.991959637951426, -0.106162437561863, 0.068888413584566], [-0.914729803833028, -0.216927583985381, 0.340898532243622], [ 0.71359604807947 , 0.381249693890056, -0.587732380488008], [-0.474155326112657, -0.407308364814402, -0.780561735336301], [-0.591841694722172, -0.199492796839383, -0.78097761324995 ], [-0.710707949131369, -0.644659216059876, -0.281618014676882], [-0.694911359167278, -0.035353677739789, -0.718225814330394], [ 0.804899676703494, 0.445747387049662, 0.391721555931008], [ 0.333972155957743, -0.576879304737105, -0.745434683128556], [-0.37538526923058 , -0.00416928121037 , -0.92685949136796 ], [-0.279278671770358, 0.899087478547319, -0.337112932729683], [ 0.438948428584403, 0.858729705926135, 0.264400395618647], [ 0.944749832347239, 0.318173766076026, -0.078823910464158], [-0.004159526898207, -0.597336374672766, -0.801980020841405], [ 0.249025546301066, 0.965898078271688, 0.070902607007887], [-0.165555811571675, 0.192000553868132, -0.967329861303358], [ 0.87689794658436 , -0.070552365549247, 0.475470666804523], [ 0.463754779464564, 0.154498195697366, -0.872388567124783], [ 0.590100729644147, 0.527876240708552, 0.610841880824205], [-0.949172667032748, 0.312559495188914, -0.037118864815489], [ 0.777444071521729, 0.626114372724306, 0.059678370652744], [ 0.156884770865897, -0.954776340193638, -0.252565454638589], [ 0.464749486306461, -0.883524826134882, 0.058239132730562], [ 0.358617711237953, -0.329507379735897, 0.873394655288224], [-0.070373362018437, -0.981466322948235, 0.178245467927498], [-0.56122865652331 , -0.697510768350718, -0.445534648632215], [-0.095058564771282, -0.759966038119736, -0.642973942060031], [-0.202220023495776, 0.968194876003073, -0.147328694349611], [ 0.179316274914743, 0.917518840114806, -0.354971620816484], [-0.166647100712017, -0.378179960817545, -0.910608950680982], [-0.648650595650254, -0.203521247518982, 0.733369965686411], [-0.946752979141536, 0.192261894822022, -0.258252125424111], [ 0.712959662393853, -0.207647602210156, 0.66975442745503 ], [ 0.638606855808292, -0.349127116688006, -0.685778054553925], [-0.275238564046921, -0.55698719917187 , -0.783587259225202], [-0.059900140997436, 0.894264850265529, -0.443511387326253], [ 0.69405440822944 , -0.634096437113273, 0.340896152600076], [-0.449372788709154, 0.106343748995522, 0.88699216671701 ], [ 0.162702130100806, 0.502365280599526, -0.84920971597646 ], [-0.688463505330094, -0.646670929812405, 0.328381958036916], [ 0.440477968819595, -0.741657497117829, 0.505888639873917], [-0.630288112515893, -0.767270424206114, 0.11846092756569 ], [-0.188687689434671, 0.730065980594231, 0.65681094679884 ], [ 0.987405530512703, -0.125923111746693, -0.095779372732114], [-0.877649322597667, 0.454050225166136, 0.15352543623269 ], [-0.634030768346919, 0.082683250814887, 0.768874804388919], [-0.198602751864554, 0.015195535053196, 0.979962265940007], [ 0.940049659364742, 0.21268425140512 , 0.266593411644917], [-0.359604339129607, -0.927383220618704, 0.103175003726865], [ 0.64495741126795 , -0.569015772870259, -0.510148005852621], [-0.133265813774489, 0.442069787463885, 0.887025662475815], [-0.55795650866218 , 0.412298588263738, -0.720204421367461], [ 0.289920688661014, 0.648555925782214, -0.703790597705808], [-0.384450071430028, -0.373544051979806, 0.844193688562024], [-0.915518394856399, 0.246411429167115, 0.317974018208066], [-0.359954103666641, -0.701293680886983, -0.615321230254191], [-0.820199914967553, -0.419668550107567, 0.388780667649294], [ 0.35227399262971 , 0.786722485126341, -0.506922839802427], [-0.569651648366847, -0.639840374598508, 0.515850069832868], [ 0.81679970060785 , -0.572582343696041, 0.070623712550921], [-0.038516449804762, 0.998497539788184, 0.038976225206932], [-0.80093356404725 , 0.360440488167845, 0.478108858390947], [-0.796653720495921, 0.126383256200926, 0.591075394659647], [ 0.271967957647126, -0.26470611587414 , -0.925183280346158], [-0.677209203612915, 0.734242176106835, -0.047708713751908], [-0.901465219807989, -0.404673484083938, -0.15362235760429 ], [-0.084622154123705, -0.005799802055982, -0.996396233095838], [-0.47558814912896 , 0.819699488317526, 0.319231360082427], [-0.345285763278275, 0.932015544681829, 0.11011251585888 ], [ 0.98441214927349 , 0.018031422650028, 0.174950244812529], [-0.485596221621728, 0.674669106644768, 0.555893790292581], [-0.631351932733037, 0.751571581608738, 0.191141033669904], [ 0.514516925156017, 0.499533278123833, -0.69694966660072 ], [-0.808272897498155, 0.327847520703709, -0.489091940578015], [ 0.853567733902433, 0.135945461501044, 0.502932356423738], [ 0.389888223103682, -0.598570773597485, 0.69978582614966 ], [-0.761207605588326, 0.582561016054552, 0.2849309456131 ], [-0.042442028792566, 0.644163334772899, -0.763709547096426], [-0.196260766712348, 0.497875514133927, -0.844749479949741], [-0.095692031984443, -0.785716226418486, 0.611140774746198], [-0.28269913609294 , 0.843417877295711, 0.456867029572394], [ 0.934925766637204, -0.00920542628358 , -0.354723936329048]])

if calculate_design_score(spherical_design_219200, 2, 19, 200) < 1e-8:
  print("Correct numerical construction of a (2, 19, 200) design")


spherical_design_219202 = np.array([[ 0.81076996368649 , 0.039961116264478, -0.583999293810108], [ 0.647346627319096, -0.64459588670608 , 0.406741302230538], [ 0.186749260680734, 0.561219895822451, -0.806323100356325], [-0.195853186825641, -0.911513632529629, -0.361641295931315], [-0.494875415184051, -0.180788764178258, -0.849949260952271], [-0.902293165778605, -0.177157988845889, -0.393042096953115], [-0.675717883998035, -0.717865756594648, -0.167553265423306], [ 0.272363249724504, -0.270583104767737, -0.923365065190231], [ 0.488235902335905, 0.838642135525272, 0.241464432560691], [ 0.265119092123748, 0.847638240898971, -0.45958816298636 ], [-0.069625290951433, -0.150845296288191, 0.986102436589453], [-0.642824807985868, -0.637056488671774, -0.425364898033585], [ 0.583056527549522, -0.091340353707334, 0.807280636127557], [-0.428841668977862, -0.494984744257901, 0.755701611682959], [ 0.534249581158651, -0.50694100906121 , -0.676452657888051], [-0.926547434683025, 0.253120445955419, 0.278280238467692], [ 0.652680883350534, -0.553561886955904, -0.517278360091146], [ 0.246001681222344, 0.957575254780407, 0.150109307732795], [ 0.756807878748951, 0.314642649178017, -0.572923937344001], [ 0.594957697888294, -0.736323790761684, -0.322261714886838], [-0.475219376672474, 0.790422301072166, -0.386521836385486], [ 0.326838550784899, -0.24724641125665 , 0.912165431180953], [ 0.812246410149005, -0.557867371965696, -0.170410576244969], [-0.682984895725692, 0.239356163110699, 0.690101629755712], [-0.820499911851931, -0.52352613273552 , -0.229565422034687], [-0.834133426351029, -0.450520019116361, 0.318202984617187], [-0.700431269593958, -0.633771608897022, 0.328221852305839], [-0.896963419643415, -0.219139814479089, 0.383971829086017], [ 0.361959182023341, 0.012138899060303, 0.93211490583436 ], [ 0.952270951104356, -0.301237645134509, -0.049355008222273], [ 0.580344573715604, -0.541635733813882, 0.608137244061529], [ 0.933770139873584, 0.324582648866844, 0.150729658445346], [-0.091586054256238, -0.958040446209228, 0.271607249706258], [ 0.115254199470393, -0.347141790859843, 0.930703522365237], [ 0.392508793291143, 0.262889418835746, -0.881377331597163], [ 0.787498756605444, 0.61490295273168 , -0.041714111122517], [ 0.333284272342984, 0.686300943338675, 0.646461606734115], [ 0.806853601862793, -0.138876647553989, 0.574195560697923], [ 0.548658419730724, 0.376148164915614, 0.746650183478984], [ 0.658860291754709, 0.045477056523191, -0.75088944144853 ], [-0.090337586151915, 0.720151444779003, -0.687910617094227], [-0.9822491487486 , 0.128545812014991, -0.136611068351202], [-0.00400643269962 , 0.863134675190444, -0.504957900206453], [-0.121214219865543, 0.111141996522891, 0.98638459513077 ], [ 0.368432367737374, -0.65961613706041 , 0.655106206758061], [-0.648853573418921, -0.51444696364883 , 0.560654405007216], [ 0.595764367980407, -0.795313141594413, 0.111990288204524], [-0.042556073650169, 0.416223007411055, 0.908266144198483], [ 0.815381577858629, -0.441231335005946, 0.374790330048206], [-0.583732709209401, -0.537963423521949, -0.608154157390704], [-0.991871494815841, 0.027534433284753, 0.12422879197463 ], [-0.321429215637727, -0.049548720206305, 0.945636390829178], [-0.156269467318464, 0.941925754615422, -0.29724691146616 ], [ 0.308200027756246, 0.528896682426685, 0.790747141764733], [ 0.801812057457875, 0.40653096475982 , 0.437984017067539], [-0.142795148867885, 0.041914411519452, -0.988864362572935], [-0.451253302588554, -0.89238686633079 , 0.004017176027518], [-0.357123867119546, -0.907317373154443, 0.221895759098057], [-0.418571566177936, -0.784257051876754, -0.457972401536307], [ 0.171633173884482, -0.983665701002221, 0.05425718656684 ], [ 0.547851862030155, -0.307914357079519, 0.777847726727024], [-0.970630545492926, -0.187443689982203, 0.150801880764005], [-0.063542880787338, -0.5814761452615 , -0.811078168115178], [ 0.665597202973365, -0.737939602545095, -0.111469755492781], [-0.749815475143678, -0.253926654004704, -0.610981184342887], [-0.905495239156663, -0.417306235511134, 0.077031666658221], [ 0.355567326884387, -0.485473326495045, 0.798678612029975], [ 0.048610499660401, -0.733659134899369, -0.677776728061296], [-0.332341034425014, 0.909397902532474, 0.250089767297361], [ 0.509474430367807, 0.77377842904384 , -0.376434253951266], [-0.192118748200356, -0.732004758162878, 0.65365083998786 ], [ 0.058404165233372, -0.935860251422498, -0.34749754429465 ], [-0.27869946224737 , -0.325628703868133, -0.90348910173847 ], [ 0.206033224245319, -0.845462201932611, -0.492690547514704], [-0.589516706783643, -0.283002370442149, 0.756557803969461], [ 0.422242397414651, -0.902059113141648, -0.089446711642357], [-0.455257210411192, -0.663420040464546, 0.593813710079718], [-0.387279950595659, 0.90445193867345 , -0.178832129374062], [-0.498911041040294, 0.064747170025559, 0.864231205813451], [ 0.726167040060172, -0.343244422860589, 0.595705209063468], [ 0.718134917572004, 0.061819148330639, 0.693152676589743], [ 0.10844275388053 , 0.216600068135256, 0.970218830787472], [ 0.442790077815399, -0.760884008452726, -0.474333714455493], [-0.985444449918194, -0.106364707537872, -0.132611406431764], [-0.159266331638951, -0.362203309863603, 0.918391527579671], [-0.060441822344003, 0.990235616398184, 0.125619306351547], [ 0.917838074966329, -0.04593709445514 , -0.39428803113349 ], [ 0.140244059367101, 0.988675437383499, -0.053408644682145], [ 0.120361693590686, 0.813981902684486, 0.568283841771104], [ 0.874188873243249, -0.481771110302902, 0.060748754515693], [ 0.314530281913865, -0.667010376750037, -0.675401998121829], [ 0.799441768362303, -0.262703821713619, -0.540258790816784], [-0.755232819543803, 0.632068898314158, -0.173528948789083], [ 0.256946108562324, -0.519581751538401, -0.814870235536288], [-0.806490204894747, 0.462130316152132, 0.36879414352991 ], [-0.27376189217876 , 0.822103902407764, -0.499198958368937], [ 0.087714032004179, -0.556079780204113, 0.826487463085627], [-0.478756625711276, 0.318694689049715, 0.818062215549074], [-0.913546593039296, 0.28551059087732 , -0.289683145597356], [-0.622534011286955, -0.774128068291346, 0.114791718666822], [ 0.911634132049336, -0.035391052742101, 0.409476107567291], [-0.81076996368649 , -0.039961116264478, 0.583999293810108], [-0.647346627319096, 0.64459588670608 , -0.406741302230538], [-0.186749260680734, -0.561219895822451, 0.806323100356325], [ 0.195853186825641, 0.911513632529629, 0.361641295931315], [ 0.494875415184051, 0.180788764178258, 0.849949260952271], [ 0.902293165778605, 0.177157988845889, 0.393042096953115], [ 0.675717883998035, 0.717865756594648, 0.167553265423306], [-0.272363249724504, 0.270583104767737, 0.923365065190231], [-0.488235902335905, -0.838642135525272, -0.241464432560691], [-0.265119092123748, -0.847638240898971, 0.45958816298636 ], [ 0.069625290951433, 0.150845296288191, -0.986102436589453], [ 0.642824807985868, 0.637056488671774, 0.425364898033585], [-0.583056527549522, 0.091340353707334, -0.807280636127557], [ 0.428841668977862, 0.494984744257901, -0.755701611682959], [-0.534249581158651, 0.50694100906121 , 0.676452657888051], [ 0.926547434683025, -0.253120445955419, -0.278280238467692], [-0.652680883350534, 0.553561886955904, 0.517278360091146], [-0.246001681222344, -0.957575254780407, -0.150109307732795], [-0.756807878748951, -0.314642649178017, 0.572923937344001], [-0.594957697888294, 0.736323790761684, 0.322261714886838], [ 0.475219376672474, -0.790422301072166, 0.386521836385486], [-0.326838550784899, 0.24724641125665 , -0.912165431180953], [-0.812246410149005, 0.557867371965696, 0.170410576244969], [ 0.682984895725692, -0.239356163110699, -0.690101629755712], [ 0.820499911851931, 0.52352613273552 , 0.229565422034687], [ 0.834133426351029, 0.450520019116361, -0.318202984617187], [ 0.700431269593958, 0.633771608897022, -0.328221852305839], [ 0.896963419643415, 0.219139814479089, -0.383971829086017], [-0.361959182023341, -0.012138899060303, -0.93211490583436 ], [-0.952270951104356, 0.301237645134509, 0.049355008222273], [-0.580344573715604, 0.541635733813882, -0.608137244061529], [-0.933770139873584, -0.324582648866844, -0.150729658445346], [ 0.091586054256238, 0.958040446209228, -0.271607249706258], [-0.115254199470393, 0.347141790859843, -0.930703522365237], [-0.392508793291143, -0.262889418835746, 0.881377331597163], [-0.787498756605444, -0.61490295273168 , 0.041714111122517], [-0.333284272342984, -0.686300943338675, -0.646461606734115], [-0.806853601862793, 0.138876647553989, -0.574195560697923], [-0.548658419730724, -0.376148164915614, -0.746650183478984], [-0.658860291754709, -0.045477056523191, 0.75088944144853 ], [ 0.090337586151915, -0.720151444779003, 0.687910617094227], [ 0.9822491487486 , -0.128545812014991, 0.136611068351202], [ 0.00400643269962 , -0.863134675190444, 0.504957900206453], [ 0.121214219865543, -0.111141996522891, -0.98638459513077 ], [-0.368432367737374, 0.65961613706041 , -0.655106206758061], [ 0.648853573418921, 0.51444696364883 , -0.560654405007216], [-0.595764367980407, 0.795313141594413, -0.111990288204524], [ 0.042556073650169, -0.416223007411055, -0.908266144198483], [-0.815381577858629, 0.441231335005946, -0.374790330048206], [ 0.583732709209401, 0.537963423521949, 0.608154157390704], [ 0.991871494815841, -0.027534433284753, -0.12422879197463 ], [ 0.321429215637727, 0.049548720206305, -0.945636390829178], [ 0.156269467318464, -0.941925754615422, 0.29724691146616 ], [-0.308200027756246, -0.528896682426685, -0.790747141764733], [-0.801812057457875, -0.40653096475982 , -0.437984017067539], [ 0.142795148867885, -0.041914411519452, 0.988864362572935], [ 0.451253302588554, 0.89238686633079 , -0.004017176027518], [ 0.357123867119546, 0.907317373154443, -0.221895759098057], [ 0.418571566177936, 0.784257051876754, 0.457972401536307], [-0.171633173884482, 0.983665701002221, -0.05425718656684 ], [-0.547851862030155, 0.307914357079519, -0.777847726727024], [ 0.970630545492926, 0.187443689982203, -0.150801880764005], [ 0.063542880787338, 0.5814761452615 , 0.811078168115178], [-0.665597202973365, 0.737939602545095, 0.111469755492781], [ 0.749815475143678, 0.253926654004704, 0.610981184342887], [ 0.905495239156663, 0.417306235511134, -0.077031666658221], [-0.355567326884387, 0.485473326495045, -0.798678612029975], [-0.048610499660401, 0.733659134899369, 0.677776728061296], [ 0.332341034425014, -0.909397902532474, -0.250089767297361], [-0.509474430367807, -0.77377842904384 , 0.376434253951266], [ 0.192118748200356, 0.732004758162878, -0.65365083998786 ], [-0.058404165233372, 0.935860251422498, 0.34749754429465 ], [ 0.27869946224737 , 0.325628703868133, 0.90348910173847 ], [-0.206033224245319, 0.845462201932611, 0.492690547514704], [ 0.589516706783643, 0.283002370442149, -0.756557803969461], [-0.422242397414651, 0.902059113141648, 0.089446711642357], [ 0.455257210411192, 0.663420040464546, -0.593813710079718], [ 0.387279950595659, -0.90445193867345 , 0.178832129374062], [ 0.498911041040294, -0.064747170025559, -0.864231205813451], [-0.726167040060172, 0.343244422860589, -0.595705209063468], [-0.718134917572004, -0.061819148330639, -0.693152676589743], [-0.10844275388053 , -0.216600068135256, -0.970218830787472], [-0.442790077815399, 0.760884008452726, 0.474333714455493], [ 0.985444449918194, 0.106364707537872, 0.132611406431764], [ 0.159266331638951, 0.362203309863603, -0.918391527579671], [ 0.060441822344003, -0.990235616398184, -0.125619306351547], [-0.917838074966329, 0.04593709445514 , 0.39428803113349 ], [-0.140244059367101, -0.988675437383499, 0.053408644682145], [-0.120361693590686, -0.813981902684486, -0.568283841771104], [-0.874188873243249, 0.481771110302902, -0.060748754515693], [-0.314530281913865, 0.667010376750037, 0.675401998121829], [-0.799441768362303, 0.262703821713619, 0.540258790816784], [ 0.755232819543803, -0.632068898314158, 0.173528948789083], [-0.256946108562324, 0.519581751538401, 0.814870235536288], [ 0.806490204894747, -0.462130316152132, -0.36879414352991 ], [ 0.27376189217876 , -0.822103902407764, 0.499198958368937], [-0.087714032004179, 0.556079780204113, -0.826487463085627], [ 0.478756625711276, -0.318694689049715, -0.818062215549074], [ 0.913546593039296, -0.28551059087732 , 0.289683145597356], [ 0.622534011286955, 0.774128068291346, -0.114791718666822], [-0.911634132049336, 0.035391052742101, -0.409476107567291]])

if calculate_design_score(spherical_design_219202, 2, 19, 202) < 1e-8:
  print("Correct numerical construction of a (2, 19, 202) design")


spherical_design_219204 = np.array([[ 0.112759927588285, 0.964647523829879, 0.238202337308366], [-0.106390190157405, 0.862383744983765, 0.494949900319262], [-0.555021025226819, -0.683485244134021, -0.474130343478699], [ 0.526163532759333, 0.563492905597028, -0.636889065800346], [ 0.117755991258363, -0.464505854426093, 0.877706008710574], [-0.790670682693614, 0.603103243028739, 0.105386667928312], [ 0.347365257034689, -0.113903154112433, 0.930786468363429], [-0.71704625100351 , -0.45224345886253 , 0.530396576005036], [-0.254872250157132, -0.747606201848897, -0.613290390481464], [ 0.389700237132336, -0.905545987806027, -0.167690754507836], [-0.661815852361742, 0.689386106959361, -0.294527711928974], [ 0.987682292272745, -0.147066010085819, 0.053434803342898], [-0.060275400340307, -0.125259694051812, 0.990291313281027], [-0.072051886044562, 0.992873781397725, 0.094921967586037], [ 0.37422778448854 , -0.774589824357425, 0.50986681527506 ], [-0.665010404563421, 0.742525162593741, -0.080109579561584], [-0.709620092871948, 0.267018289036406, 0.652028033992772], [-0.413185204319234, 0.804974574967569, 0.4257862381377 ], [ 0.180218140919247, -0.66191587086828 , 0.72759109503642 ], [ 0.697384752341372, 0.613193064327301, -0.371010475677787], [-0.227634210501128, 0.543530962818822, 0.807933635063408], [ 0.424417425872679, -0.902548778282589, 0.072632991375921], [-0.452025363187486, 0.842430873620771, -0.293229081446172], [ 0.24364137517129 , 0.254368024353666, -0.935914413015985], [ 0.278871999997162, -0.238759306111362, -0.930174392983812], [ 0.626109024555783, 0.425698340055355, 0.65327514313949 ], [-0.517302881849938, -0.831211168750064, -0.203680439352683], [ 0.932459634312434, -0.348715809527162, -0.094426238746109], [ 0.200173439739681, -0.93312780962215 , -0.298668854975794], [ 0.356161853642229, -0.366721607676112, 0.859455639619392], [ 0.067566194097476, -0.147266462189886, -0.986786399647491], [ 0.31931283397154 , -0.001403629666437, -0.947648322894534], [ 0.923309299755903, 0.212947225066723, 0.319614480774333], [ 0.052368068397051, -0.714898253410003, -0.697264707757168], [-0.348116862346875, -0.167949073893498, -0.922283990280696], [-0.117511628016102, -0.854646963387373, 0.505736675804462], [ 0.352100036014931, 0.926234468480357, -0.134592994012297], [-0.115982668270602, 0.823105355539313, -0.555918694004198], [-0.878803818227338, -0.413443828404482, 0.238260466345757], [-0.76366053467585 , -0.629266159286238, 0.144383823732933], [ 0.884351084005527, -0.331140656836242, -0.329042589353281], [-0.198229702285047, 0.976436901404196, -0.085299253854608], [-0.682395642829361, 0.460071856267667, 0.568040556402381], [-0.451452862972828, -0.740141182608757, 0.498378713751047], [-0.138542759397648, -0.366874325608898, -0.91989626210106 ], [-0.516911246362774, -0.27178254158612 , -0.811749353848017], [ 0.020115269688135, 0.962953004491772, -0.268917993198005], [ 0.675403281311411, 0.115534322353959, 0.728342109143767], [-0.534558250494441, -0.332727490070004, 0.776878300752465], [ 0.07657051299974 , 0.680916410966845, -0.728347305764899], [ 0.311256747792964, 0.94050043553966 , 0.136301752384395], [-0.973108836688369, -0.212650367261497, 0.088538202277659], [ 0.522175302877456, -0.204944461380478, -0.827847039503254], [ 0.554053426450004, -0.359217438405632, 0.750991100203129], [-0.535513945206056, 0.057101791540957, -0.842593733594464], [ 0.864969667643228, -0.493943095839197, 0.088587200711805], [ 0.804963197119934, -0.501893482814104, 0.316444597348702], [-0.896209234936879, -0.443145046379397, -0.020771978318311], [-0.393637179680384, 0.580132451678374, -0.713089131373429], [-0.491354341075118, -0.115504070736628, 0.863266888713974], [-0.870946286179115, -0.275832090290561, 0.406656150275324], [-0.97879254961442 , 0.090153340167418, 0.183949775960615], [-0.949460742831377, 0.024763495281155, -0.312907441783589], [ 0.282870680113705, 0.881711935110187, -0.377582099438333], [-0.502752593524448, 0.58430017923287 , 0.63705033572935 ], [ 0.437507917807215, -0.416142797193432, -0.797127338760892], [-0.147000236576792, -0.08318407909649 , -0.985632456563417], [ 0.840214898009953, -0.02779327873317 , 0.541540819162679], [-0.07368015436388 , 0.42558233664685 , 0.901915134359726], [-0.806251587928431, -0.216885267362271, -0.55038092060325 ], [ 0.302757455073255, -0.722971287846412, -0.621007600877208], [ 0.760979095104706, 0.44876870211749 , 0.468526913648946], [ 0.838180152293597, -0.265688788608659, 0.476301899963401], [ 0.930412604472979, -0.273863350860105, 0.243580069984148], [-0.945296998425997, -0.061720004851547, 0.320318943816823], [ 0.12375640510357 , 0.622999619804214, 0.772370264782149], [ 0.279645103181428, 0.67780330713721 , -0.679986244787724], [-0.546018714043715, -0.791029825916372, 0.275926400376552], [-0.657762648605456, 0.022425183322641, 0.752891366169437], [ 0.120305266938426, 0.879548416586991, 0.460349025877314], [ 0.130527626711281, -0.223102153064626, 0.966016546423015], [ 0.326292669922255, 0.460135263253586, 0.825717041767674], [ 0.990167181046289, 0.11411108898777 , 0.080917321686214], [ 0.822444409878912, 0.040446522576653, -0.567405737960399], [ 0.198526328959382, -0.91843149016183 , 0.342156242949084], [-0.719833697785165, -0.236508416594645, 0.652612608223921], [-0.448902625432258, -0.592058642478856, -0.66929290803513 ], [-0.879924186592497, -0.416518343583802, -0.228573610260981], [ 0.698159829758681, -0.447284591767179, 0.559025353699632], [-0.600741450766155, 0.78580207394259 , 0.147053765401976], [ 0.604314112971707, -0.697166819310206, -0.385697911474912], [ 0.096647936016998, 0.457368251578763, -0.884009874894755], [ 0.861279664187962, -0.174137628394778, -0.477360897469948], [-0.137240206800326, -0.987312497633502, 0.079869629109567], [-0.567440334718344, 0.629128467030707, -0.531233318332281], [ 0.735168153344096, 0.672318024196697, 0.086696370448142], [ 0.375924659027601, 0.835242696963637, 0.401310712421051], [ 0.709218602525501, 0.634126170395952, 0.308047031881074], [ 0.772003398140662, -0.537297241662377, -0.339591559614299], [-0.340955524619514, -0.458908448254903, 0.820458631712598], [ 0.562675045177952, 0.825903442391331, -0.035781243412434], [-0.694824418139447, 0.174245776717058, -0.697751701720209], [-0.112759927588285, -0.964647523829879, -0.238202337308366], [ 0.106390190157405, -0.862383744983765, -0.494949900319262], [ 0.555021025226819, 0.683485244134021, 0.474130343478699], [-0.526163532759333, -0.563492905597028, 0.636889065800346], [-0.117755991258363, 0.464505854426093, -0.877706008710574], [ 0.790670682693614, -0.603103243028739, -0.105386667928312], [-0.347365257034689, 0.113903154112433, -0.930786468363429], [ 0.71704625100351 , 0.45224345886253 , -0.530396576005036], [ 0.254872250157132, 0.747606201848897, 0.613290390481464], [-0.389700237132336, 0.905545987806027, 0.167690754507836], [ 0.661815852361742, -0.689386106959361, 0.294527711928974], [-0.987682292272745, 0.147066010085819, -0.053434803342898], [ 0.060275400340307, 0.125259694051812, -0.990291313281027], [ 0.072051886044562, -0.992873781397725, -0.094921967586037], [-0.37422778448854 , 0.774589824357425, -0.50986681527506 ], [ 0.665010404563421, -0.742525162593741, 0.080109579561584], [ 0.709620092871948, -0.267018289036406, -0.652028033992772], [ 0.413185204319234, -0.804974574967569, -0.4257862381377 ], [-0.180218140919247, 0.66191587086828 , -0.72759109503642 ], [-0.697384752341372, -0.613193064327301, 0.371010475677787], [ 0.227634210501128, -0.543530962818822, -0.807933635063408], [-0.424417425872679, 0.902548778282589, -0.072632991375921], [ 0.452025363187486, -0.842430873620771, 0.293229081446172], [-0.24364137517129 , -0.254368024353666, 0.935914413015985], [-0.278871999997162, 0.238759306111362, 0.930174392983812], [-0.626109024555783, -0.425698340055355, -0.65327514313949 ], [ 0.517302881849938, 0.831211168750064, 0.203680439352683], [-0.932459634312434, 0.348715809527162, 0.094426238746109], [-0.200173439739681, 0.93312780962215 , 0.298668854975794], [-0.356161853642229, 0.366721607676112, -0.859455639619392], [-0.067566194097476, 0.147266462189886, 0.986786399647491], [-0.31931283397154 , 0.001403629666437, 0.947648322894534], [-0.923309299755903, -0.212947225066723, -0.319614480774333], [-0.052368068397051, 0.714898253410003, 0.697264707757168], [ 0.348116862346875, 0.167949073893498, 0.922283990280696], [ 0.117511628016102, 0.854646963387373, -0.505736675804462], [-0.352100036014931, -0.926234468480357, 0.134592994012297], [ 0.115982668270602, -0.823105355539313, 0.555918694004198], [ 0.878803818227338, 0.413443828404482, -0.238260466345757], [ 0.76366053467585 , 0.629266159286238, -0.144383823732933], [-0.884351084005527, 0.331140656836242, 0.329042589353281], [ 0.198229702285047, -0.976436901404196, 0.085299253854608], [ 0.682395642829361, -0.460071856267667, -0.568040556402381], [ 0.451452862972828, 0.740141182608757, -0.498378713751047], [ 0.138542759397648, 0.366874325608898, 0.91989626210106 ], [ 0.516911246362774, 0.27178254158612 , 0.811749353848017], [-0.020115269688135, -0.962953004491772, 0.268917993198005], [-0.675403281311411, -0.115534322353959, -0.728342109143767], [ 0.534558250494441, 0.332727490070004, -0.776878300752465], [-0.07657051299974 , -0.680916410966845, 0.728347305764899], [-0.311256747792964, -0.94050043553966 , -0.136301752384395], [ 0.973108836688369, 0.212650367261497, -0.088538202277659], [-0.522175302877456, 0.204944461380478, 0.827847039503254], [-0.554053426450004, 0.359217438405632, -0.750991100203129], [ 0.535513945206056, -0.057101791540957, 0.842593733594464], [-0.864969667643228, 0.493943095839197, -0.088587200711805], [-0.804963197119934, 0.501893482814104, -0.316444597348702], [ 0.896209234936879, 0.443145046379397, 0.020771978318311], [ 0.393637179680384, -0.580132451678374, 0.713089131373429], [ 0.491354341075118, 0.115504070736628, -0.863266888713974], [ 0.870946286179115, 0.275832090290561, -0.406656150275324], [ 0.97879254961442 , -0.090153340167418, -0.183949775960615], [ 0.949460742831377, -0.024763495281155, 0.312907441783589], [-0.282870680113705, -0.881711935110187, 0.377582099438333], [ 0.502752593524448, -0.58430017923287 , -0.63705033572935 ], [-0.437507917807215, 0.416142797193432, 0.797127338760892], [ 0.147000236576792, 0.08318407909649 , 0.985632456563417], [-0.840214898009953, 0.02779327873317 , -0.541540819162679], [ 0.07368015436388 , -0.42558233664685 , -0.901915134359726], [ 0.806251587928431, 0.216885267362271, 0.55038092060325 ], [-0.302757455073255, 0.722971287846412, 0.621007600877208], [-0.760979095104706, -0.44876870211749 , -0.468526913648946], [-0.838180152293597, 0.265688788608659, -0.476301899963401], [-0.930412604472979, 0.273863350860105, -0.243580069984148], [ 0.945296998425997, 0.061720004851547, -0.320318943816823], [-0.12375640510357 , -0.622999619804214, -0.772370264782149], [-0.279645103181428, -0.67780330713721 , 0.679986244787724], [ 0.546018714043715, 0.791029825916372, -0.275926400376552], [ 0.657762648605456, -0.022425183322641, -0.752891366169437], [-0.120305266938426, -0.879548416586991, -0.460349025877314], [-0.130527626711281, 0.223102153064626, -0.966016546423015], [-0.326292669922255, -0.460135263253586, -0.825717041767674], [-0.990167181046289, -0.11411108898777 , -0.080917321686214], [-0.822444409878912, -0.040446522576653, 0.567405737960399], [-0.198526328959382, 0.91843149016183 , -0.342156242949084], [ 0.719833697785165, 0.236508416594645, -0.652612608223921], [ 0.448902625432258, 0.592058642478856, 0.66929290803513 ], [ 0.879924186592497, 0.416518343583802, 0.228573610260981], [-0.698159829758681, 0.447284591767179, -0.559025353699632], [ 0.600741450766155, -0.78580207394259 , -0.147053765401976], [-0.604314112971707, 0.697166819310206, 0.385697911474912], [-0.096647936016998, -0.457368251578763, 0.884009874894755], [-0.861279664187962, 0.174137628394778, 0.477360897469948], [ 0.137240206800326, 0.987312497633502, -0.079869629109567], [ 0.567440334718344, -0.629128467030707, 0.531233318332281], [-0.735168153344096, -0.672318024196697, -0.086696370448142], [-0.375924659027601, -0.835242696963637, -0.401310712421051], [-0.709218602525501, -0.634126170395952, -0.308047031881074], [-0.772003398140662, 0.537297241662377, 0.339591559614299], [ 0.340955524619514, 0.458908448254903, -0.820458631712598], [-0.562675045177952, -0.825903442391331, 0.035781243412434], [ 0.694824418139447, -0.174245776717058, 0.697751701720209]])

if calculate_design_score(spherical_design_219204, 2, 19, 204) < 1e-8:
  print("Correct numerical construction of a (2, 19, 204) design")


spherical_design_221234 = np.array([[-0.95968079966586 , -0.281052917216377, 0.004692598092644], [-0.759530498390543, 0.286342093718494, -0.584056185122222], [-0.813731112359348, -0.376438350731981, 0.442872266998741], [-0.934789340375849, 0.080320817818877, 0.346002103092729], [-0.268299764488768, 0.877753029851228, -0.396944397821988], [ 0.634252954137002, 0.732366781248964, -0.247713721645609], [-0.635809806972457, 0.109454765704434, -0.764045511486217], [-0.65631649825689 , -0.754074737142177, 0.024898693133797], [-0.678644543312489, 0.731087986006376, -0.070370025929536], [-0.481184455376351, 0.631126972778964, 0.608391538513666], [-0.450035573968919, 0.038783895719096, -0.892168028790158], [ 0.622851094775359, -0.665304520019097, 0.41161439403803 ], [ 0.607622312531988, 0.768421090619635, 0.200808746831928], [-0.473470779179603, -0.880798466011383, -0.004414015751438], [-0.468263753436675, -0.829620727589547, 0.304069902442164], [ 0.084470612987351, -0.978420029898558, -0.188570837180212], [ 0.296550247071693, 0.83395075733099 , -0.465385953063462], [ 0.169564216633694, 0.658216642891649, 0.733484033539824], [-0.926734525130241, 0.165292420325997, -0.337404113363191], [ 0.91553687452995 , 0.359725407569599, 0.179971838143685], [-0.262723280134452, 0.900873002410353, 0.345549289687521], [ 0.852910794424532, -0.521780032040217, 0.016993378658282], [ 0.815495358564018, -0.219953119320268, -0.535339093903881], [-0.006638053941181, 0.537349537370533, 0.843333511090095], [-0.748306244547025, 0.618255331669059, -0.24041237329786 ], [ 0.309552428639459, -0.177928376211632, 0.934087140936061], [ 0.641221631664846, 0.59207540958216 , -0.488140889962304], [-0.631166728183411, 0.519089640594711, -0.576345821761122], [-0.522144999960346, -0.531744621093395, -0.666792514171126], [-0.524989176541228, -0.693800956090938, -0.49297727923492 ], [-0.401606547392641, 0.353620265122791, -0.844786889804671], [-0.513412441511633, 0.441147294839385, 0.736068426952928], [ 0.225009832078759, 0.871096927128832, 0.436532609337029], [ 0.014953263918033, 0.994730273680933, -0.101430185452132], [-0.66385985595224 , -0.404956106303162, 0.628729388229033], [-0.439297699116356, -0.84067841186116 , -0.316665974461536], [-0.26659355768791 , -0.65003806036958 , 0.711602694676079], [-0.89848755235775 , -0.408123996978009, 0.161724832196152], [ 0.3614067704148 , -0.217088775659188, -0.906784213460479], [ 0.848619601883189, -0.097968921962391, 0.519852730712405], [ 0.127846865980104, -0.674209830659184, -0.727390048805714], [ 0.518428207576108, -0.823401565002646, 0.230742402563156], [ 0.965056871867185, 0.098101242619735, 0.24298432101349 ], [-0.934683028372294, 0.351575637228494, 0.052556710134627], [ 0.707789749366661, 0.470433278269278, 0.526997344763993], [ 0.35461179026488 , 0.32787342314871 , 0.875642333717302], [ 0.169263126412443, -0.958490636454655, 0.22944649455992 ], [ 0.897204536050857, 0.125533041556993, 0.423397538924374], [ 0.053405023330982, 0.798962286212656, 0.599005149137187], [-0.214392991657586, 0.976527526721527, 0.02072763091258 ], [-0.701173022143272, -0.61941510739519 , -0.353102418214709], [-0.337319106238865, -0.738300741067728, -0.584061500447562], [ 0.855454843104286, 0.032835154741454, -0.516835432243694], [-0.563453613197477, 0.253554274935478, 0.786276195389795], [ 0.89684946354039 , -0.312628551257369, -0.312928791717914], [ 0.234619802773941, -0.516212672722683, 0.823697775077013], [-0.820468425679856, 0.484230450469671, 0.303895431522683], [ 0.106165249286681, -0.383417549463431, 0.917452953893198], [ 0.46062638190961 , -0.559461685165811, 0.68907616351191 ], [-0.128451572202936, -0.39014055915067 , -0.911751357390926], [-0.99818578240369 , -0.037770749322386, -0.046888317337679], [ 0.80179929349083 , -0.423337883189688, 0.421785407066295], [-0.804566033787352, -0.570902199371167, -0.163536466969465], [-0.081848194665441, 0.168305121722016, -0.982331033324381], [-0.76940454523402 , -0.183197959025899, 0.611927408752032], [ 0.977185629162279, 0.100624330556218, -0.18703740336851 ], [ 0.554440966673463, 0.289944664324093, -0.780081602208503], [-0.75834635262225 , 0.633333593764864, 0.154270439402406], [-0.008491106926242, -0.913495136395269, 0.40676103167014 ], [-0.33408502422444 , -0.526462997983736, -0.781808101993662], [-0.84900082306119 , -0.371978116586729, -0.375273078200411], [-0.773055448850525, -0.561233072538487, 0.29563949548652 ], [ 0.097533512908734, 0.552265684141076, -0.82794313088513 ], [ 0.396487961581455, -0.905217150893539, -0.152902603147124], [-0.156942025448506, -0.117851444932517, 0.980550986729114], [ 0.716329788343574, -0.390820365939193, -0.578040721661377], [ 0.1354827949157 , 0.786093007288871, -0.603077272141282], [ 0.591460836034874, 0.32861537593873 , 0.73633281478797 ], [ 0.295376160857764, -0.738385899836039, -0.606250102284735], [ 0.063137945916516, -0.661448390688032, 0.747328325598365], [-0.105956818842678, 0.810680444812803, -0.575821473148454], [-0.740300949246364, -0.04845256448884 , -0.670527295148667], [-0.395898731497225, 0.910000754529212, -0.123137407619101], [-0.459229119717476, -0.535017994533705, 0.70912929789189 ], [-0.173118341352334, -0.943623524038699, 0.282143021831487], [-0.111879104107783, -0.047328841803889, -0.992594099719287], [-0.668374557660195, 0.565043064170959, 0.48373731125977 ], [ 0.971214627743025, -0.191147875764232, 0.142143014069633], [ 0.442736949914095, -0.022757225887771, -0.896362706637585], [-0.814066372714852, -0.579399234241068, 0.039905741137687], [-0.335921238892307, -0.931700287257657, 0.138171979739229], [-0.069239016401664, -0.945856249511274, -0.317114985246988], [ 0.543914376852561, 0.154215723488594, 0.824848265611306], [ 0.883236690938167, -0.407078629293189, 0.232765842325235], [-0.148378345187521, -0.987793112745917, -0.047419754229064], [ 0.168446477133938, -0.080997375678236, -0.982377325407302], [-0.565717695085367, 0.819367798593091, 0.092735646307188], [-0.305643046310456, 0.69908931793727 , -0.646418172538541], [-0.33674440610552 , -0.406298431524696, 0.849426153056999], [ 0.21057652161176 , -0.378369747359299, -0.901384414569695], [-0.11220228583309 , 0.864104841450705, 0.490645972199183], [ 0.585835521585389, -0.316343479846518, 0.746139091863823], [-0.624961192944043, -0.07181621891027 , 0.77734544316886 ], [ 0.304518598517172, 0.936557712301829, 0.173574406768688], [-0.769983175875939, -0.243176582438041, -0.589907669573601], [ 0.912814934858053, 0.2249169811309 , -0.340853702193552], [ 0.702700417028564, -0.108557996422875, -0.703155235577844], [-0.020762580020667, 0.247893405659036, 0.968564801498408], [ 0.471490505794945, 0.702378564068787, -0.533255150637943], [-0.435888809888133, 0.769198774735496, -0.467262442701871], [ 0.617520443715174, -0.719419534767414, -0.317968606294478], [-0.282586943274157, -0.138319754741044, -0.949216658587146], [-0.308452269253734, 0.518848529588824, 0.797278747324759], [-0.448162089696472, 0.794752660181282, 0.409290789651646], [-0.982371608678594, 0.146985128931625, 0.115504953725736], [ 0.131286358448062, 0.348547842579634, -0.928050803306872], [-0.373560644298026, -0.179315857590749, 0.910108932077989], [ 0.95968079966586 , 0.281052917216377, -0.004692598092644], [ 0.759530498390543, -0.286342093718494, 0.584056185122222], [ 0.813731112359348, 0.376438350731981, -0.442872266998741], [ 0.934789340375849, -0.080320817818877, -0.346002103092729], [ 0.268299764488768, -0.877753029851228, 0.396944397821988], [-0.634252954137002, -0.732366781248964, 0.247713721645609], [ 0.635809806972457, -0.109454765704434, 0.764045511486217], [ 0.65631649825689 , 0.754074737142177, -0.024898693133797], [ 0.678644543312489, -0.731087986006376, 0.070370025929536], [ 0.481184455376351, -0.631126972778964, -0.608391538513666], [ 0.450035573968919, -0.038783895719096, 0.892168028790158], [-0.622851094775359, 0.665304520019097, -0.41161439403803 ], [-0.607622312531988, -0.768421090619635, -0.200808746831928], [ 0.473470779179603, 0.880798466011383, 0.004414015751438], [ 0.468263753436675, 0.829620727589547, -0.304069902442164], [-0.084470612987351, 0.978420029898558, 0.188570837180212], [-0.296550247071693, -0.83395075733099 , 0.465385953063462], [-0.169564216633694, -0.658216642891649, -0.733484033539824], [ 0.926734525130241, -0.165292420325997, 0.337404113363191], [-0.91553687452995 , -0.359725407569599, -0.179971838143685], [ 0.262723280134452, -0.900873002410353, -0.345549289687521], [-0.852910794424532, 0.521780032040217, -0.016993378658282], [-0.815495358564018, 0.219953119320268, 0.535339093903881], [ 0.006638053941181, -0.537349537370533, -0.843333511090095], [ 0.748306244547025, -0.618255331669059, 0.24041237329786 ], [-0.309552428639459, 0.177928376211632, -0.934087140936061], [-0.641221631664846, -0.59207540958216 , 0.488140889962304], [ 0.631166728183411, -0.519089640594711, 0.576345821761122], [ 0.522144999960346, 0.531744621093395, 0.666792514171126], [ 0.524989176541228, 0.693800956090938, 0.49297727923492 ], [ 0.401606547392641, -0.353620265122791, 0.844786889804671], [ 0.513412441511633, -0.441147294839385, -0.736068426952928], [-0.225009832078759, -0.871096927128832, -0.436532609337029], [-0.014953263918033, -0.994730273680933, 0.101430185452132], [ 0.66385985595224 , 0.404956106303162, -0.628729388229033], [ 0.439297699116356, 0.84067841186116 , 0.316665974461536], [ 0.26659355768791 , 0.65003806036958 , -0.711602694676079], [ 0.89848755235775 , 0.408123996978009, -0.161724832196152], [-0.3614067704148 , 0.217088775659188, 0.906784213460479], [-0.848619601883189, 0.097968921962391, -0.519852730712405], [-0.127846865980104, 0.674209830659184, 0.727390048805714], [-0.518428207576108, 0.823401565002646, -0.230742402563156], [-0.965056871867185, -0.098101242619735, -0.24298432101349 ], [ 0.934683028372294, -0.351575637228494, -0.052556710134627], [-0.707789749366661, -0.470433278269278, -0.526997344763993], [-0.35461179026488 , -0.32787342314871 , -0.875642333717302], [-0.169263126412443, 0.958490636454655, -0.22944649455992 ], [-0.897204536050857, -0.125533041556993, -0.423397538924374], [-0.053405023330982, -0.798962286212656, -0.599005149137187], [ 0.214392991657586, -0.976527526721527, -0.02072763091258 ], [ 0.701173022143272, 0.61941510739519 , 0.353102418214709], [ 0.337319106238865, 0.738300741067728, 0.584061500447562], [-0.855454843104286, -0.032835154741454, 0.516835432243694], [ 0.563453613197477, -0.253554274935478, -0.786276195389795], [-0.89684946354039 , 0.312628551257369, 0.312928791717914], [-0.234619802773941, 0.516212672722683, -0.823697775077013], [ 0.820468425679856, -0.484230450469671, -0.303895431522683], [-0.106165249286681, 0.383417549463431, -0.917452953893198], [-0.46062638190961 , 0.559461685165811, -0.68907616351191 ], [ 0.128451572202936, 0.39014055915067 , 0.911751357390926], [ 0.99818578240369 , 0.037770749322386, 0.046888317337679], [-0.80179929349083 , 0.423337883189688, -0.421785407066295], [ 0.804566033787352, 0.570902199371167, 0.163536466969465], [ 0.081848194665441, -0.168305121722016, 0.982331033324381], [ 0.76940454523402 , 0.183197959025899, -0.611927408752032], [-0.977185629162279, -0.100624330556218, 0.18703740336851 ], [-0.554440966673463, -0.289944664324093, 0.780081602208503], [ 0.75834635262225 , -0.633333593764864, -0.154270439402406], [ 0.008491106926242, 0.913495136395269, -0.40676103167014 ], [ 0.33408502422444 , 0.526462997983736, 0.781808101993662], [ 0.84900082306119 , 0.371978116586729, 0.375273078200411], [ 0.773055448850525, 0.561233072538487, -0.29563949548652 ], [-0.097533512908734, -0.552265684141076, 0.82794313088513 ], [-0.396487961581455, 0.905217150893539, 0.152902603147124], [ 0.156942025448506, 0.117851444932517, -0.980550986729114], [-0.716329788343574, 0.390820365939193, 0.578040721661377], [-0.1354827949157 , -0.786093007288871, 0.603077272141282], [-0.591460836034874, -0.32861537593873 , -0.73633281478797 ], [-0.295376160857764, 0.738385899836039, 0.606250102284735], [-0.063137945916516, 0.661448390688032, -0.747328325598365], [ 0.105956818842678, -0.810680444812803, 0.575821473148454], [ 0.740300949246364, 0.04845256448884 , 0.670527295148667], [ 0.395898731497225, -0.910000754529212, 0.123137407619101], [ 0.459229119717476, 0.535017994533705, -0.70912929789189 ], [ 0.173118341352334, 0.943623524038699, -0.282143021831487], [ 0.111879104107783, 0.047328841803889, 0.992594099719287], [ 0.668374557660195, -0.565043064170959, -0.48373731125977 ], [-0.971214627743025, 0.191147875764232, -0.142143014069633], [-0.442736949914095, 0.022757225887771, 0.896362706637585], [ 0.814066372714852, 0.579399234241068, -0.039905741137687], [ 0.335921238892307, 0.931700287257657, -0.138171979739229], [ 0.069239016401664, 0.945856249511274, 0.317114985246988], [-0.543914376852561, -0.154215723488594, -0.824848265611306], [-0.883236690938167, 0.407078629293189, -0.232765842325235], [ 0.148378345187521, 0.987793112745917, 0.047419754229064], [-0.168446477133938, 0.080997375678236, 0.982377325407302], [ 0.565717695085367, -0.819367798593091, -0.092735646307188], [ 0.305643046310456, -0.69908931793727 , 0.646418172538541], [ 0.33674440610552 , 0.406298431524696, -0.849426153056999], [-0.21057652161176 , 0.378369747359299, 0.901384414569695], [ 0.11220228583309 , -0.864104841450705, -0.490645972199183], [-0.585835521585389, 0.316343479846518, -0.746139091863823], [ 0.624961192944043, 0.07181621891027 , -0.77734544316886 ], [-0.304518598517172, -0.936557712301829, -0.173574406768688], [ 0.769983175875939, 0.243176582438041, 0.589907669573601], [-0.912814934858053, -0.2249169811309 , 0.340853702193552], [-0.702700417028564, 0.108557996422875, 0.703155235577844], [ 0.020762580020667, -0.247893405659036, -0.968564801498408], [-0.471490505794945, -0.702378564068787, 0.533255150637943], [ 0.435888809888133, -0.769198774735496, 0.467262442701871], [-0.617520443715174, 0.719419534767414, 0.317968606294478], [ 0.282586943274157, 0.138319754741044, 0.949216658587146], [ 0.308452269253734, -0.518848529588824, -0.797278747324759], [ 0.448162089696472, -0.794752660181282, -0.409290789651646], [ 0.982371608678594, -0.146985128931625, -0.115504953725736], [-0.131286358448062, -0.348547842579634, 0.928050803306872], [ 0.373560644298026, 0.179315857590749, -0.910108932077989]])

if calculate_design_score(spherical_design_221234, 2, 21, 234) < 1e-8:
  print("Correct numerical construction of a (2, 21, 234) design")


spherical_design_221236 = np.array([[-0.130049733548272, -0.198315377061914, 0.971472119015679], [-0.467158138795434, 0.773051280945484, -0.429132835361887], [ 0.756728007883333, 0.609946607167402, 0.235218746042695], [-0.522277828130915, -0.16609323311722 , 0.836444205046292], [-0.769430975461327, 0.316667059251264, -0.554705279933216], [ 0.075036767649907, 0.816161473445561, -0.572931001747876], [-0.404835642382793, -0.699093753615368, -0.589386143638011], [-0.738140995298722, -0.577443767715882, 0.348864681768588], [-0.343978588609081, 0.033629820890114, -0.938375066657999], [-0.247090735607287, -0.391355614096429, -0.886446248619879], [-0.782118640227369, -0.399114952538716, 0.478537028106404], [-0.561453902184243, -0.688345768846834, 0.459292519243181], [ 0.549986401597466, -0.023597920701215, 0.834840162064841], [ 0.313269942444699, -0.549242452115239, 0.774722319257122], [-0.901020772076067, 0.281711301313467, -0.329848921477276], [-0.403397093096616, -0.547480181718345, -0.733168627197624], [ 0.351893458507228, -0.755125615293721, -0.553133165690778], [-0.496457929753154, 0.571983375582531, -0.652965957797514], [-0.592896663724166, -0.534784032937118, 0.602062774351866], [-0.293640347277308, 0.954889421438962, 0.044289268167654], [-0.864137527345027, 0.475004612599853, -0.166243652037865], [-0.022285276982973, 0.996971075750867, 0.074512016117887], [-0.039870879239023, -0.968736412149282, 0.244867467755202], [ 0.529034502060967, 0.365567484464862, -0.765821722028788], [-0.879982000914753, 0.097540006539148, -0.464884528878313], [-0.62876376387488 , 0.62104124239213 , -0.467935790986256], [ 0.484018960285602, 0.344529607248678, 0.804378639580339], [ 0.353909047321396, 0.17670057592356 , 0.918436330233263], [-0.718936217398896, 0.135000737750483, -0.68183980238689 ], [ 0.023126092559461, -0.91740731663115 , -0.397276980499203], [ 0.958456814340821, 0.276916940702362, -0.068392565353914], [-0.993615330426117, -0.021691559772247, -0.110716084544408], [-0.139056169511406, 0.067243034993911, 0.98799886435442 ], [ 0.400412292164468, 0.572549989090136, -0.71544147648601 ], [ 0.31467669521971 , 0.931637245782903, -0.181743279808772], [ 0.09121505262507 , -0.886001892054152, 0.454621228553033], [ 0.546864832303891, 0.712711935813963, 0.439295517547757], [ 0.36881108969074 , -0.607950380975723, -0.703117852420633], [ 0.882764802401155, 0.406080560239165, -0.236272897805684], [ 0.917519287606697, -0.351076712313571, -0.186824781388318], [-0.713011183980602, 0.701066278494568, 0.011005665648058], [-0.775342017457766, -0.06165786397076 , -0.628524513265063], [-0.879268715093871, 0.475483109277074, 0.028325597070122], [ 0.963376438124077, -0.226008510753093, 0.144312132319273], [-0.122364269191716, -0.168215453643238, -0.978126038289942], [-0.12591349747165 , 0.690771582013665, 0.712025570212753], [ 0.822316545777494, 0.563813419817197, -0.076876044218004], [-0.684183718603553, -0.726248747787162, -0.066748749317076], [ 0.008346720036828, 0.352458451748178, 0.935790239346353], [-0.862268325859334, -0.205733735608518, -0.4627817674156 ], [ 0.159050009296946, 0.573972679329501, 0.803279812970521], [-0.480562626743567, 0.308398666217106, 0.820944470992304], [ 0.109934857083254, -0.503506056471188, 0.856969064957958], [ 0.719309935700241, 0.287660224605597, -0.632332832915367], [ 0.812947467827785, -0.128470438145846, -0.567989226196243], [ 0.143874705481766, 0.726194953173484, 0.672265542109587], [ 0.329818889571066, 0.100496026929846, -0.938679950064677], [ 0.377448469110321, -0.911109845950174, 0.165564192321065], [ 0.67517921278914 , 0.714681142160659, -0.182657865030155], [-0.609563873655078, -0.17459374911632 , -0.773271560775465], [ 0.352772352231593, -0.353344839490301, 0.866428930672656], [-0.180353969362575, -0.983336214041183, 0.022854668895354], [-0.862981875281451, -0.372411715964795, -0.341426121947146], [-0.891190718999409, 0.233540030286151, 0.388893502932693], [-0.584525105578924, 0.627180177712575, 0.514757637760113], [-0.548384943410661, -0.795945539015767, -0.256407201071627], [-0.610540927462819, -0.494945951370896, -0.618278481846489], [-0.140919172740739, 0.722458892559092, -0.67690097895952 ], [ 0.186156217010011, 0.494018193276855, -0.849289048310405], [-0.21233837634394 , -0.835889235733621, -0.506163609435016], [ 0.20317181574375 , -0.95041898091021 , -0.235425092148192], [ 0.195301135026627, 0.901206849280099, -0.386889753635256], [ 0.772504155888509, -0.485556598310825, 0.409233575078833], [ 0.869309580540478, 0.084414801667176, -0.487006154416995], [ 0.79477384758641 , -0.499140178897338, -0.345244280188722], [-0.057856493767142, -0.645588929188281, 0.76149035492152 ], [-0.480262026987473, 0.849313810847623, 0.219121966350642], [-0.956841620517007, 0.060499096129446, 0.284242805738172], [-0.680192850510711, 0.24902269726147 , 0.689438454369015], [-0.396741584674935, 0.11473338575712 , 0.910731774554029], [ 0.595144038867438, -0.058700146643555, -0.801472311302505], [-0.498303844665939, -0.83094297349584 , 0.247441009513452], [ 0.52308176805482 , -0.243325373103916, 0.816809786139028], [-0.127436523593907, 0.982100647506982, -0.138702020968212], [-0.752088283593831, 0.409229297981217, 0.516618423359723], [ 0.378748491682404, 0.799929260254667, -0.465470470209147], [-0.216060477856299, 0.20742250334793 , -0.954093168937597], [ 0.989090339437212, 0.075013346518626, -0.126780512209319], [-0.511107684306104, -0.859503314219693, 0.004794568657958], [ 0.329350039085926, 0.878823358052719, 0.345250716284618], [-0.063503109414511, 0.037195106667508, -0.997288262808042], [-0.744895383001652, 0.638509282039268, 0.193485826696741], [ 0.314266813377034, 0.352712168324447, -0.881379881961071], [ 0.569290087372375, -0.473862828899887, -0.671835408270918], [-0.120579715538416, -0.961704740161421, -0.246139238951712], [-0.26187757013158 , 0.897251579217503, -0.355471154742093], [-0.722827509691682, -0.317518408463718, -0.613760907454677], [ 0.947102011541918, 0.029884213116786, 0.319538281806175], [-0.535259185701219, 0.8442854136012 , 0.026071910223823], [-0.274582073046366, -0.711261455771267, 0.647079459336872], [ 0.306532761160754, -0.447358396149272, -0.840183391724637], [ 0.724757323227605, 0.07942789118479 , -0.68441071918103 ], [-0.565739665665465, 0.793473207481602, -0.224363320758064], [ 0.353683942716768, 0.927410345459133, 0.121728056748207], [-0.911779475442355, -0.212215774518691, 0.35160013254761 ], [ 0.314947409845297, -0.860960962081638, -0.399442550065963], [-0.954713204461325, -0.245284517535246, -0.16839893908422 ], [-0.588051066433063, 0.727808519824825, 0.352832398933736], [-0.316736133051606, 0.748368949236216, -0.582771085280144], [-0.722955428210676, -0.537784801392322, -0.433731433276559], [-0.019099566709024, 0.340088465110929, -0.940199469501031], [ 0.608331448615299, -0.404846294042449, 0.682665603956803], [-0.077761252733988, 0.823296080208836, 0.562260395089328], [-0.982030085383455, 0.18047311605592 , 0.055193892622567], [-0.203731650875146, 0.26548316160085 , 0.942343942166609], [-0.88185417506372 , -0.461268419580393, -0.097799074742515], [ 0.082358777248902, -0.533753109180482, -0.841620252994327], [ 0.727088618637696, -0.651694509913794, 0.21595463967173 ], [ 0.130049733548272, 0.198315377061914, -0.971472119015679], [ 0.467158138795434, -0.773051280945484, 0.429132835361887], [-0.756728007883333, -0.609946607167402, -0.235218746042695], [ 0.522277828130915, 0.16609323311722 , -0.836444205046292], [ 0.769430975461327, -0.316667059251264, 0.554705279933216], [-0.075036767649907, -0.816161473445561, 0.572931001747876], [ 0.404835642382793, 0.699093753615368, 0.589386143638011], [ 0.738140995298722, 0.577443767715882, -0.348864681768588], [ 0.343978588609081, -0.033629820890114, 0.938375066657999], [ 0.247090735607287, 0.391355614096429, 0.886446248619879], [ 0.782118640227369, 0.399114952538716, -0.478537028106404], [ 0.561453902184243, 0.688345768846834, -0.459292519243181], [-0.549986401597466, 0.023597920701215, -0.834840162064841], [-0.313269942444699, 0.549242452115239, -0.774722319257122], [ 0.901020772076067, -0.281711301313467, 0.329848921477276], [ 0.403397093096616, 0.547480181718345, 0.733168627197624], [-0.351893458507228, 0.755125615293721, 0.553133165690778], [ 0.496457929753154, -0.571983375582531, 0.652965957797514], [ 0.592896663724166, 0.534784032937118, -0.602062774351866], [ 0.293640347277308, -0.954889421438962, -0.044289268167654], [ 0.864137527345027, -0.475004612599853, 0.166243652037865], [ 0.022285276982973, -0.996971075750867, -0.074512016117887], [ 0.039870879239023, 0.968736412149282, -0.244867467755202], [-0.529034502060967, -0.365567484464862, 0.765821722028788], [ 0.879982000914753, -0.097540006539148, 0.464884528878313], [ 0.62876376387488 , -0.62104124239213 , 0.467935790986256], [-0.484018960285602, -0.344529607248678, -0.804378639580339], [-0.353909047321396, -0.17670057592356 , -0.918436330233263], [ 0.718936217398896, -0.135000737750483, 0.68183980238689 ], [-0.023126092559461, 0.91740731663115 , 0.397276980499203], [-0.958456814340821, -0.276916940702362, 0.068392565353914], [ 0.993615330426117, 0.021691559772247, 0.110716084544408], [ 0.139056169511406, -0.067243034993911, -0.98799886435442 ], [-0.400412292164468, -0.572549989090136, 0.71544147648601 ], [-0.31467669521971 , -0.931637245782903, 0.181743279808772], [-0.09121505262507 , 0.886001892054152, -0.454621228553033], [-0.546864832303891, -0.712711935813963, -0.439295517547757], [-0.36881108969074 , 0.607950380975723, 0.703117852420633], [-0.882764802401155, -0.406080560239165, 0.236272897805684], [-0.917519287606697, 0.351076712313571, 0.186824781388318], [ 0.713011183980602, -0.701066278494568, -0.011005665648058], [ 0.775342017457766, 0.06165786397076 , 0.628524513265063], [ 0.879268715093871, -0.475483109277074, -0.028325597070122], [-0.963376438124077, 0.226008510753093, -0.144312132319273], [ 0.122364269191716, 0.168215453643238, 0.978126038289942], [ 0.12591349747165 , -0.690771582013665, -0.712025570212753], [-0.822316545777494, -0.563813419817197, 0.076876044218004], [ 0.684183718603553, 0.726248747787162, 0.066748749317076], [-0.008346720036828, -0.352458451748178, -0.935790239346353], [ 0.862268325859334, 0.205733735608518, 0.4627817674156 ], [-0.159050009296946, -0.573972679329501, -0.803279812970521], [ 0.480562626743567, -0.308398666217106, -0.820944470992304], [-0.109934857083254, 0.503506056471188, -0.856969064957958], [-0.719309935700241, -0.287660224605597, 0.632332832915367], [-0.812947467827785, 0.128470438145846, 0.567989226196243], [-0.143874705481766, -0.726194953173484, -0.672265542109587], [-0.329818889571066, -0.100496026929846, 0.938679950064677], [-0.377448469110321, 0.911109845950174, -0.165564192321065], [-0.67517921278914 , -0.714681142160659, 0.182657865030155], [ 0.609563873655078, 0.17459374911632 , 0.773271560775465], [-0.352772352231593, 0.353344839490301, -0.866428930672656], [ 0.180353969362575, 0.983336214041183, -0.022854668895354], [ 0.862981875281451, 0.372411715964795, 0.341426121947146], [ 0.891190718999409, -0.233540030286151, -0.388893502932693], [ 0.584525105578924, -0.627180177712575, -0.514757637760113], [ 0.548384943410661, 0.795945539015767, 0.256407201071627], [ 0.610540927462819, 0.494945951370896, 0.618278481846489], [ 0.140919172740739, -0.722458892559092, 0.67690097895952 ], [-0.186156217010011, -0.494018193276855, 0.849289048310405], [ 0.21233837634394 , 0.835889235733621, 0.506163609435016], [-0.20317181574375 , 0.95041898091021 , 0.235425092148192], [-0.195301135026627, -0.901206849280099, 0.386889753635256], [-0.772504155888509, 0.485556598310825, -0.409233575078833], [-0.869309580540478, -0.084414801667176, 0.487006154416995], [-0.79477384758641 , 0.499140178897338, 0.345244280188722], [ 0.057856493767142, 0.645588929188281, -0.76149035492152 ], [ 0.480262026987473, -0.849313810847623, -0.219121966350642], [ 0.956841620517007, -0.060499096129446, -0.284242805738172], [ 0.680192850510711, -0.24902269726147 , -0.689438454369015], [ 0.396741584674935, -0.11473338575712 , -0.910731774554029], [-0.595144038867438, 0.058700146643555, 0.801472311302505], [ 0.498303844665939, 0.83094297349584 , -0.247441009513452], [-0.52308176805482 , 0.243325373103916, -0.816809786139028], [ 0.127436523593907, -0.982100647506982, 0.138702020968212], [ 0.752088283593831, -0.409229297981217, -0.516618423359723], [-0.378748491682404, -0.799929260254667, 0.465470470209147], [ 0.216060477856299, -0.20742250334793 , 0.954093168937597], [-0.989090339437212, -0.075013346518626, 0.126780512209319], [ 0.511107684306104, 0.859503314219693, -0.004794568657958], [-0.329350039085926, -0.878823358052719, -0.345250716284618], [ 0.063503109414511, -0.037195106667508, 0.997288262808042], [ 0.744895383001652, -0.638509282039268, -0.193485826696741], [-0.314266813377034, -0.352712168324447, 0.881379881961071], [-0.569290087372375, 0.473862828899887, 0.671835408270918], [ 0.120579715538416, 0.961704740161421, 0.246139238951712], [ 0.26187757013158 , -0.897251579217503, 0.355471154742093], [ 0.722827509691682, 0.317518408463718, 0.613760907454677], [-0.947102011541918, -0.029884213116786, -0.319538281806175], [ 0.535259185701219, -0.8442854136012 , -0.026071910223823], [ 0.274582073046366, 0.711261455771267, -0.647079459336872], [-0.306532761160754, 0.447358396149272, 0.840183391724637], [-0.724757323227605, -0.07942789118479 , 0.68441071918103 ], [ 0.565739665665465, -0.793473207481602, 0.224363320758064], [-0.353683942716768, -0.927410345459133, -0.121728056748207], [ 0.911779475442355, 0.212215774518691, -0.35160013254761 ], [-0.314947409845297, 0.860960962081638, 0.399442550065963], [ 0.954713204461325, 0.245284517535246, 0.16839893908422 ], [ 0.588051066433063, -0.727808519824825, -0.352832398933736], [ 0.316736133051606, -0.748368949236216, 0.582771085280144], [ 0.722955428210676, 0.537784801392322, 0.433731433276559], [ 0.019099566709024, -0.340088465110929, 0.940199469501031], [-0.608331448615299, 0.404846294042449, -0.682665603956803], [ 0.077761252733988, -0.823296080208836, -0.562260395089328], [ 0.982030085383455, -0.18047311605592 , -0.055193892622567], [ 0.203731650875146, -0.26548316160085 , -0.942343942166609], [ 0.88185417506372 , 0.461268419580393, 0.097799074742515], [-0.082358777248902, 0.533753109180482, 0.841620252994327], [-0.727088618637696, 0.651694509913794, -0.21595463967173 ]])

if calculate_design_score(spherical_design_221236, 2, 21, 236) < 1e-8:
  print("Correct numerical construction of a (2, 21, 236) design")


spherical_design_221238 = np.array([[ 0.81249573268336 , 0.560353609717294, -0.16079339678024 ], [-0.16482544319716 , 0.004672562942575, -0.986311685234646], [ 0.407813979848653, -0.230908392225371, -0.883385007932496], [ 0.251979645618198, 0.658903379403351, -0.708768364703851], [-0.540930226009474, -0.029906952454385, -0.840535582104785], [ 0.489609885575068, 0.722779419818319, -0.487721303855249], [-0.601961598182293, 0.120784339553169, 0.789337302825936], [-0.354693310611025, 0.929761502306162, 0.098672206001391], [-0.402502873632046, -0.884941445707631, 0.234243621870123], [-0.007000469841647, -0.576591344499447, -0.817002701874551], [ 0.236360767466825, 0.923781536256665, 0.301266096455891], [-0.363705494200998, -0.802997678743734, -0.472136676631017], [ 0.918085965538279, -0.394861812788798, 0.034674323104995], [-0.98125576901743 , 0.139079002652145, 0.13339470301064 ], [-0.233373209856578, -0.259491685766903, 0.937123796485336], [-0.688694720243075, -0.725048174475002, -0.002174166456299], [ 0.103457420374242, 0.989475672095301, 0.101165490662872], [-0.223928808286279, 0.917615889217207, 0.328385396562638], [-0.698925931009656, 0.266420354802502, 0.663718869333391], [-0.616039425784603, 0.184109879171849, -0.765897498540309], [ 0.786181805750765, -0.616437086791029, 0.043857568731704], [ 0.426163560872964, 0.226160032262367, 0.87592023563288 ], [ 0.525528612907421, 0.649185474094298, 0.549888986287745], [ 0.601964557972525, -0.739361361974431, 0.301634625605643], [-0.798326230918766, 0.054136010401517, 0.599787063385701], [-0.077358934784 , -0.38641315287582 , -0.919075878528891], [-0.959820197165142, -0.201992522606446, -0.194792735811559], [ 0.310591110866121, -0.770831317481508, -0.556194428092076], [ 0.975414780544893, -0.201646162053529, 0.088909117775629], [ 0.439760089811694, -0.890644305108066, 0.115601839031135], [ 0.774070384915531, 0.172368022570884, -0.6091833090226 ], [-0.709708437504222, 0.683213841856458, 0.171851040238499], [-0.331175868944102, 0.549571003941606, 0.767003425973923], [ 0.890428750164741, 0.168420240527381, -0.422813509080252], [ 0.069770835985886, -0.749928176360281, 0.657829583362419], [-0.887034506201946, 0.136073400272175, 0.441195891351492], [-0.634003862514774, 0.474076238804203, 0.610975304016134], [-0.653370681729851, 0.580558026576795, -0.485859166871561], [-0.178257116054146, 0.249105958884995, 0.951929945859487], [-0.699153040518442, -0.072284631342812, -0.711308623598401], [ 0.584133014875016, -0.397713748835174, 0.707539677276474], [-0.288819239299618, -0.700564518245209, -0.65252800919673 ], [-0.185889575571547, 0.588699807751665, -0.786687741131755], [ 0.008812681913485, 0.17520224383518 , -0.984493022013163], [-0.429722369590709, -0.500118047176519, -0.751811561471152], [ 0.19309176561142 , -0.859351979093988, 0.473529034041537], [-0.285520027934038, -0.956043315955482, 0.066779425464686], [-0.161111686762673, 0.694660499370741, 0.701063345927084], [ 0.809800045864949, -0.359332309016314, -0.463793248564628], [ 0.348387624425149, -0.048447428093938, 0.936097703158428], [ 0.422360145400795, -0.833674602612237, -0.355806920304184], [-0.334127512635018, 0.458957502821912, -0.823235577404069], [ 0.038112818140511, -0.968074691452344, -0.247747461869616], [-0.516635449608924, -0.800624740980035, -0.303459777133704], [ 0.768280486127189, -0.21726537190342 , 0.602113654394117], [ 0.059839559517732, 0.584791223090598, -0.808973703227074], [-0.163375078555158, -0.018935560684349, 0.986382293154467], [ 0.48722351713536 , 0.375753076994594, -0.788303792632855], [-0.365446829811014, 0.022989200548375, 0.930548285280902], [ 0.705858436856415, 0.636647291457619, -0.310554493444058], [ 0.774514744193523, -0.577845832171455, 0.257334617315511], [-0.318871582327595, -0.814036559505626, 0.485453801892764], [ 0.859862313199189, -0.484909110225449, -0.159687060089739], [-0.084215772224927, 0.853239476267011, 0.514674751516097], [ 0.897065154280675, 0.134772796295872, 0.420844867325219], [-0.546622589142211, 0.822419234487433, 0.15757648233341 ], [ 0.381562229022277, 0.914103214933773, 0.137206333058253], [-0.880132680010993, 0.391455124076368, -0.268569081264063], [-0.486189743430545, 0.634982238124252, 0.600347474925693], [ 0.460869184641123, -0.763101875958771, 0.453072975972339], [ 0.658420448920876, 0.340919644434264, -0.671011407117381], [-0.574307663938855, -0.780624995049633, 0.246567078590908], [-0.919926849977267, 0.298375643280816, 0.254374853685782], [ 0.619638404095969, 0.563374600666058, -0.546495478017669], [-0.975368920106106, -0.18724200948354 , 0.116601456147057], [-0.111330867105209, -0.877671127348214, -0.466153226148782], [-0.055779058196455, -0.990808277490197, 0.123238199936648], [-0.898884720636225, 0.208309896944842, -0.385503885637442], [-0.190880061817832, 0.285477394798523, -0.939184464873365], [ 0.831777974356188, 0.341868272508442, 0.43734595645554 ], [ 0.185117290767502, 0.926846396140613, -0.326630290420313], [-0.154940538438227, 0.970676461122499, -0.183795096156492], [ 0.594600167396344, -0.69201579569851 , -0.409346771620343], [ 0.011770852514425, -0.919891754201297, 0.391995672914313], [-0.880265472342786, -0.360667353767362, 0.308304651485474], [ 0.256151849620821, 0.507250017105839, 0.822850928225738], [ 0.624387532558428, 0.082566687299175, -0.776738663473139], [-0.319454379997306, 0.716308874695171, -0.620363196146801], [ 0.115798422277506, 0.801695170724191, -0.586409054019079], [ 0.960021584345108, -0.048720675062933, 0.275653502449945], [ 0.771774707373872, 0.430623558362465, -0.467896518517954], [-0.487668846212993, 0.572918162091961, -0.658744165802216], [ 0.336127797008051, -0.895146524187894, 0.292798231403209], [ 0.020558622607963, -0.413162029767739, 0.910425439118803], [-0.75023664736288 , 0.544465109148985, 0.375103609517542], [-0.572546743037252, -0.334543384978351, -0.748512491949641], [ 0.958552320799364, 0.027766433825978, -0.283560352381537], [-0.998232962301715, -0.042121701237462, -0.041913187175452], [ 0.484648357155739, -0.392228241151593, -0.781839482726072], [ 0.0899672476694 , 0.73145673036921 , 0.675926731195311], [-0.452470939706731, -0.170434388446213, 0.87534117231847 ], [ 0.645554540464184, 0.520815088506475, 0.558579429329498], [ 0.771047039725103, 0.532196364691187, 0.34964766829001 ], [ 0.248250345200719, 0.43470878430466 , -0.865678946813476], [-0.429873070597755, -0.569528867969574, 0.700604033476937], [-0.829885098451968, 0.008114875740668, -0.557875319546474], [ 0.173833923589282, -0.984182808782873, -0.034146242923803], [-0.250717354359357, -0.247778688684074, -0.935813298504058], [-0.419580755258212, 0.255677317567212, -0.870965613040253], [ 0.013259211005451, 0.151256567738677, 0.988405607045725], [-0.730464222439584, -0.280637476869427, -0.622627196894012], [-0.183979473480971, 0.439486242817646, 0.87920611673924 ], [ 0.923965472613249, 0.376985619520291, -0.064572812339858], [-0.662472779065771, -0.695969805603098, -0.277048455483259], [ 0.541341797850531, 0.84072912700047 , 0.011117234952214], [-0.622157715454826, 0.77912459768128 , -0.07671139672836 ], [ 0.762466847626801, -0.411641532706522, 0.499194906645765], [-0.81278906725599 , -0.575681514791841, -0.089245311787848], [ 0.903296656160861, 0.390343228295633, 0.178009311813595], [-0.81249573268336 , -0.560353609717294, 0.16079339678024 ], [ 0.16482544319716 , -0.004672562942575, 0.986311685234646], [-0.407813979848653, 0.230908392225371, 0.883385007932496], [-0.251979645618198, -0.658903379403351, 0.708768364703851], [ 0.540930226009474, 0.029906952454385, 0.840535582104785], [-0.489609885575068, -0.722779419818319, 0.487721303855249], [ 0.601961598182293, -0.120784339553169, -0.789337302825936], [ 0.354693310611025, -0.929761502306162, -0.098672206001391], [ 0.402502873632046, 0.884941445707631, -0.234243621870123], [ 0.007000469841647, 0.576591344499447, 0.817002701874551], [-0.236360767466825, -0.923781536256665, -0.301266096455891], [ 0.363705494200998, 0.802997678743734, 0.472136676631017], [-0.918085965538279, 0.394861812788798, -0.034674323104995], [ 0.98125576901743 , -0.139079002652145, -0.13339470301064 ], [ 0.233373209856578, 0.259491685766903, -0.937123796485336], [ 0.688694720243075, 0.725048174475002, 0.002174166456299], [-0.103457420374242, -0.989475672095301, -0.101165490662872], [ 0.223928808286279, -0.917615889217207, -0.328385396562638], [ 0.698925931009656, -0.266420354802502, -0.663718869333391], [ 0.616039425784603, -0.184109879171849, 0.765897498540309], [-0.786181805750765, 0.616437086791029, -0.043857568731704], [-0.426163560872964, -0.226160032262367, -0.87592023563288 ], [-0.525528612907421, -0.649185474094298, -0.549888986287745], [-0.601964557972525, 0.739361361974431, -0.301634625605643], [ 0.798326230918766, -0.054136010401517, -0.599787063385701], [ 0.077358934784 , 0.38641315287582 , 0.919075878528891], [ 0.959820197165142, 0.201992522606446, 0.194792735811559], [-0.310591110866121, 0.770831317481508, 0.556194428092076], [-0.975414780544893, 0.201646162053529, -0.088909117775629], [-0.439760089811694, 0.890644305108066, -0.115601839031135], [-0.774070384915531, -0.172368022570884, 0.6091833090226 ], [ 0.709708437504222, -0.683213841856458, -0.171851040238499], [ 0.331175868944102, -0.549571003941606, -0.767003425973923], [-0.890428750164741, -0.168420240527381, 0.422813509080252], [-0.069770835985886, 0.749928176360281, -0.657829583362419], [ 0.887034506201946, -0.136073400272175, -0.441195891351492], [ 0.634003862514774, -0.474076238804203, -0.610975304016134], [ 0.653370681729851, -0.580558026576795, 0.485859166871561], [ 0.178257116054146, -0.249105958884995, -0.951929945859487], [ 0.699153040518442, 0.072284631342812, 0.711308623598401], [-0.584133014875016, 0.397713748835174, -0.707539677276474], [ 0.288819239299618, 0.700564518245209, 0.65252800919673 ], [ 0.185889575571547, -0.588699807751665, 0.786687741131755], [-0.008812681913485, -0.17520224383518 , 0.984493022013163], [ 0.429722369590709, 0.500118047176519, 0.751811561471152], [-0.19309176561142 , 0.859351979093988, -0.473529034041537], [ 0.285520027934038, 0.956043315955482, -0.066779425464686], [ 0.161111686762673, -0.694660499370741, -0.701063345927084], [-0.809800045864949, 0.359332309016314, 0.463793248564628], [-0.348387624425149, 0.048447428093938, -0.936097703158428], [-0.422360145400795, 0.833674602612237, 0.355806920304184], [ 0.334127512635018, -0.458957502821912, 0.823235577404069], [-0.038112818140511, 0.968074691452344, 0.247747461869616], [ 0.516635449608924, 0.800624740980035, 0.303459777133704], [-0.768280486127189, 0.21726537190342 , -0.602113654394117], [-0.059839559517732, -0.584791223090598, 0.808973703227074], [ 0.163375078555158, 0.018935560684349, -0.986382293154467], [-0.48722351713536 , -0.375753076994594, 0.788303792632855], [ 0.365446829811014, -0.022989200548375, -0.930548285280902], [-0.705858436856415, -0.636647291457619, 0.310554493444058], [-0.774514744193523, 0.577845832171455, -0.257334617315511], [ 0.318871582327595, 0.814036559505626, -0.485453801892764], [-0.859862313199189, 0.484909110225449, 0.159687060089739], [ 0.084215772224927, -0.853239476267011, -0.514674751516097], [-0.897065154280675, -0.134772796295872, -0.420844867325219], [ 0.546622589142211, -0.822419234487433, -0.15757648233341 ], [-0.381562229022277, -0.914103214933773, -0.137206333058253], [ 0.880132680010993, -0.391455124076368, 0.268569081264063], [ 0.486189743430545, -0.634982238124252, -0.600347474925693], [-0.460869184641123, 0.763101875958771, -0.453072975972339], [-0.658420448920876, -0.340919644434264, 0.671011407117381], [ 0.574307663938855, 0.780624995049633, -0.246567078590908], [ 0.919926849977267, -0.298375643280816, -0.254374853685782], [-0.619638404095969, -0.563374600666058, 0.546495478017669], [ 0.975368920106106, 0.18724200948354 , -0.116601456147057], [ 0.111330867105209, 0.877671127348214, 0.466153226148782], [ 0.055779058196455, 0.990808277490197, -0.123238199936648], [ 0.898884720636225, -0.208309896944842, 0.385503885637442], [ 0.190880061817832, -0.285477394798523, 0.939184464873365], [-0.831777974356188, -0.341868272508442, -0.43734595645554 ], [-0.185117290767502, -0.926846396140613, 0.326630290420313], [ 0.154940538438227, -0.970676461122499, 0.183795096156492], [-0.594600167396344, 0.69201579569851 , 0.409346771620343], [-0.011770852514425, 0.919891754201297, -0.391995672914313], [ 0.880265472342786, 0.360667353767362, -0.308304651485474], [-0.256151849620821, -0.507250017105839, -0.822850928225738], [-0.624387532558428, -0.082566687299175, 0.776738663473139], [ 0.319454379997306, -0.716308874695171, 0.620363196146801], [-0.115798422277506, -0.801695170724191, 0.586409054019079], [-0.960021584345108, 0.048720675062933, -0.275653502449945], [-0.771774707373872, -0.430623558362465, 0.467896518517954], [ 0.487668846212993, -0.572918162091961, 0.658744165802216], [-0.336127797008051, 0.895146524187894, -0.292798231403209], [-0.020558622607963, 0.413162029767739, -0.910425439118803], [ 0.75023664736288 , -0.544465109148985, -0.375103609517542], [ 0.572546743037252, 0.334543384978351, 0.748512491949641], [-0.958552320799364, -0.027766433825978, 0.283560352381537], [ 0.998232962301715, 0.042121701237462, 0.041913187175452], [-0.484648357155739, 0.392228241151593, 0.781839482726072], [-0.0899672476694 , -0.73145673036921 , -0.675926731195311], [ 0.452470939706731, 0.170434388446213, -0.87534117231847 ], [-0.645554540464184, -0.520815088506475, -0.558579429329498], [-0.771047039725103, -0.532196364691187, -0.34964766829001 ], [-0.248250345200719, -0.43470878430466 , 0.865678946813476], [ 0.429873070597755, 0.569528867969574, -0.700604033476937], [ 0.829885098451968, -0.008114875740668, 0.557875319546474], [-0.173833923589282, 0.984182808782873, 0.034146242923803], [ 0.250717354359357, 0.247778688684074, 0.935813298504058], [ 0.419580755258212, -0.255677317567212, 0.870965613040253], [-0.013259211005451, -0.151256567738677, -0.988405607045725], [ 0.730464222439584, 0.280637476869427, 0.622627196894012], [ 0.183979473480971, -0.439486242817646, -0.87920611673924 ], [-0.923965472613249, -0.376985619520291, 0.064572812339858], [ 0.662472779065771, 0.695969805603098, 0.277048455483259], [-0.541341797850531, -0.84072912700047 , -0.011117234952214], [ 0.622157715454826, -0.77912459768128 , 0.07671139672836 ], [-0.762466847626801, 0.411641532706522, -0.499194906645765], [ 0.81278906725599 , 0.575681514791841, 0.089245311787848], [-0.903296656160861, -0.390343228295633, -0.178009311813595]])

if calculate_design_score(spherical_design_221238, 2, 21, 238) < 1e-8:
  print("Correct numerical construction of a (2, 21, 238) design")


**Prompt used**

The problem of finding spherical t-designs

Act as an expert in optimization and computational geometry. Your goal is to solve the following problem: for a given dimension `d`, a degree `t`, and a number of points `N`, find a configuration of `N` points on the unit sphere $S^d \subset R^(d+1)$ that best approximates a "spherical t-design".

A spherical t-design is a set of points whose average value for any polynomial of degree up to `t` matches the true average over the whole sphere. We will measure the quality of your proposed set of points using a "design error" function. Your goal is to find a set of points that minimizes this error, which corresponds to maximizing the score (`score = -error`). A perfect design has an error of 0, and thus a score of 0.

Your task is to produce a search function that finds the best configuration of `N` points. The points should be represented as a list or NumPy array of `N` vectors, where each vector has `d+1` coordinates. Your solution will be evaluated by the following scoring function, which you have access to but do not need to implement:

```
def calculate_design_score(points: np.ndarray, d: int, t: int, N: int) -> float:
    """Calculates the score for a potential spherical design."""
    # --- Safety checks ---
    if not isinstance(points, np.ndarray):
        points = np.array(points, dtype=np.float64)
    
    if points.ndim != 2 or points.shape != (N, d + 1):
        return -1_000_000.0
    
    if np.any(np.isnan(points)) or np.any(np.isinf(points)):
        return -1_000_000.0

    # --- Normalization ---
    norms = np.linalg.norm(points, axis=1, keepdims=True)
    if np.any(norms < 1e-9):
        return -1_000_000.0
    points_normalized = points / norms

    # --- Score Calculation ---
    dot_products = np.clip(points_normalized @ points_normalized.T, -1.0, 1.0)
    
    total_error = 0.0
    lam = (d - 1) / 2.0

    for k in range(1, t + 1):
        dim_k = get_dim_k(k, d)
        if abs(dim_k) < 1e-9:
            continue
            
        gegen_poly = get_gegenbauer_poly(k, d)
        
        denom = binom(k + d - 2, k)
        if abs(denom) < 1e-9:
            continue
            
        poly_vals = gegen_poly(dot_products)
        error_k = np.sum(poly_vals)
        total_error += dim_k * error_k / denom
        
    return -total_error
```

You can code up any search method you wish. You are allowed to call `calculate_design_score()` as many times as you want. Your objective is to make the score as high (close to zero) as possible.

Your search function will have 200 seconds to run. After this time, it must return the best configuration of points it has found. If it fails to return within the time limit, it will receive a score of negative infinity.

You can access the best construction we have found so far through the spherical_design_xyz variable, where x should be replaced by d, y should be replaced by t, and z should be replaced by N. An example of how to load this is provided in the code.


In [ ]:
#@title An example of a final program generated by AlphaEvolve

import itertools
import logging
import time
from scipy import integrate
import numpy as np
from scipy import optimize
import warnings
import random
import re
from typing import Any, Callable, Mapping, List, Tuple
import scipy.linalg as la
import collections
import copy
import math
import numba
from scipy.special import gegenbauer, binom
import scipy.stats # Added for random rotations

njit = numba.njit

# Global counter for evaluations
global_eval_count = 0
# Global variable for gradient annealing (tapering)
global_current_t_for_gradient_focus = -1 # Sentinel value. Will be set by search function.


def _calculate_pair_error_contribution_for_k(points_normalized: np.ndarray, d: int, k_val: int) -> np.ndarray:
    """
    Calculates the Gegenbauer polynomial values for each pair of points for a specific k.
    Returns an N x N matrix where element (i,j) is G_k(<y_i,y_j>).
    """
    N = points_normalized.shape[0]
    lam = (d - 1) / 2.0

    # G_0(x) = 1. G_k is defined for k >= 0.
    # The error objective typically only uses k >= 1.
    if k_val < 0: # Should not happen, but for safety
        return np.zeros((N,N))
    elif k_val == 0:
        return np.ones((N,N))

    # C_k^l(x)
    poly_func = gegenbauer(k_val, lam)
    dot_products = np.clip(points_normalized @ points_normalized.T, -1.0, 1.0)

    return poly_func(dot_products)


def _objective_and_grad_single_point(
    flat_point_to_optimize: np.ndarray,
    d: int, t: int, # 't' is the global design degree, which is what we always optimize for in SSPR
    other_points_normalized: np.ndarray
) -> Tuple[float, np.ndarray]:
    """
    Calculates the contribution to the design error and its gradient for a single point,
    assuming all other points are fixed. This is for local optimization of one point.

    The objective for optimizing y_m given y_j (for j != m) is to minimize:
    E_m(y_m) = sum_{k=1 to t} C_k * 2 * sum_{j != m} P_k(y_m . y_j)
    """
    # Do NOT increment global_eval_count here. This is an internal helper for a sub-problem.

    point_to_optimize = flat_point_to_optimize.reshape(1, d + 1)
    norm_p = np.linalg.norm(point_to_optimize)

    if norm_p < 1e-9:
        grad = np.zeros_like(point_to_optimize)
        return 1e18, grad.flatten()

    point_normalized = point_to_optimize / norm_p

    # Calculate dot products between the single point and all other points
    # Shape (1, N-1)
    dot_products_p_others = np.clip(point_normalized @ other_points_normalized.T, -1.0, 1.0)

    current_point_error_contrib = 0.0
    lam = (d - 1) / 2.0
    grad_p = np.zeros_like(point_normalized) # Gradient w.r.t. the single point

    # Use the global gradient focus t
    t_for_grad_focus = t # Default to full t for single point optimization as well
    # This check ensures that if the search function doesn't set global_current_t_for_gradient_focus,
    # or sets it to the sentinel, it defaults to full 't'.
    # This also allows the search function to control the gradient focus for sequential refinement.
    if global_current_t_for_gradient_focus != -1:
        t_for_grad_focus = global_current_t_for_gradient_focus


    for k in range(1, t + 1): # Use 't' directly here for error calculation
        dim_k = get_dim_k(k, d) # Use the global get_dim_k
        if abs(dim_k) < 1e-9:
            continue
        denom = binom(k + d - 2, k)
        if abs(denom) < 1e-9:
            continue

        constant_k = dim_k / denom

        # We need to compute 2 * sum_{j!=m} P_k(y_m . y_j)
        poly_vals_p_others = gegenbauer(k, lam)(dot_products_p_others)
        current_point_error_contrib += constant_k * 2 * np.sum(poly_vals_p_others)

        # Only include this k's gradient contribution if k is within the current focus
        if k <= t_for_grad_focus:
            # Derivative calculation for this specific objective
            if d > 1: # lam > 0
                poly_deriv_vals_p_others = (d - 1) * gegenbauer(k - 1, lam + 1)(dot_products_p_others)
            elif d == 1 and k >= 1: # lam = 0
                poly_deriv_vals_p_others = k * gegenbauer(k - 1, 1)(dot_products_p_others)
            else: # k=0 or d=1 and k=0
                poly_deriv_vals_p_others = np.zeros_like(dot_products_p_others)

            # Gradient part from this point to others: sum_j P_k'(<y_p,y_j>) y_j
            # This is (1, N-1) @ (N-1, d+1) -> (1, d+1)
            grad_contrib_p = poly_deriv_vals_p_others @ other_points_normalized
            grad_p += constant_k * 2 * grad_contrib_p

    # Project ambient gradient onto tangent space and scale for unnormalized coordinates
    tangent_grad_p = grad_p - np.sum(
        grad_p * point_normalized, axis=1, keepdims=True
    ) * point_normalized
    final_grad_p = tangent_grad_p / norm_p

    if np.any(np.isnan(final_grad_p)) or np.any(np.isinf(final_grad_p)):
        return 1e18, np.zeros_like(flat_point_to_optimize)

    return current_point_error_contrib, final_grad_p.flatten()


def _refine_points_sequentially(
    points: np.ndarray, d: int, t: int, N: int, num_passes: int, maxiter_per_point: int,
    indices_to_refine: np.ndarray = None
) -> np.ndarray:
    """
    Refines the point configuration by optimizing each point individually
    while keeping all other points fixed. Can operate on a subset of points.
    """
    current_points = points.copy()

    # Store normalized points for efficiency
    norms = np.linalg.norm(current_points, axis=1, keepdims=True)
    points_normalized = current_points / np.where(norms < 1e-9, 1.0, norms)

    # Initial error evaluation using the global 't'
    current_best_error_stage, _ = _objective_and_grad(points_normalized.flatten(), d, t, N)


    for pass_idx in range(num_passes):
        # Randomize the order in which points are processed for better convergence
        if indices_to_refine is None:
            point_indices = np.random.permutation(N)
        else:
            # Shuffle the provided subset of indices
            point_indices = np.random.permutation(indices_to_refine)

        for i in point_indices:
            # Create a view of other points
            other_points_normalized = np.delete(points_normalized, i, axis=0)

            # Optimize point i
            opt_result = optimize.minimize(
                _objective_and_grad_single_point,
                points_normalized[i].flatten(), # Start with current normalized point
                args=(d, t, other_points_normalized), # Pass global 't'
                method='L-BFGS-B',
                jac=True,
                options={
                    'maxiter': maxiter_per_point,
                    'ftol': 1e-12, # Use stricter tolerances for local refinement
                    'gtol': 1e-10,
                    'disp': False
                },
            )

            optimized_point = opt_result.x.reshape(1, d + 1)
            # Re-normalize the optimized single point
            norm_opt_p = np.linalg.norm(optimized_point)
            new_normalized_point = optimized_point / np.where(norm_opt_p < 1e-9, 1.0, norm_opt_p)

            # Update the current set of points for the next iteration in this pass
            points_normalized[i] = new_normalized_point

        # After a full pass, re-evaluate the total error for the current stage
        new_error_stage, _ = _objective_and_grad(points_normalized.flatten(), d, t, N)
        if new_error_stage < current_best_error_stage:
            # Update best points and error if this pass resulted in improvement for this stage
            current_best_error_stage = new_error_stage
            current_points = points_normalized.copy() # Store the result of this pass
        else:
            # If a pass didn't improve for the current stage, further passes might be less effective.
            # Continue for robustness but log.
            pass

    return current_points # Return the best points found during sequential refinement

def get_dim_k(k: int, d: int) -> float:
  """Computes the dimension of the space of spherical harmonics of degree k on S^d."""
  d_ambient = d + 1
  # Using formula for dimension of harmonic polynomials in R^n of degree k
  # where n = d+1 is the ambient dimension. dim = C(k+n-1,n-1)-C(k+n-3,n-1)
  # scipy.special.binom(N, K) returns 0 if K < 0 or K > N.
  # This property handles k=0 and k=1 cases for term2 correctly.
  term1 = binom(k + d_ambient - 1, d_ambient - 1)
  term2 = binom(k + d_ambient - 3, d_ambient - 1)
  # For k=0, term1=1, term2=0. Result 1.
  # For k=1, term1=d+1, term2=0. Result d+1.
  return float(term1 - term2)


def _objective_and_grad(
    flat_points: np.ndarray, d: int, t: int, N: int
) -> Tuple[float, np.ndarray]:
    """Calculates design error and its gradient for scipy.optimize.minimize."""
    global global_eval_count
    global_eval_count += 1

    points = flat_points.reshape(N, d + 1)
    norms = np.linalg.norm(points, axis=1, keepdims=True)

    if np.any(norms < 1e-9):
        grad = np.zeros_like(points)
        return 1e18, grad.flatten()

    points_normalized = points / norms
    dot_products = np.clip(points_normalized @ points_normalized.T, -1.0, 1.0)

    total_error = 0.0
    lam = (d - 1) / 2.0
    grad_y = np.zeros_like(points_normalized)

    # Gradient annealing: only include contributions for k <= current_t_for_gradient_focus
    # if global_current_t_for_gradient_focus is set to a specific value.
    # Otherwise, use the full 't'.
    t_for_grad_focus = t
    if global_current_t_for_gradient_focus != -1:
        t_for_grad_focus = global_current_t_for_gradient_focus

    for k in range(1, t + 1): # Full 't' for error calculation
        dim_k = get_dim_k(k, d) # Use the global get_dim_k
        if abs(dim_k) < 1e-9:
            continue
        denom = binom(k + d - 2, k)
        if abs(denom) < 1e-9:
            continue

        constant_k = dim_k / denom
        poly_vals = gegenbauer(k, lam)(dot_products)
        total_error += constant_k * np.sum(poly_vals)

        # Only include this k's gradient contribution if k is within the current focus
        if k <= t_for_grad_focus:
            if d > 1: # lam > 0, use derivative identity C_k^l'(x) = 2l C_{k-1}^{l+1}(x)
                poly_deriv_vals = (d - 1) * gegenbauer(k - 1, lam + 1)(dot_products)
            else: # d=1 (circle S^1), lam = 0.
                  # C_k^0(x) = T_k(x), (T_k(x))' = k U_{k-1}(x) = k C_{k-1}^1(x).
                if k > 0: # For k=0, derivative is 0.
                    poly_deriv_vals = k * gegenbauer(k-1, 1.0)(dot_products) # U_{k-1} = C_{k-1}^1
                else: # k=0
                    poly_deriv_vals = np.zeros_like(dot_products)

            # The gradient d/dy_m sum_{i,j} G_k(<y_i,y_j>) = 2 * sum_j G_k'(<y_m,y_j>) y_j
            # This is 2 * (poly_deriv_vals @ points_normalized)_m
            grad_y += constant_k * 2 * (poly_deriv_vals @ points_normalized)

    # Project ambient gradient onto tangent space and scale for unnormalized coordinates
    tangent_grad = grad_y - np.sum(
        grad_y * points_normalized, axis=1, keepdims=True
    ) * points_normalized
    final_grad = tangent_grad / norms

    if np.any(np.isnan(final_grad)) or np.any(np.isinf(final_grad)):
        return 1e18, np.zeros_like(flat_points)

    return total_error, final_grad.flatten()

def _objective_and_grad_antipodal(
    flat_points_half: np.ndarray, d: int, t: int, N_total: int
) -> Tuple[float, np.ndarray]:
    """
    Calculates design error and its gradient for antipodal configurations.
    N_total is the total number of points in the design (must be even).
    flat_points_half represents N_total/2 points.
    """
    global global_eval_count
    global_eval_count += 1

    N_half = N_total // 2
    points_half = flat_points_half.reshape(N_half, d + 1)
    norms_half = np.linalg.norm(points_half, axis=1, keepdims=True)

    if np.any(norms_half < 1e-9):
        grad_half = np.zeros_like(points_half)
        return 1e18, grad_half.flatten()

    points_half_normalized = points_half / norms_half

    # Full dot products not needed for error calculation if done efficiently
    # For gradient, we work with points_half_normalized (Y in notation)
    dot_products_half = np.clip(points_half_normalized @ points_half_normalized.T, -1.0, 1.0)

    total_error = 0.0
    lam = (d - 1) / 2.0
    grad_y_half = np.zeros_like(points_half_normalized)

    # Gradient annealing for antipodal:
    t_for_grad_focus = t
    if global_current_t_for_gradient_focus != -1:
        t_for_grad_focus = global_current_t_for_gradient_focus

    for k in range(1, t + 1):
        if k % 2 != 0:  # Odd k terms are zero for antipodal designs
            continue

        dim_k = get_dim_k(k, d) # Use the global get_dim_k
        if abs(dim_k) < 1e-9: # Added safety check for dim_k
            continue
        denom = binom(k + d - 2, k)
        if abs(denom) < 1e-9: # Added safety check for denom
            continue

        constant_k = dim_k / denom

        # Error for even k: 4 * sum_{y_i, y_j in Y} G_k(<y_i, y_j>)
        poly_vals_half = gegenbauer(k, lam)(dot_products_half)
        total_error += constant_k * 4 * np.sum(poly_vals_half)

        # Only include this k's gradient contribution if k is within the current focus
        if k <= t_for_grad_focus:
            # Derivative of Gegenbauer polynomial C_k^l(x) is 2l C_{k-1}^{l+1}(x) for l > 0.
            # For l=0 (d=1), C_k^0(x) = T_k(x), and T_k'(x) = k U_{k-1}(x) = k C_{k-1}^1(x).
            if d > 1: # lam > 0
                poly_deriv_vals_half = (d - 1) * gegenbauer(k - 1, lam + 1)(dot_products_half)
            elif d == 1 and k >= 1: # lam = 0
                poly_deriv_vals_half = k * gegenbauer(k - 1, 1)(dot_products_half)
            else: # d=0 or k=0, or other unexpected cases where derivative is zero or undefined
                poly_deriv_vals_half = np.zeros_like(dot_products_half)

            # Applying the same gradient logic as in the non-antipodal case:
            # The sum term G_k'(<y_m,y_j>) y_j becomes (poly_deriv_vals_half @ points_half_normalized)_m
            # Factor 4 from error term, factor 2 from derivative of sum structure
            grad_y_half += constant_k * 4 * 2 * (poly_deriv_vals_half @ points_half_normalized)

    # Project ambient gradient onto tangent space and scale for unnormalized coordinates
    tangent_grad_half = grad_y_half - np.sum(
        grad_y_half * points_half_normalized, axis=1, keepdims=True
    ) * points_half_normalized
    final_grad_half = tangent_grad_half / norms_half

    if np.any(np.isnan(final_grad_half)) or np.any(np.isinf(final_grad_half)):
        return 1e18, np.zeros_like(flat_points_half)

    return total_error, final_grad_half.flatten()

# --- Recursive Zonal Construction (RZC) for Initialization ---
def _rzc_construction(d_target: int, N_target: int, t_design_degree: int) -> np.ndarray:
    """
    Constructs N_target points on S^d_target using a recursive zonal approach.
    t_design_degree is used to guide the number of latitudes.
    """

    memo = {} # Memoization for (d, N) subproblems to handle shared sub-structures if they arise

    def rzc_recursive_helper(d_curr: int, N_curr: int, recursion_depth: int) -> np.ndarray:
        if N_curr == 0:
            return np.empty((0, d_curr + 1))

        state_key = (d_curr, N_curr)
        if state_key in memo:
            # If a solution for this (d,N) is memoized, we need to ensure its points are
            # rotated differently if used multiple times at the same level of recursion,
            # or ensure the number of points matches.
            # For simplicity, let's assume memoized results are used as is.
            # A more robust memoization would store canonical orientations and apply new rotations.
            # For now, let's skip memoization for simplicity of implementation within time constraints.
            # It adds significant complexity for ensuring varied orientations.
            pass


        # Base case d_curr = 1 (S^1 in R^2, i.e., a circle)
        if d_curr == 1:
            pts = np.zeros((N_curr, 2))
            if N_curr > 0:
                for j in range(N_curr):
                    angle = 2 * np.pi * j / N_curr
                    pts[j, 0] = np.cos(angle)
                    pts[j, 1] = np.sin(angle)
            return pts

        # Base case d_curr = 0 (S^0 in R^1, i.e., {-1, 1})
        if d_curr == 0:
            pts = np.zeros((N_curr, 1))
            if N_curr > 0:
                if N_curr == 1:
                    pts[0, 0] = 1.0
                else: # N_curr >= 2
                    for j in range(N_curr):
                        pts[j, 0] = 1.0 if (j % 2 == 0) else -1.0
            return pts

        # Recursive step for d_curr >= 2 (for S^2 and higher)
        all_pts_list = []
        N_remaining_for_layers = N_curr

        # Add poles (North and South)
        # These are (d_curr+1)-dimensional vectors
        # Pole N: (0, ..., 0, 1)
        # Pole S: (0, ..., 0, -1)
        # The "height" coordinate is the last one (index d_curr)
        use_poles = True # Default to using poles
        if d_curr == 0: use_poles = False # No poles for S^0 itself.

        if use_poles:
            if N_remaining_for_layers >= 1:
                poleN = np.zeros(d_curr + 1)
                poleN[d_curr] = 1.0
                all_pts_list.append(poleN)
                N_remaining_for_layers -= 1
            if N_remaining_for_layers >= 1:
                poleS = np.zeros(d_curr + 1)
                poleS[d_curr] = -1.0
                all_pts_list.append(poleS)
                N_remaining_for_layers -= 1

        if N_remaining_for_layers <= 0:
            return np.array(all_pts_list) if all_pts_list else np.empty((0, d_curr + 1))

        # Determine number of latitudes (rings of points, excluding poles)
        # Heuristic: Link to t_design_degree for relevance to design strength.
        # Adjusted to be less aggressive for high t or low d_curr.
        num_latitudes = max(1, int(round(t_design_degree / (d_curr +1.0) )))
        # Fallback if t_design_degree is small, ensure some latitudes based on N.
        if num_latitudes < 2 and N_remaining_for_layers > 5 : #and d_curr >1:
             num_latitudes = max(num_latitudes, int(round(N_remaining_for_layers**(1.0/d_curr) / 2.0 )))
        num_latitudes = max(1, num_latitudes)


        # Heights zk for these latitudes (interior to (-1,1))
        # Use roots of Chebyshev polynomials (scaled) for z-coords of layers for good spacing.
        # z_j = cos( (2j+1)pi / (2*num_latitudes) ) for j = 0..num_latitudes-1. These are symmetric.
        # Or, simpler: equally spaced angles phi_j = pi * (idx+1)/(num_latitudes+1), z_j = cos(phi_j)
        lat_heights_z = [np.cos(np.pi * (j + 1) / (num_latitudes + 1)) for j in range(num_latitudes)]

        # Distribute N_remaining_for_layers among these latitudes
        # Ensure robust distribution even if N_remaining_for_layers < num_latitudes
        points_per_latitude = [0] * num_latitudes
        if num_latitudes > 0:
            base_alloc = N_remaining_for_layers // num_latitudes
            extra_alloc = N_remaining_for_layers % num_latitudes
            for j in range(num_latitudes):
                points_per_latitude[j] = base_alloc + (1 if j < extra_alloc else 0)

        current_sum_Mk = sum(points_per_latitude)
        if current_sum_Mk != N_remaining_for_layers:
            # This indicates a logic error in distribution, fallback or correction needed.
            # For now, this should be correct by construction.
            pass


        for k_lat_idx in range(num_latitudes):
            zk = lat_heights_z[k_lat_idx]
            Mk = points_per_latitude[k_lat_idx]

            if Mk == 0:
                continue

            rk = np.sqrt(max(0.0, 1.0 - zk**2)) # max(0,..) for robustness

            if rk < 1e-9: # Latitude is effectively a pole (should not happen with chosen z_k)
                # If it does, place Mk points at this pole.
                pole_pt_on_lat = np.zeros(d_curr + 1)
                pole_pt_on_lat[d_curr] = np.sign(zk) if zk != 0 else 1.0
                for _ in range(Mk):
                    all_pts_list.append(pole_pt_on_lat.copy())
                continue

            # Get Mk points on S^(d_curr-1)
            # These points are (d_curr)-dimensional vectors
            points_on_Sd_minus_1 = rzc_recursive_helper(d_curr - 1, Mk, recursion_depth + 1)

            if points_on_Sd_minus_1.shape[0] != Mk and Mk > 0 :
                # Sub-problem failed to return the expected number of points.
                # Fallback: Generate Mk random points on the unit sphere for d_curr - 1.
                print(f"RZC: Fallback to random for d={d_curr-1}, N={Mk} (expected {Mk}, got {points_on_Sd_minus_1.shape[0]})")
                temp_random_points = np.random.randn(Mk, d_curr)
                norms_temp = np.linalg.norm(temp_random_points, axis=1, keepdims=True)
                points_on_Sd_minus_1 = temp_random_points / np.where(norms_temp < 1e-9, 1.0, norms_temp)
                # Ensure the number of points is exactly Mk, if possible (can be 0 if Mk=0)
                if points_on_Sd_minus_1.shape[0] != Mk: # If random generation failed or Mk was very small
                    if Mk > 0:
                        # As a last resort, add identity vector or zeros if N becomes 0.
                        points_on_Sd_minus_1 = np.eye(Mk, d_curr) # Creates Mk identity-like vectors
                        # Pad with zeros if d_curr > Mk
                        if d_curr > Mk:
                            points_on_Sd_minus_1 = np.hstack((points_on_Sd_minus_1, np.zeros((Mk, d_curr - Mk))))
                        # Truncate if d_curr < Mk
                        elif d_curr < Mk:
                            points_on_Sd_minus_1 = points_on_Sd_minus_1[:, :d_curr]
                        # Normalize these canonical points
                        norms_id = np.linalg.norm(points_on_Sd_minus_1, axis=1, keepdims=True)
                        points_on_Sd_minus_1 = points_on_Sd_minus_1 / np.where(norms_id < 1e-9, 1.0, norms_id)
                    else:
                        points_on_Sd_minus_1 = np.empty((0, d_curr)) # If Mk was 0, return empty.


            # Apply random rotation to points_on_Sd_minus_1 if its dimension > 0 (i.e., d_curr-1 >= 1)
            # Rotation is in R^(d_curr) for the (d_curr)-dimensional child points.
            if d_curr - 1 >= 1 and points_on_Sd_minus_1.shape[0] > 0:
                # ortho_group.rvs needs dim > 0. points_on_Sd_minus_1 are (Mk, d_curr)
                if points_on_Sd_minus_1.shape[1] > 0: # Check if child vectors have dimension
                    R_matrix = scipy.stats.ortho_group.rvs(points_on_Sd_minus_1.shape[1])
                    points_on_Sd_minus_1_rotated = (R_matrix @ points_on_Sd_minus_1.T).T
                else: # Child vectors are 0-dim (should not happen if d_curr-1 >=1)
                    points_on_Sd_minus_1_rotated = points_on_Sd_minus_1
            else:
                points_on_Sd_minus_1_rotated = points_on_Sd_minus_1

            # Embed these points into S^d_curr at height zk
            for p_child_idx in range(points_on_Sd_minus_1_rotated.shape[0]):
                p_child_vector = points_on_Sd_minus_1_rotated[p_child_idx, :] # (d_curr)-dim vector

                new_pt = np.zeros(d_curr + 1) # (d_curr+1)-dim vector for S^d_curr
                new_pt[:d_curr] = rk * p_child_vector # Scale and place in first d_curr coords
                new_pt[d_curr] = zk                 # Set height coordinate (last one)
                all_pts_list.append(new_pt)

        #memo[state_key] = np.array(all_pts_list) if all_pts_list else np.empty((0,d_curr+1))
        #return memo[state_key]
        return np.array(all_pts_list) if all_pts_list else np.empty((0, d_curr + 1))

    # Initial call to the recursive helper
    final_points = rzc_recursive_helper(d_target, N_target, 0)

    # Ensure correct number of points and normalization as a final check
    if final_points.shape[0] != N_target and N_target > 0:
        # If point count mismatch, this is an issue. Could pad or truncate, or signal error.
        # For now, let this pass and rely on optimizer / subsequent checks.
        # Fallback to random if severe mismatch?
        print(f"RZC Warning: Expected {N_target} points, got {final_points.shape[0]} for d={d_target}. Might indicate issue in RZC logic.")
        # If too few points, could fill with random points.
        # If too many, could truncate.
        # This state should ideally not be reached if distribution logic is perfect.

    if final_points.shape[0] > 0:
        norms = np.linalg.norm(final_points, axis=1, keepdims=True)
        final_points = final_points / np.where(norms < 1e-9, 1.0, norms)

    return final_points
# --- End of RZC ---


def _gram_matrix_reconstruct_perturbation(
    points_opt: np.ndarray, d: int, t: int, N_opt: int,
    perturb_strength: float,
    use_antipodal: bool
) -> np.ndarray:
    """
    Perturbs a point configuration by moving in the Gram matrix space,
    projecting onto valid Gram matrices, and reconstructing points.
    This acts as a "large jump" restart strategy.
    """
    # 1. Normalize points_opt to be absolutely sure
    norms = np.linalg.norm(points_opt, axis=1, keepdims=True)
    current_points_normalized = points_opt / np.where(norms < 1e-9, 1.0, norms)

    # 2. Construct current Gram matrix (dot product matrix)
    current_A = current_points_normalized @ current_points_normalized.T

    # 3. Calculate "gradient-like" direction in A-space
    # This matrix tells us how each A_ij should change to reduce the overall error.
    lam = (d - 1) / 2.0
    A_gradient_direction = np.zeros_like(current_A)

    # Use the full 't' for the Gram matrix perturbation.
    t_for_perturbation = t

    for k in range(1, t_for_perturbation + 1):
        if use_antipodal and k % 2 != 0: # Only even k terms contribute for antipodal designs
            continue

        dim_k = get_dim_k(k, d)
        denom = binom(k + d - 2, k)
        if abs(dim_k) < 1e-9 or abs(denom) < 1e-9:
            continue
        constant_k = dim_k / denom
        if use_antipodal:
            constant_k *= 4 # From antipodal error function definition (factor 4 for sum_ij)

        # Derivative of Gegenbauer polynomial C_k^l(x) is 2l C_{k-1}^{l+1}(x) for l > 0.
        # For l=0 (d=1), C_k^0(x) = T_k(x), and T_k'(x) = k U_{k-1}(x) = k C_{k-1}^1(x).
        if d > 1: # lam > 0
            poly_deriv_vals_for_k = (d - 1) * gegenbauer(k - 1, lam + 1)(current_A)
        elif d == 1 and k >= 1: # lam = 0
            poly_deriv_vals_for_k = k * gegenbauer(k - 1, 1.0)(current_A)
        else: # k=0 (not in loop), or d=1 and k=0
            poly_deriv_vals_for_k = np.zeros_like(current_A)

        A_gradient_direction += constant_k * 2 * poly_deriv_vals_for_k

    # Apply a gradient step in the Gram matrix space
    A_perturbed = current_A - perturb_strength * A_gradient_direction

    # 4. Project A_perturbed back to the space of valid Gram matrices for d+1 dimensions
    # a. Symmetrize to handle any numerical asymmetry introduced by operations
    A_perturbed = (A_perturbed + A_perturbed.T) / 2.0

    # b. Force diagonal elements to 1.0 (A_ii = <y_i, y_i> = ||y_i||^2 = 1)
    np.fill_diagonal(A_perturbed, 1.0)

    # c. Force Positive Semi-Definite (PSD) and rank constraint (d+1)
    # Using np.linalg.eigh for symmetric matrices
    try:
        eigenvalues, eigenvectors = np.linalg.eigh(A_perturbed)
    except np.linalg.LinAlgError:
        # Fallback if eigh fails (very rare, indicates severe numerical issues)
        logging.warning("np.linalg.eigh failed, falling back to np.linalg.eig for Gram matrix reconstruction.")
        eigenvalues_complex, eigenvectors_complex = np.linalg.eig(A_perturbed)
        eigenvalues = np.real(eigenvalues_complex)
        eigenvectors = np.real(eigenvectors_complex)

    # Sort eigenvalues in descending order and corresponding eigenvectors
    sorted_indices = np.argsort(eigenvalues)[::-1]
    sorted_eigenvalues = eigenvalues[sorted_indices]
    sorted_eigenvectors = eigenvectors[:, sorted_indices]

    # Truncate to d+1 dimensions and set negative eigenvalues to zero (or a small positive epsilon)
    num_dims_to_keep = min(N_opt, d + 1)

    # Ensure eigenvalues are non-negative, and take only the top `num_dims_to_keep` for rank constraint
    truncated_eigenvalues_positive = np.maximum(0.0, sorted_eigenvalues[:num_dims_to_keep])

    # Reconstruct the projected Gram matrix
    # A_projected = V_trunc @ diag(sqrt(Lambda_trunc_positive)^2) @ V_trunc.T
    # This ensures A_projected is PSD and has rank at most num_dims_to_keep
    A_projected = sorted_eigenvectors[:, :num_dims_to_keep] @ np.diag(truncated_eigenvalues_positive) @ sorted_eigenvectors[:, :num_dims_to_keep].T

    # Due to numerical precision after reconstruction, diagonal elements might slightly deviate from 1.0.
    # Re-normalize them or force them back to 1.0 to ensure points have unit norm.
    # Force diagonal to 1.0. This can sometimes make the matrix not perfectly PSD if there were near-zero eigenvalues.
    np.fill_diagonal(A_projected, 1.0)

    # 5. Extract new points from A_projected
    # We need Y such that Y @ Y.T = A_projected.
    # SVD is robust: A_projected = U S V^T. Since A_projected is symmetric PSD, V = U.
    # So A_projected = U diag(s) U^T. Then Y = U diag(sqrt(s)).

    # We only need the first d+1 singular values/vectors for the points.
    U, s, _ = np.linalg.svd(A_projected)

    # Take the square root of singular values, clamping to avoid issues with very small negatives
    s_positive_sqrt = np.sqrt(np.maximum(0.0, s[:d+1]))

    # Construct the points from the singular value decomposition
    # new_points will be N_opt x (d+1)
    new_points = U[:, :d+1] @ np.diag(s_positive_sqrt)

    # Final normalization to ensure points are precisely on the unit sphere
    norms_final_gmr = np.linalg.norm(new_points, axis=1, keepdims=True)
    new_points_normalized = new_points / np.where(norms_final_gmr < 1e-9, 1.0, norms_final_gmr)

    return new_points_normalized

def _objective_and_grad_negative(
    flat_points: np.ndarray, d: int, t: int, N: int
) -> Tuple[float, np.ndarray]:
    """Calculates negative design error and its negative gradient for maximization."""
    error, grad = _objective_and_grad(flat_points, d, t, N)
    return -error, -grad

def _objective_and_grad_antipodal_negative(
    flat_points_half: np.ndarray, d: int, t: int, N_total: int
) -> Tuple[float, np.ndarray]:
    """Calculates negative design error and its negative gradient for antipodal maximization."""
    error, grad = _objective_and_grad_antipodal(flat_points_half, d, t, N_total)
    return -error, -grad


def _gram_matrix_reconstruct_perturbation(
    points_opt: np.ndarray, d: int, t: int, N_opt: int,
    perturb_strength: float,
    use_antipodal: bool
) -> np.ndarray:
    """
    Perturbs a point configuration by moving in the Gram matrix space,
    projecting onto valid Gram matrices, and reconstructing points.
    This acts as a "large jump" restart strategy.
    """
    # 1. Normalize points_opt to be absolutely sure
    norms = np.linalg.norm(points_opt, axis=1, keepdims=True)
    current_points_normalized = points_opt / np.where(norms < 1e-9, 1.0, norms)

    # 2. Construct current Gram matrix (dot product matrix)
    current_A = current_points_normalized @ current_points_normalized.T

    # 3. Calculate "gradient-like" direction in A-space
    # This matrix tells us how each A_ij should change to reduce the overall error.
    lam = (d - 1) / 2.0
    A_gradient_direction = np.zeros_like(current_A)

    # Use the full 't' for the Gram matrix perturbation.
    t_for_perturbation = t

    for k in range(1, t_for_perturbation + 1):
        if use_antipodal and k % 2 != 0: # Only even k terms contribute for antipodal designs
            continue

        dim_k = get_dim_k(k, d)
        denom = binom(k + d - 2, k)
        if abs(dim_k) < 1e-9 or abs(denom) < 1e-9:
            continue
        constant_k = dim_k / denom
        if use_antipodal:
             constant_k *= 4 # From antipodal error function definition (factor 4 for sum_ij)

        # Derivative of Gegenbauer polynomial C_k^lam(x)
        # Note: The derivative of Sum_{i,j} G_k(A_ij) w.r.t. A_uv is G_k'(A_uv) (plus symmetry).
        # We sum over all k.
        if d > 1: # lam > 0
            poly_deriv_vals_for_k = (d - 1) * gegenbauer(k - 1, lam + 1)(current_A)
        elif d == 1 and k >= 1: # lam = 0
            poly_deriv_vals_for_k = k * gegenbauer(k - 1, 1.0)(current_A)
        else: # k=0 (not in loop), or d=1 and k=0
            poly_deriv_vals_for_k = np.zeros_like(current_A)

        A_gradient_direction += constant_k * 2 * poly_deriv_vals_for_k

    # Apply a gradient step in the Gram matrix space
    A_perturbed = current_A - perturb_strength * A_gradient_direction

    # 4. Project A_perturbed back to the space of valid Gram matrices for d+1 dimensions
    # a. Symmetrize to handle any numerical asymmetry introduced by operations
    A_perturbed = (A_perturbed + A_perturbed.T) / 2.0

    # b. Force diagonal elements to 1.0 (A_ii = <y_i, y_i> = ||y_i||^2 = 1)
    np.fill_diagonal(A_perturbed, 1.0)

    # c. Force Positive Semi-Definite (PSD) and rank constraint (d+1)
    # Using np.linalg.eigh for symmetric matrices
    try:
        eigenvalues, eigenvectors = np.linalg.eigh(A_perturbed)
    except np.linalg.LinAlgError:
        # Fallback if eigh fails (very rare, indicates severe numerical issues)
        logging.warning("np.linalg.eigh failed, falling back to np.linalg.eig for Gram matrix reconstruction.")
        eigenvalues_complex, eigenvectors_complex = np.linalg.eig(A_perturbed)
        eigenvalues = np.real(eigenvalues_complex)
        eigenvectors = np.real(eigenvectors_complex)

    # Sort eigenvalues in descending order and corresponding eigenvectors
    sorted_indices = np.argsort(eigenvalues)[::-1]
    sorted_eigenvalues = eigenvalues[sorted_indices]
    sorted_eigenvectors = eigenvectors[:, sorted_indices]

    # Truncate to d+1 dimensions and set negative eigenvalues to zero (or a small positive epsilon)
    num_dims_to_keep = min(N_opt, d + 1)

    # Ensure eigenvalues are non-negative, and take only the top `num_dims_to_keep` for rank constraint
    truncated_eigenvalues_positive = np.maximum(0.0, sorted_eigenvalues[:num_dims_to_keep])

    # Reconstruct the projected Gram matrix
    # A_projected = V_trunc @ diag(sqrt(Lambda_trunc_positive)^2) @ V_trunc.T
    # This ensures A_projected is PSD and has rank at most num_dims_to_keep
    A_projected = sorted_eigenvectors[:, :num_dims_to_keep] @ np.diag(truncated_eigenvalues_positive) @ sorted_eigenvectors[:, :num_dims_to_keep].T

    # Due to numerical precision after reconstruction, diagonal elements might slightly deviate from 1.0.
    # Re-normalize them or force them back to 1.0 to ensure points have unit norm.
    # Force diagonal to 1.0. This can sometimes make the matrix not perfectly PSD if there were near-zero eigenvalues.
    np.fill_diagonal(A_projected, 1.0)

    # 5. Extract new points from A_projected
    # We need Y such that Y @ Y.T = A_projected.
    # SVD is robust: A_projected = U S V^T. Since A_projected is symmetric PSD, V = U.
    # So A_projected = U diag(s) U^T. Then Y = U diag(sqrt(s)).

    # We only need the first d+1 singular values/vectors for the points.
    U, s, _ = np.linalg.svd(A_projected)

    # Take the square root of singular values, clamping to avoid issues with very small negatives
    s_positive_sqrt = np.sqrt(np.maximum(0.0, s[:d+1]))

    # Construct the points from the singular value decomposition
    # new_points will be N_opt x (d+1)
    new_points = U[:, :d+1] @ np.diag(s_positive_sqrt)

    # Final normalization to ensure points are precisely on the unit sphere
    norms_final_gmr = np.linalg.norm(new_points, axis=1, keepdims=True)
    new_points_normalized = new_points / np.where(norms_final_gmr < 1e-9, 1.0, norms_final_gmr)

    return new_points_normalized


# New function for Evolutionary Jump Strategy
def _perform_evolutionary_jump(
    current_best_opt_config_points: np.ndarray,
    d: int, t: int, N_total: int, # N_total for the objective function.
    obj_grad_func: Callable, # Pass the correct objective func (antipodal/standard)
    args_for_optimizer: Tuple, # Tuple of args (d, t, N_total) for obj_grad_func
    N_opt: int, # N_opt points in the configuration being optimized (N or N/2)
    pop_size: int,
    mini_generations: int,
    mutation_strength: float,
    seq_refine_maxiter_per_point: int
) -> np.ndarray:
    """
    Performs an evolutionary jump to escape local minima.
    Generates a population of perturbed points, refines them briefly, and selects the best.
    """
    global global_current_t_for_gradient_focus # Needed to set gradient focus temporarily

    initial_t_grad_focus = global_current_t_for_gradient_focus # Store current setting

    population = []
    scores = []

    # Initialize population with perturbed copies of the current best
    for i in range(pop_size):
        if i == 0:
            candidate_points = current_best_opt_config_points.copy()
        else:
            # Apply tangential noise perturbation
            candidate_points = current_best_opt_config_points.copy()
            noise_raw = np.random.randn(N_opt, d + 1)
            candidate_points_normalized = candidate_points / np.linalg.norm(candidate_points, axis=1, keepdims=True)
            tangent_noise = noise_raw - np.sum(noise_raw * candidate_points_normalized, axis=1, keepdims=True) * candidate_points_normalized
            candidate_points += tangent_noise * mutation_strength

            # Normalize after perturbation
            norms = np.linalg.norm(candidate_points, axis=1, keepdims=True)
            candidate_points = candidate_points / np.where(norms < 1e-9, 1.0, norms)
        population.append(candidate_points)

    best_pop_score = -1e18
    best_pop_points = None

    for gen in range(mini_generations):
        # Evaluate and briefly refine each candidate
        new_population = []
        new_scores = []
        for cand_idx, cand_points in enumerate(population):
            # Temporarily set gradient focus to full 't' for refinement during evolutionary jump
            # Or perhaps to a smaller t (e.g., t/2) for quicker evaluation in jump
            # Let's keep it to full 't' for accurate evaluation of candidates against the target.
            global_current_t_for_gradient_focus = t # Optimize for full t during jump refinement

            # Sequential refinement for this candidate
            refined_cand_points = _refine_points_sequentially(
                cand_points, d, t, N_opt,
                num_passes=1, # One pass for quick local improvement
                maxiter_per_point=seq_refine_maxiter_per_point,
                indices_to_refine=np.random.permutation(N_opt) # Shuffle indices for diversity
            )

            # Evaluate score of the refined candidate using *full* t.
            # obj_grad_func requires (d, t, N_total) for its args.
            # Make sure N_total is correctly passed to the inner obj_grad_func
            global_current_t_for_gradient_focus = -1 # Always evaluate true score for selection
            current_error, _ = obj_grad_func(refined_cand_points.flatten(), *args_for_optimizer)
            current_score = -current_error

            new_population.append(refined_cand_points)
            new_scores.append(current_score)

            if current_score > best_pop_score:
                best_pop_score = current_score
                best_pop_points = refined_cand_points.copy()

        # Selection: Keep the top candidates
        sorted_indices = np.argsort(new_scores)[::-1] # Sort descending by score

        # Keep top few (e.g., 2) for next generation, or just the best.
        # Let's just create new perturbations from the current best found in the population.
        # This simplifies "crossover" and focuses on exploitation of the best path.
        if gen < mini_generations - 1:
            next_gen_population = []
            for i in range(pop_size):
                # Mutate the overall best point found so far in this jump
                mutated_points = best_pop_points.copy()
                noise_raw = np.random.randn(N_opt, d + 1)
                mutated_points_normalized = mutated_points / np.linalg.norm(mutated_points, axis=1, keepdims=True)
                tangent_noise = noise_raw - np.sum(noise_raw * mutated_points_normalized, axis=1, keepdims=True) * mutated_points_normalized
                mutated_points += tangent_noise * mutation_strength

                norms = np.linalg.norm(mutated_points, axis=1, keepdims=True)
                mutated_points = mutated_points / np.where(norms < 1e-9, 1.0, norms)
                next_gen_population.append(mutated_points)
            population = next_gen_population

    # Restore global gradient focus
    global_current_t_for_gradient_focus = initial_t_grad_focus

    if best_pop_points is None: # Fallback if for some reason no points were generated (e.g., N=0)
        return current_best_opt_config_points

    return best_pop_points


def _coulomb_objective_and_grad(
    flat_points: np.ndarray, d: int, N: int, alpha: float = 2.0
) -> Tuple[float, np.ndarray]:
    """Calculates Coulomb potential and its gradient for scipy.optimize.minimize.
    Objective: minimize sum_{i<j} 1 / ||points_i - points_j||^alpha
    """
    points = flat_points.reshape(N, d + 1)

    norms = np.linalg.norm(points, axis=1, keepdims=True)
    if np.any(norms < 1e-9):
        grad = np.zeros_like(points)
        return 1e18, grad.flatten()
    points_normalized = points / norms

    energy = 0.0
    grad_normalized = np.zeros_like(points_normalized)

    epsilon = 1e-12 # Small epsilon to prevent division by zero for very close points

    dot_products = np.clip(points_normalized @ points_normalized.T, -1.0, 1.0)
    sq_distances = 2 * (1 - dot_products)

    indices_i, indices_j = np.triu_indices(N, k=1) # Upper triangle, excluding diagonal

    clamped_dist_sq_pairs = np.maximum(sq_distances[indices_i, indices_j], epsilon)

    if alpha == 2.0: # Optimized for the common alpha=2 case (spread energy)
        inv_dist_sq_pairs = 1.0 / clamped_dist_sq_pairs
        energy = np.sum(inv_dist_sq_pairs)

        grad_factor_pairs = -2.0 * (inv_dist_sq_pairs / clamped_dist_sq_pairs)

    else: # General alpha case
        distances_pairs = np.sqrt(clamped_dist_sq_pairs)
        inv_dist_alpha_pairs = 1.0 / (distances_pairs**alpha)
        energy = np.sum(inv_dist_alpha_pairs)

        grad_factor_pairs = -alpha * inv_dist_alpha_pairs / clamped_dist_sq_pairs

    # Populate full matrix for efficient gradient calculation
    full_grad_factor = np.zeros((N, N))
    full_grad_factor[indices_i, indices_j] = grad_factor_pairs
    full_grad_factor[indices_j, indices_i] = grad_factor_pairs # Symmetric

    for i in range(N):
        grad_normalized[i] += np.sum(full_grad_factor[i, :, np.newaxis] * (points_normalized[i] - points_normalized), axis=0)

    # Project gradient onto tangent space for each point
    tangent_grad = grad_normalized - np.sum(
        grad_normalized * points_normalized, axis=1, keepdims=True
    ) * points_normalized

    # Scale for unnormalized coordinates
    final_grad = tangent_grad / norms

    if np.any(np.isnan(final_grad)) or np.any(np.isinf(final_grad)):
        return 1e18, np.zeros_like(flat_points)

    return energy, final_grad.flatten()


def _refine_points_sequentially(
    points: np.ndarray, d: int, t: int, N: int, num_passes: int, maxiter_per_point: int, ftol: float, gtol: float, obj_grad_func: Callable, args_for_optimizer: Tuple
) -> np.ndarray:
    """Refines points by optimizing them one by one."""
    current_points = points.copy()

    stuck_points_counter = np.zeros(N, dtype=int)
    max_stuck_tolerance = 5

    for pass_idx in range(num_passes):
        indices = np.arange(N)
        np.random.shuffle(indices)

        pass_improved = False

        for i in indices:
            if stuck_points_counter[i] >= max_stuck_tolerance:
                if random.random() < 0.1:
                    # Perturb the stuck point to try to free it
                    random_vec = np.random.randn(d + 1)
                    norm_rand = np.linalg.norm(random_vec)
                    current_points[i] = random_vec / np.where(norm_rand < 1e-9, 1.0, norm_rand)
                    stuck_points_counter[i] = 0
                continue

            # Create a temporary full point set, with point `i` being the variable
            # This is slow, but avoids re-writing the full objective for 1 point.
            # The current objective expects N points as input, so we must assemble the full set.
            def local_objective_and_grad_for_point_i(flat_point_i: np.ndarray) -> Tuple[float, np.ndarray]:
                temp_points = current_points.copy()
                point_i_candidate = flat_point_i.reshape(d + 1)
                temp_points[i] = point_i_candidate

                # The full objective function `obj_grad_func` expects its N parameter
                # to correspond to the number of points in its flattened input.
                # If antipodal, N_total is passed. If not, N_opt (N_total for standard).
                # The crucial part is that obj_grad_func takes N_total for antipodal.

                # Check if this sub-optimization is for antipodal strategy or standard
                if len(args_for_optimizer) == 3 and args_for_optimizer[2] == N: # Standard case: N is N_opt
                    error, grad = obj_grad_func(temp_points.flatten(), d, t, N)
                elif len(args_for_optimizer) == 3 and args_for_optimizer[2] == N * 2: # Antipodal case: N is N_opt, original N_total=N*2
                    error, grad = obj_grad_func(temp_points.flatten(), d, t, N * 2) # Pass N_total
                else: # Fallback or error
                    raise ValueError("Unexpected args_for_optimizer format in sequential refinement.")

                # The gradient is for all N points. We only need the gradient for point i.
                return error, grad.reshape(N, d + 1)[i].flatten()

            initial_local_error, _ = local_objective_and_grad_for_point_i(current_points[i].flatten())

            opt_result_i = optimize.minimize(
                local_objective_and_grad_for_point_i,
                current_points[i].flatten(),
                method='L-BFGS-B',
                jac=True,
                options={
                    'maxiter': maxiter_per_point,
                    'ftol': ftol,
                    'gtol': gtol,
                    'disp': False
                },
            )

            updated_point_i = opt_result_i.x.reshape(d + 1)
            norm_updated_i = np.linalg.norm(updated_point_i)
            if norm_updated_i > 1e-9:
                current_points[i] = updated_point_i / norm_updated_i
            else:
                random_vec = np.random.randn(d + 1)
                norm_rand = np.linalg.norm(random_vec)
                current_points[i] = random_vec / np.where(norm_rand < 1e-9, 1.0, norm_rand)

            final_local_error, _ = local_objective_and_grad_for_point_i(current_points[i].flatten())

            if final_local_error < initial_local_error - ftol:
                stuck_points_counter[i] = 0
                pass_improved = True
            else:
                stuck_points_counter[i] += 1

        if not pass_improved and pass_idx > 0:
            break

    return current_points


def search_for_best_spherical_design(d: int, t: int, N: int) -> np.ndarray:
  """Finds the best spherical t-design using gradient-based optimization."""
  global global_eval_count
  global global_current_t_for_gradient_focus # Declare global variable here
  global_eval_count = 0
  start_time = time.time()

  # Initialize gradient focus to k=1. This means initially, the gradient
  # will only consider the contribution from k=1 Gegenbauer polynomial.
  # This corresponds to minimizing the L2 norm of the center of mass of the points.
  global_current_t_for_gradient_focus = 1

  # --- Strategy Selection & Initialization ---
  use_antipodal_strategy = (N % 2 == 0 and N > 0)
  variable_name = f'spherical_design_{d}{t}{N}'
  initial_points_source_info = "unspecified" # Will be set below

  # Determine strategy based on N and set up objective/gradient function
  use_antipodal_strategy = (N % 2 == 0 and N > 0)
  if use_antipodal_strategy:
    print("--- Using antipodal strategy for even N ---")
    N_opt = N // 2
    obj_grad_func = _objective_and_grad_antipodal
    args_for_optimizer = (d, t, N) # Pass total N to the antipodal objective
  else:
    print("--- Using standard strategy ---")
    N_opt = N
    obj_grad_func = _objective_and_grad
    args_for_optimizer = (d, t, N) # Pass total N to the standard objective

  # Initialize points
  variable_name = f'spherical_design_{d}{t}{N}'
  initial_points_source_info = "unspecified"
  initial_opt_points = None

  if variable_name in globals():
    loaded_full_points = np.array(globals()[variable_name], dtype=np.float64)
    # Ensure loaded points are normalized before potentially slicing
    norms_full = np.linalg.norm(loaded_full_points, axis=1, keepdims=True)
    loaded_full_points /= np.where(norms_full < 1e-9, 1.0, norms_full)

    if use_antipodal_strategy:
      initial_opt_points = loaded_full_points[:N_opt, :].copy()
      initial_points_source_info = f"loaded {variable_name} (used first N/2 points)"
    else:
      initial_opt_points = loaded_full_points.copy()
      initial_points_source_info = f"loaded {variable_name}"

  if initial_opt_points is None: # If points were not loaded from globals
    if d == 2 and not use_antipodal_strategy: # Fibonacci spiral for S^2 (when not antipodal, to use full N_opt)
      initial_opt_points = np.zeros((N_opt, d + 1))
      golden_ratio = (1 + np.sqrt(5)) / 2
      for i in range(N_opt):
          phi = 2 * np.pi * i / golden_ratio
          z_coord = 1 - (2 * i + 1) / N_opt
          r_coord = np.sqrt(max(0, 1 - z_coord*z_coord))
          initial_opt_points[i, 0] = r_coord * np.cos(phi)
          initial_opt_points[i, 1] = r_coord * np.sin(phi)
          initial_opt_points[i, 2] = z_coord
      initial_points_source_info = "Fibonacci spiral (d=2)"
    else: # Use Recursive Zonal Construction (RZC) as a structured initialisation
      initial_points_source_info = "Recursive Zonal Construction (RZC)"
      # RZC function already handles d=0, d=1 base cases and general d >= 2 recursion.
      # It takes d_target, N_target, t_design_degree
      initial_opt_points = _rzc_construction(d, N_opt, t)

      # It's good practice to run a brief Coulomb pre-optimization even on RZC points
      # to ensure good local spacing before optimizing for the design criterion.
      if N_opt > 0: # Only run if there are points to optimize
          coulomb_pre_opt_result = optimize.minimize(
              _coulomb_objective_and_grad,
              initial_opt_points.flatten(),
              args=(d, N_opt, float(d)), # alpha=d, N_opt here
              method='L-BFGS-B',
              jac=True,
              options={'maxiter': 100, 'ftol': 1e-5, 'gtol': 1e-4, 'disp': False}, # Reduced maxiter for pre-opt
          )
          initial_opt_points = coulomb_pre_opt_result.x.reshape(N_opt, d + 1)

  # Normalize initial_opt_points (which has N_opt points)
  # This normalization is crucial before first score calculation and optimization
  norms = np.linalg.norm(initial_opt_points, axis=1, keepdims=True)
  initial_opt_points /= np.where(norms < 1e-9, 1.0, norms)

  best_opt_config_points = initial_opt_points.copy() # Stores N_opt points

  # Calculate initial error using the chosen objective function
  # Note: obj_grad_func expects flattened points corresponding to N_opt
  best_error, _ = obj_grad_func(best_opt_config_points.flatten(), *args_for_optimizer)

  if np.isinf(best_error) or np.isnan(best_error):
      print(f"Warning: Initial points from {initial_points_source_info} led to Inf/NaN error. Falling back to new random.")
      fallback_points = np.random.randn(N_opt, d + 1)
      norms_fallback = np.linalg.norm(fallback_points, axis=1, keepdims=True)
      fallback_points /= np.where(norms_fallback < 1e-9, 1.0, norms_fallback)
      best_opt_config_points = fallback_points.copy()
      best_error, _ = obj_grad_func(best_opt_config_points.flatten(), *args_for_optimizer)
      initial_points_source_info = "random fallback"
  print(f"Initial score: {-best_error:.5f} (from {initial_points_source_info})")

  # --- Initial Sequential Refinement (Polishing) ---
  MAXITER_PER_POINT_SEQ = 20 # Max iterations for L-BFGS-B per point during sequential refinement
  NUM_PASSES_SEQ_INITIAL = 3 # Number of full passes over all points for initial polish

  print(f"Starting initial sequential refinement (passes={NUM_PASSES_SEQ_INITIAL}, maxiter_per_point={MAXITER_PER_POINT_SEQ})...")
  best_opt_config_points = _refine_points_sequentially(
      best_opt_config_points, d, t, N_opt,
      num_passes=NUM_PASSES_SEQ_INITIAL, maxiter_per_point=MAXITER_PER_POINT_SEQ,
      ftol=1e-10, gtol=1e-9, # Use stricter tolerances for initial polish
      obj_grad_func=obj_grad_func, args_for_optimizer=args_for_optimizer
  )
  # Update best_error after initial sequential refinement
  best_error, _ = obj_grad_func(best_opt_config_points.flatten(), *args_for_optimizer)
  print(f"After initial sequential refinement, score: {-best_error:.5f}")

  # --- Multi-start optimization with annealing and diverse exploration ---
  num_restarts = 0
  time_limit = 195  # Leave a 5s buffer

  # --- Parameters for restart strategy ---
  # Probabilities for different restart types at the start and end of the search
  probs_start = {
      "random": 0.06, "partial_reset": 0.10, "coulomb_jiggle": 0.07,
      "gradient_kick": 0.10, "jolt": 0.05, "rotation": 0.08, "fine_tune": 0.07,
      "tangent_eigen_perturbation": 0.08, "collective_gradient_nudge": 0.07,
      "gradient_vortex": 0.07, "energy_peak_migration": 0.07, "adaptive_tangent_scramble": 0.07,
      "sequential_refinement": 0.08 # Added sequential refinement
  }
  probs_final = {
      "random": 0.01, "partial_reset": 0.02, "coulomb_jiggle": 0.02,
      "gradient_kick": 0.10, "jolt": 0.02, "rotation": 0.08, "fine_tune": 0.25,
      "tangent_eigen_perturbation": 0.10, "collective_gradient_nudge": 0.10,
      "gradient_vortex": 0.05, "energy_peak_migration": 0.05, "adaptive_tangent_scramble": 0.05,
      "sequential_refinement": 0.15 # Sequential refinement emphasized more at the end
  }

  # --- Parameters for perturbation types ---
  partial_reset_fraction = 0.4
  jolt_fraction = 0.1
  gradient_kick_fraction = 0.25
  tangent_eigen_perturb_fraction = 0.15 # Fraction of points to perturb with eigen_perturbation
  adaptive_scramble_fraction = 0.2 # Fraction of points for adaptive_tangent_scramble

  # --- Annealing parameters ---
  annealing_duration_ratio = 0.9
  # Strengths for perturbations, will be annealed
  kick_strength_initial = 0.2
  kick_strength_final = 0.01
  coulomb_strength_initial = 0.05
  coulomb_strength_final = 0.001
  noise_strength_initial = 0.02
  noise_strength_final = 0.0005
  eigen_perturb_strength_initial = 0.08 # For tangent_eigen_perturbation
  eigen_perturb_strength_final = 0.005
  nudge_strength_initial = 0.05 # For collective_gradient_nudge
  nudge_strength_final = 0.002
  scramble_strength_initial = 0.1 # For adaptive_tangent_scramble
  scramble_strength_final = 0.005

  # --- Parameters for L-BFGS-B optimizer ---
  maxiter_map = {
      "initial": 1200, "random": 500, "partial_reset": 700, "coulomb_jiggle": 600,
      "gradient_kick": 800, "jolt": 600, "rotation": 800, "gradient_vortex": 900,
      "energy_peak_migration": 850, "fine_tune": 1000,
      "tangent_eigen_perturbation": 800, "collective_gradient_nudge": 750,
      "adaptive_tangent_scramble": 800,
      "sequential_refinement": 7 # Number of passes for sequential refinement during main loop
  }
  # Define the maxiter_per_point for sequential refinement runs
  MAXITER_PER_POINT_SEQ_GENERAL = 15 # Iterations per point for sequential refinement during general runs
  MAXITER_PER_POINT_SEQ_FINAL = 30 # Iterations per point for final polish
  NUM_PASSES_SEQ_FINAL = 5 # Number of passes for final polish

  initial_ftol = 1e-7
  final_ftol = 1e-12
  initial_gtol = 1e-6
  final_gtol = 1e-10

  while time.time() - start_time < time_limit:
    num_restarts += 1

    elapsed_time = time.time() - start_time
    # progress will go from 0 to 1 over the annealing duration
    progress = min(1.0, elapsed_time / (time_limit * annealing_duration_ratio + 1e-9))
    # Ensure progress doesn't exceed 1.0, even if annealing_duration_ratio is less than 1
    progress = np.clip(progress, 0.0, 1.0)

    # --- Anneal parameters ---
    current_ftol = initial_ftol * (1 - progress) + final_ftol * progress
    current_gtol = initial_gtol * (1 - progress) + final_gtol * progress
    current_kick_strength = kick_strength_initial * (1-progress) + kick_strength_final * progress
    current_coulomb_strength = coulomb_strength_initial * (1-progress) + coulomb_strength_final * progress
    current_noise_strength = noise_strength_initial * (1-progress) + noise_strength_final * progress
    current_eigen_perturb_strength = eigen_perturb_strength_initial * (1-progress) + eigen_perturb_strength_final * progress
    current_nudge_strength = nudge_strength_initial * (1-progress) + nudge_strength_final * progress
    current_scramble_strength = scramble_strength_initial * (1-progress) + scramble_strength_final * progress

    # --- Anneal the gradient focus (global_current_t_for_gradient_focus) ---
    # This implements the curriculum learning strategy. As time progresses (progress goes from 0 to 1),
    # the maximum degree 'k' whose gradient contribution is considered gradually increases from 1 to 't'.
    # This means the optimization first focuses on simpler, lower-degree polynomial properties,
    # and then progressively incorporates higher-degree complexities.
    global_current_t_for_gradient_focus = max(1, min(t, int(t * progress) + 1))

    # Anneal probabilities and normalize
    current_probs = {k: probs_start[k] * (1 - progress) + probs_final[k] * progress for k in probs_start}
    total_prob = sum(current_probs.values())
    current_probs = {k: v / total_prob for k, v in current_probs.items()}

    # --- Select restart strategy ---
    current_start_points = best_opt_config_points.copy()
    if num_restarts == 1: # The very first main loop run after initial polish
        run_type = "initial" # This will run a full L-BFGS-B on the polished initial config
    else:
        run_type = np.random.choice(list(current_probs.keys()), p=list(current_probs.values()))

    # Determine maxiter for the L-BFGS-B step, or num_passes for sequential refinement
    current_lbfgs_maxiter_for_run = maxiter_map[run_type]
    current_seq_refine_num_passes = maxiter_map[run_type] if run_type == "sequential_refinement" else 0

    skip_lbfgs_minimize = False # Flag to skip L-BFGS-B if sequential refinement handles optimization

    # --- Generate starting points based on strategy ---
    if run_type == "initial":
        run_type_info = f"initial refine (maxiter={current_lbfgs_maxiter_for_run})"
        # For initial run, use fine-tune tolerances
        current_ftol, current_gtol = final_ftol, final_gtol
    elif run_type == "random":
        current_start_points = np.random.randn(N_opt, d + 1)
        run_type_info = f"random start ({N_opt} pts)"
    elif run_type == "sequential_refinement":
        run_type_info = f"sequential refinement (passes={current_seq_refine_num_passes}, maxiter_per_point={MAXITER_PER_POINT_SEQ})"
        # Sequential refinement itself is an optimization process; L-BFGS-B will be skipped
        skip_lbfgs_minimize = True
        current_start_points = _refine_points_sequentially(
            current_start_points, d, t, N_opt,
            num_passes=current_seq_refine_num_passes,
            maxiter_per_point=MAXITER_PER_POINT_SEQ,
            ftol=current_ftol, gtol=current_gtol,
            obj_grad_func=obj_grad_func, args_for_optimizer=args_for_optimizer
        )
    elif run_type == "partial_reset":
        num_to_reset = max(1, int(N_opt * partial_reset_fraction))
        indices = np.random.choice(N_opt, num_to_reset, replace=False)
        current_start_points[indices] = np.random.randn(num_to_reset, d + 1)
        run_type_info = f"partial reset ({num_to_reset}/{N_opt} pts)"
    elif run_type == "coulomb_jiggle":
        _, grad_coulomb_flat = _coulomb_objective_and_grad(current_start_points.flatten(), d, N_opt, alpha=float(d))
        grad_coulomb = grad_coulomb_flat.reshape(N_opt, d + 1)
        current_start_points -= current_coulomb_strength * grad_coulomb
        run_type_info = f"coulomb jiggle (str={current_coulomb_strength:.4f})"
    elif run_type == "gradient_kick":
        _, grad_design_flat = obj_grad_func(current_start_points.flatten(), *args_for_optimizer)
        grad_design = grad_design_flat.reshape(N_opt, d + 1)
        grad_norms = np.linalg.norm(grad_design, axis=1)
        num_to_kick = max(1, int(N_opt * gradient_kick_fraction))
        indices_to_kick = np.argsort(grad_norms)[-num_to_kick:]
        current_start_points[indices_to_kick] -= current_kick_strength * grad_design[indices_to_kick]
        run_type_info = f"gradient kick (str={current_kick_strength:.4f})"
    elif run_type == "jolt":
        num_to_jolt = max(1, int(N_opt * jolt_fraction))
        indices = np.random.choice(N_opt, num_to_jolt, replace=False)
        current_start_points[indices] = np.random.randn(num_to_jolt, d + 1)
        run_type_info = f"jolt start ({num_to_jolt}/{N_opt} pts)"
    elif run_type == "rotation":
        R = scipy.stats.ortho_group.rvs(d + 1)
        current_start_points = (R @ current_start_points.T).T
        run_type_info = "random rotation"
    elif run_type == "gradient_vortex":
        # Calculate current gradient to inform the vortex direction
        _, current_grad_flat = obj_grad_func(current_start_points.flatten(), *args_for_optimizer)
        current_grad = current_grad_flat.reshape(N_opt, d + 1)

        # Normalize the gradient vectors for direction
        grad_norms = np.linalg.norm(current_grad, axis=1, keepdims=True)
        # Avoid division by zero for zero gradients
        normalized_grad = np.where(grad_norms < 1e-9, 0.0, current_grad / grad_norms)

        # Generate a random vector in ambient space
        random_ambient_vec = np.random.randn(N_opt, d + 1)

        # Project random_ambient_vec onto the tangent space of each point
        # A tangent vector v_t for point p satisfies p . v_t = 0
        # Use current_start_points for tangent projection as these are the points being perturbed
        # Ensure current_start_points are normalized for correct tangent projection
        current_start_points_norms = np.linalg.norm(current_start_points, axis=1, keepdims=True)
        current_start_points_normalized_for_proj = current_start_points / np.where(current_start_points_norms < 1e-9, 1.0, current_start_points_norms)
        tangent_random_vec = random_ambient_vec - np.sum(random_ambient_vec * current_start_points_normalized_for_proj, axis=1, keepdims=True) * current_start_points_normalized_for_proj

        # If the gradient is non-zero, make the random component orthogonal to the gradient too (if d+1 > 2)
        # This gives a component that's truly "orthogonal" to the 2D plane defined by point and gradient,
        # providing a "swirling" motion.
        if d + 1 > 2: # This check ensures there's enough dimension to be orthogonal to two vectors
            # Project tangent_random_vec orthogonal to normalized_grad within the tangent plane
            # This requires normalized_grad to be non-zero and distinct from point direction.
            # Handle cases where normalized_grad is zero.
            dot_product_grad_tangent = np.sum(tangent_random_vec * normalized_grad, axis=1, keepdims=True)
            tangent_random_vec = np.where(grad_norms < 1e-9, tangent_random_vec, tangent_random_vec - dot_product_grad_tangent * normalized_grad)

        # Re-normalize random_tangent to get unit directions for the vortex component
        tangent_random_norms = np.linalg.norm(tangent_random_vec, axis=1, keepdims=True)
        tangent_random_normalized = np.where(tangent_random_norms < 1e-9, 0.0, tangent_random_vec / tangent_random_norms)

        # Combine gradient-aligned push and tangential vortex
        # The 'push' component moves points along their current negative gradient (scaled by a random factor for variety)
        # The 'vortex' component swirls them tangentially (scaled by a random factor for variety)
        push_component = -normalized_grad * current_kick_strength * np.random.rand(N_opt, 1) # Random magnitude for push
        vortex_component = tangent_random_normalized * current_kick_strength * np.random.rand(N_opt, 1) # Random magnitude for vortex

        current_start_points += (push_component + vortex_component)
        run_type_info = f"gradient vortex (str={current_kick_strength:.4f})"

    elif run_type == "energy_peak_migration":
        # Calculate current gradient
        _, current_grad_flat = obj_grad_func(current_start_points.flatten(), *args_for_optimizer)
        current_grad = current_grad_flat.reshape(N_opt, d + 1)

        # Perturb a subset of points (e.g., 20%) along the *positive* gradient (uphill)
        num_to_migrate = max(1, int(N_opt * 0.2))

        # Select points with highest current gradient magnitude for migration, as these are "active"
        grad_magnitudes = np.linalg.norm(current_grad, axis=1)
        # Ensure there are enough points with non-zero gradients, otherwise just pick randomly
        if np.sum(grad_magnitudes) > 1e-9:
            indices_to_migrate = np.argsort(grad_magnitudes)[-num_to_migrate:] # Points with largest gradients
        else:
            indices_to_migrate = np.random.choice(N_opt, num_to_migrate, replace=False)

        # Apply an uphill kick to selected points, tangential to the sphere
        positive_grad_kick = current_grad[indices_to_migrate] # Move along positive gradient

        # Project this kick onto the tangent space of the point
        current_points_to_migrate = current_start_points[indices_to_migrate]
        tangent_positive_grad_kick = positive_grad_kick - np.sum(
            positive_grad_kick * current_points_to_migrate, axis=1, keepdims=True
        ) * current_points_to_migrate

        current_start_points[indices_to_migrate] += tangent_positive_grad_kick * current_kick_strength * 1.5 # Larger kick
        run_type_info = f"energy peak migration (str={current_kick_strength*1.5:.4f})"

    elif run_type == "fine_tune":
        noise_raw = np.random.randn(N_opt, d + 1)
        # Use current_start_points for tangent projection as these are the points being perturbed
        # Ensure current_start_points are normalized for correct tangent projection
        current_start_points_norms = np.linalg.norm(current_start_points, axis=1, keepdims=True)
        current_start_points_normalized_for_proj = current_start_points / np.where(current_start_points_norms < 1e-9, 1.0, current_start_points_norms)
        tangent_noise = noise_raw - np.sum(noise_raw * current_start_points_normalized_for_proj, axis=1, keepdims=True) * current_start_points_normalized_for_proj
        current_start_points += tangent_noise * current_noise_strength
        run_type_info = f"fine-tune w/ tangent noise (str={current_noise_strength:.5f})"

    elif run_type == "tangent_eigen_perturbation":
        # Select a fraction of points to perturb
        num_to_perturb = max(1, int(N_opt * tangent_eigen_perturb_fraction))
        indices_to_perturb = np.random.choice(N_opt, num_to_perturb, replace=False)

        for idx in indices_to_perturb:
            p = current_start_points[idx]
            # Generate a random vector in ambient space
            v_rand = np.random.randn(d + 1)
            # Project v_rand onto the tangent space of p
            v_tangent = v_rand - np.dot(v_rand, p) * p
            v_tangent_norm = np.linalg.norm(v_tangent)
            if v_tangent_norm > 1e-9:
                v_tangent_normalized = v_tangent / v_tangent_norm
                current_start_points[idx] += current_eigen_perturb_strength * v_tangent_normalized
        run_type_info = f"tangent eigen perturbation (str={current_eigen_perturb_strength:.5f})"

    elif run_type == "collective_gradient_nudge":
        # Get the current gradient for all points
        _, current_grad_flat = obj_grad_func(current_start_points.flatten(), *args_for_optimizer)
        current_grad = current_grad_flat.reshape(N_opt, d + 1)

        # Project each individual gradient onto its tangent space
        # (This is already done in _objective_and_grad, but we need it here for the mean)
        points_normalized_for_grad = current_start_points / np.linalg.norm(current_start_points, axis=1, keepdims=True)
        tangent_grads = current_grad - np.sum(
            current_grad * points_normalized_for_grad, axis=1, keepdims=True
        ) * points_normalized_for_grad

        # Compute the mean of these tangent-projected gradients
        avg_tangent_grad_direction = np.mean(tangent_grads, axis=0)

        # Normalize the overall direction, if not zero
        avg_tangent_grad_direction_norm = np.linalg.norm(avg_tangent_grad_direction)
        if avg_tangent_grad_direction_norm > 1e-9:
            avg_tangent_grad_normalized = avg_tangent_grad_direction / avg_tangent_grad_direction_norm

            # Apply this collective nudge to all points, tangent-projected for each point
            for i in range(N_opt):
                p_i = current_start_points[i]
                nudge_vector = avg_tangent_grad_normalized - np.dot(avg_tangent_grad_normalized, p_i) * p_i
                current_start_points[i] += current_nudge_strength * nudge_vector
        run_type_info = f"collective gradient nudge (str={current_nudge_strength:.5f})"

    elif run_type == "adaptive_tangent_scramble":
        # Calculate current gradient
        _, current_grad_flat = obj_grad_func(current_start_points.flatten(), *args_for_optimizer)
        current_grad = current_grad_flat.reshape(N_opt, d + 1)

        num_to_scramble = max(1, int(N_opt * adaptive_scramble_fraction))
        indices_to_scramble = np.random.choice(N_opt, num_to_scramble, replace=False)

        # Perturb points based on their gradient magnitude and scramble strength
        for idx in indices_to_scramble:
            p = current_start_points[idx]
            grad_p = current_grad[idx]

            # Generate random noise in ambient space
            noise_raw = np.random.randn(d + 1)

            # Project noise onto the tangent space of p
            tangent_noise = noise_raw - np.dot(noise_raw, p) * p
            tangent_noise_norm = np.linalg.norm(tangent_noise)
            if tangent_noise_norm > 1e-9:
                tangent_noise_normalized = tangent_noise / tangent_noise_norm

                # Scale perturbation by gradient magnitude and scramble strength
                perturb_magnitude = current_scramble_strength * np.linalg.norm(grad_p)
                # Cap the magnitude to prevent extreme jumps if gradient is huge
                perturb_magnitude = np.clip(perturb_magnitude, 0, current_scramble_strength * 10)

                current_start_points[idx] += tangent_noise_normalized * perturb_magnitude
        run_type_info = f"adaptive tangent scramble ({num_to_scramble}/{N_opt} pts, str={current_scramble_strength:.4f})"

    # --- Run Optimizer ---
    # Normalize current_start_points before running the main L-BFGS-B or checking its current error
    norms = np.linalg.norm(current_start_points, axis=1, keepdims=True)
    current_start_points /= np.where(norms < 1e-9, 1.0, norms)

    print(f"--- Starting opt run #{num_restarts} ({run_type_info}) ---")

    if skip_lbfgs_minimize:
        # If sequential refinement was performed, current_start_points already contains the "optimized" result
        optimized_error, _ = obj_grad_func(current_start_points.flatten(), *args_for_optimizer)
        optimized_opt_points = current_start_points
    else:
        # Otherwise, run the main L-BFGS-B optimization
        opt_result = optimize.minimize(
            obj_grad_func, current_start_points.flatten(), args=args_for_optimizer,
            method='L-BFGS-B', jac=True,
            options={'maxiter': current_lbfgs_maxiter_for_run, 'ftol': current_ftol, 'gtol': current_gtol, 'disp': False},
        )
        optimized_error = opt_result.fun
        optimized_opt_points = opt_result.x.reshape(N_opt, d + 1)

    if np.isnan(optimized_error) or np.isinf(optimized_error):
        print(f"[{time.time() - start_time:.1f}s] Run resulted in NaN/Inf error. Discarding.")
    elif optimized_error < best_error:
      best_error = optimized_error
      norms_opt = np.linalg.norm(optimized_opt_points, axis=1, keepdims=True)
      best_opt_config_points = optimized_opt_points / np.where(norms_opt < 1e-9, 1.0, norms_opt)
      print(f"[{time.time() - start_time:.1f}s] New best score: {-best_error:.5f}")
    else:
      print(f"[{time.time() - start_time:.1f}s] Run finished. Score did not improve from {-best_error:.5f} (this run: {-optimized_error:.5f}).")

  print(f'\nFinal score: {-best_error:.5f}')
  print(f'Total evaluations of objective/gradient: {global_eval_count}')

  # Construct final points based on strategy
  if use_antipodal_strategy:
    final_points_to_return = np.vstack((best_opt_config_points, -best_opt_config_points))
  else:
    final_points_to_return = best_opt_config_points

  # Final sequential refinement to polish the best found solution
  print(f"\n--- Starting final sequential refinement (passes={NUM_PASSES_SEQ_FINAL}, maxiter_per_point={MAXITER_PER_POINT_SEQ_FINAL}) ---")
  best_opt_config_points = _refine_points_sequentially(
      best_opt_config_points, d, t, N_opt,
      num_passes=NUM_PASSES_SEQ_FINAL, maxiter_per_point=MAXITER_PER_POINT_SEQ_FINAL,
      ftol=final_ftol, gtol=final_gtol, # Use the strictest tolerances for final polish
      obj_grad_func=obj_grad_func, args_for_optimizer=args_for_optimizer
  )
  final_error_after_polish, _ = obj_grad_func(best_opt_config_points.flatten(), *args_for_optimizer)
  print(f"Final score after polish: {-final_error_after_polish:.5f}")

  # Construct final points based on strategy
  if use_antipodal_strategy:
    final_points_to_return = np.vstack((best_opt_config_points, -best_opt_config_points))
  else:
    final_points_to_return = best_opt_config_points

  # Final normalization to be absolutely sure
  norms_final = np.linalg.norm(final_points_to_return, axis=1, keepdims=True)
  final_points_to_return /= np.where(norms_final < 1e-9, 1.0, norms_final)

  return final_points_to_return

